In [ ]:
import config
import os
import csv
import pandas as pd

from user_prompt import generate_user_prompt
from feedback import handle_feedback
from datetime import datetime
from LLM_config import llm_config  # Import the LLMTestConfig class
from rag_pipeline import graph
from rag_pipeline import config  # assuming `graph` is properly imported from wherever you define it
from iteration_execution import execute_code  # assuming `execute_code` is defined elsewhere
from iteration_execution import extract_parent_with_children, replace_function_by_index  # assuming defined elsewhere

# File to store results
csv_file = '../outputs/results_logs_medium.csv'
## Load existing CSV or create a new DataFrame
if os.path.exists(csv_file):
    results_logs = pd.read_csv(csv_file)
else:
    header = ['Run_ID', 'Iteration', 'Timestamp', 'model_name', 'reasoning_factor','context_type','Feedback', 'AI_Message', 'Updated_Coulomb_input']
    results_logs = pd.DataFrame(columns=header)

# Function to get the next run_id
def get_next_run_id(result_logs):
    if not results_logs.empty and "Run_ID" in results_logs.columns:
        return int(results_logs["Run_ID"].max()) + 1 if pd.notna(results_logs["Run_ID"].max()) else 1
    return 1

# Initialize variables
run_id = get_next_run_id(results_logs)  # Get next run_id
Iteration = {}
feedback = ""
iteration = 1
n = llm_config.max_iterations
last_output = ""
patience = llm_config.patience  # Initialize patience from LLM config

# Check if the CSV file exists, if not, create it and write the header
try:
    with open(csv_file, mode='r', newline='', encoding='utf-8') as file:
        pass  # If file exists, do nothing
except FileNotFoundError:
    with open(csv_file, mode='w', newline='', encoding='utf-8') as file:
        writer = csv.writer(file)
        writer.writerow(header)  # Write the header if the file does not exist

# Read Coulomb_input for the first time
with open("Coulomb_input.py", "r") as file:
    Coulomb_input = file.read()

while iteration <= n:  # Ensure it runs for max_iterations
    print(f"================================ Iteration {iteration} ================================\n")
    
    # Generate the user input message only for the first iteration
    input_message = generate_user_prompt(Coulomb_input, feedback)

    # Process LLM response with the graph pipeline
    for step in graph.stream(
        {"messages": [{"role": "user", "content": input_message}]},
        stream_mode="values",
        config=config,
    ):
        step["messages"][-1].pretty_print()
    text = step["messages"][-1].content

    functions_filled = {}

    function_names = [
        "compute_real_energies",
        "compute_fourier_energies",
        "compute_intra_energies",
        "compute_self_energies"
    ]

    for function_name in function_names:
        extracted_function = extract_parent_with_children(text, function_name)
        if extracted_function:
            functions_filled[function_name] = extracted_function

    if not functions_filled:
        feedback = "No Python code found. Please review the code more clearly and provide a Python code after changes."
    else:
        for function_name, function_code in functions_filled.items():
            Coulomb_input = replace_function_by_index(Coulomb_input, function_code, function_name)
    
        print("executing the code")
        result = execute_code(Coulomb_input)

        # Check if the output has repeated
        if llm_config.stop_on_repeat and last_output and result['stdout'] and last_output == result['stdout'][-1]:
            patience -= 1  # Reduce patience
            print(f"Output repeated. Remaining patience: {patience}")

            if patience <= 0:
                print("The output has repeated for several iterations. The loop has stopped due to patience reaching 0.")
                break  # Exit the loop
        else:
            patience = llm_config.patience  # Reset patience if output changes""

        # Handle feedback and determine next steps
        feedback, iteration, n, last_output = handle_feedback(result, iteration, n, Iteration, last_output)
    
    # Append data to DataFrame
    new_row = {
        'Run_ID': run_id,
        'Iteration': iteration,
        'Timestamp': datetime.now().strftime("%Y-%m-%d %H:%M:%S"),
        'model_name': llm_config.model_name,
        'reasoning_factor': llm_config.reasoning_factor,
        'context_type': llm_config.context_type,
        'Feedback': feedback,
        'AI_Message': text,
        'Updated_Coulomb_input': Coulomb_input
    }
    
    results_logs = pd.concat([results_logs, pd.DataFrame([new_row])], ignore_index=True)

    if feedback == "LLM did a great job! Exiting from the loop":
        break

results_logs.to_csv(csv_file, index=False)
print(f"Results saved to {csv_file}")


================================ Iteration 1 ================================

================================ Human Message =================================

import numpy as np
import pandas as pd
from scipy.special import erfc,erf
import math 

# defining all variables
atom_properties = {
    'O': {'type': 'O', 'sigma': 3.165558, 'epsilon': 78.197431, 'charge': -0.8476, 'num_particles': 1},
    'H': {'type': 'H', 'sigma': 0.000000, 'epsilon': 0.000000, 'charge': 0.4238, 'num_particles': 2},
}

# Trying for small configurations first
file_paths = [
#        'spce_sample_config_periodic4.txt',
#        'spce_sample_config_periodic2.txt',
#        'spce_sample_config_periodic3.txt',
        '../data/spce_sample_config_periodic1_modified.txt'
    ]

NIST_SPC_E_Water = {
        'Configuration': [1, 2, 3, 4],
        'M (number of SPC/E molecules)': [100, 200, 300, 750],
        'Lx=Ly=Lz (Å)': [20.0, 20.0, 20.0, 30.0],
        'Edisp/kB (K)': [9.95387E+04, 1.93712E+05, 3.54344E+05, 4.4

In [ ]:
import config
import os
import csv
import pandas as pd

from user_prompt import generate_user_prompt
from feedback import handle_feedback
from datetime import datetime
from LLM_config import llm_config  # Import the LLMTestConfig class
from rag_pipeline import graph
from rag_pipeline import config  # assuming `graph` is properly imported from wherever you define it
from iteration_execution import execute_code  # assuming `execute_code` is defined elsewhere
from iteration_execution import extract_parent_with_children, replace_function_by_index  # assuming defined elsewhere

# File to store results
csv_file = '../outputs/results_logs_medium.csv'
## Load existing CSV or create a new DataFrame
if os.path.exists(csv_file):
    results_logs = pd.read_csv(csv_file)
else:
    header = ['Run_ID', 'Iteration', 'Timestamp', 'model_name', 'reasoning_factor','context_type','Feedback', 'AI_Message', 'Updated_Coulomb_input']
    results_logs = pd.DataFrame(columns=header)

# Function to get the next run_id
def get_next_run_id(result_logs):
    if not results_logs.empty and "Run_ID" in results_logs.columns:
        return int(results_logs["Run_ID"].max()) + 1 if pd.notna(results_logs["Run_ID"].max()) else 1
    return 1

# Initialize variables
run_id = get_next_run_id(results_logs)  # Get next run_id
Iteration = {}
feedback = ""
iteration = 1
n = llm_config.max_iterations
last_output = ""
patience = llm_config.patience  # Initialize patience from LLM config

# Check if the CSV file exists, if not, create it and write the header
try:
    with open(csv_file, mode='r', newline='', encoding='utf-8') as file:
        pass  # If file exists, do nothing
except FileNotFoundError:
    with open(csv_file, mode='w', newline='', encoding='utf-8') as file:
        writer = csv.writer(file)
        writer.writerow(header)  # Write the header if the file does not exist

# Read Coulomb_input for the first time
with open("Coulomb_input.py", "r") as file:
    Coulomb_input = file.read()

while iteration <= n:  # Ensure it runs for max_iterations
    print(f"================================ Iteration {iteration} ================================\n")
    
    # Generate the user input message only for the first iteration
    input_message = generate_user_prompt(Coulomb_input, feedback)

    # Process LLM response with the graph pipeline
    for step in graph.stream(
        {"messages": [{"role": "user", "content": input_message}]},
        stream_mode="values",
        config=config,
    ):
        step["messages"][-1].pretty_print()
    text = step["messages"][-1].content

    functions_filled = {}

    function_names = [
        "compute_real_energies",
        "compute_fourier_energies",
        "compute_intra_energies",
        "compute_self_energies"
    ]

    for function_name in function_names:
        extracted_function = extract_parent_with_children(text, function_name)
        if extracted_function:
            functions_filled[function_name] = extracted_function

    if not functions_filled:
        feedback = "No Python code found. Please review the code more clearly and provide a Python code after changes."
    else:
        for function_name, function_code in functions_filled.items():
            Coulomb_input = replace_function_by_index(Coulomb_input, function_code, function_name)
    
        print("executing the code")
        result = execute_code(Coulomb_input)

        # Check if the output has repeated
        if llm_config.stop_on_repeat and last_output and result['stdout'] and last_output == result['stdout'][-1]:
            patience -= 1  # Reduce patience
            print(f"Output repeated. Remaining patience: {patience}")

            if patience <= 0:
                print("The output has repeated for several iterations. The loop has stopped due to patience reaching 0.")
                break  # Exit the loop
        else:
            patience = llm_config.patience  # Reset patience if output changes""

        # Handle feedback and determine next steps
        feedback, iteration, n, last_output = handle_feedback(result, iteration, n, Iteration, last_output)
    
    # Append data to DataFrame
    new_row = {
        'Run_ID': run_id,
        'Iteration': iteration,
        'Timestamp': datetime.now().strftime("%Y-%m-%d %H:%M:%S"),
        'model_name': llm_config.model_name,
        'reasoning_factor': llm_config.reasoning_factor,
        'context_type': llm_config.context_type,
        'Feedback': feedback,
        'AI_Message': text,
        'Updated_Coulomb_input': Coulomb_input
    }
    
    results_logs = pd.concat([results_logs, pd.DataFrame([new_row])], ignore_index=True)

    if feedback == "LLM did a great job! Exiting from the loop":
        break

results_logs.to_csv(csv_file, index=False)
print(f"Results saved to {csv_file}")


================================ Iteration 1 ================================

================================ Human Message =================================

import numpy as np
import pandas as pd
from scipy.special import erfc,erf
import math 

# defining all variables
atom_properties = {
    'O': {'type': 'O', 'sigma': 3.165558, 'epsilon': 78.197431, 'charge': -0.8476, 'num_particles': 1},
    'H': {'type': 'H', 'sigma': 0.000000, 'epsilon': 0.000000, 'charge': 0.4238, 'num_particles': 2},
}

# Trying for small configurations first
file_paths = [
#        'spce_sample_config_periodic4.txt',
#        'spce_sample_config_periodic2.txt',
#        'spce_sample_config_periodic3.txt',
        '../data/spce_sample_config_periodic1_modified.txt'
    ]

NIST_SPC_E_Water = {
        'Configuration': [1, 2, 3, 4],
        'M (number of SPC/E molecules)': [100, 200, 300, 750],
        'Lx=Ly=Lz (Å)': [20.0, 20.0, 20.0, 30.0],
        'Edisp/kB (K)': [9.95387E+04, 1.93712E+05, 3.54344E+05, 4.4

In [ ]:
import config
import os
import csv
import pandas as pd

from user_prompt import generate_user_prompt
from feedback import handle_feedback
from datetime import datetime
from LLM_config import llm_config  # Import the LLMTestConfig class
from rag_pipeline import graph
from rag_pipeline import config  # assuming `graph` is properly imported from wherever you define it
from iteration_execution import execute_code  # assuming `execute_code` is defined elsewhere
from iteration_execution import extract_parent_with_children, replace_function_by_index  # assuming defined elsewhere

# File to store results
csv_file = '../outputs/results_logs_medium.csv'
## Load existing CSV or create a new DataFrame
if os.path.exists(csv_file):
    results_logs = pd.read_csv(csv_file)
else:
    header = ['Run_ID', 'Iteration', 'Timestamp', 'model_name', 'reasoning_factor','context_type','Feedback', 'AI_Message', 'Updated_Coulomb_input']
    results_logs = pd.DataFrame(columns=header)

# Function to get the next run_id
def get_next_run_id(result_logs):
    if not results_logs.empty and "Run_ID" in results_logs.columns:
        return int(results_logs["Run_ID"].max()) + 1 if pd.notna(results_logs["Run_ID"].max()) else 1
    return 1

# Initialize variables
run_id = get_next_run_id(results_logs)  # Get next run_id
Iteration = {}
feedback = ""
iteration = 1
n = llm_config.max_iterations
last_output = ""
patience = llm_config.patience  # Initialize patience from LLM config

# Check if the CSV file exists, if not, create it and write the header
try:
    with open(csv_file, mode='r', newline='', encoding='utf-8') as file:
        pass  # If file exists, do nothing
except FileNotFoundError:
    with open(csv_file, mode='w', newline='', encoding='utf-8') as file:
        writer = csv.writer(file)
        writer.writerow(header)  # Write the header if the file does not exist

# Read Coulomb_input for the first time
with open("Coulomb_input.py", "r") as file:
    Coulomb_input = file.read()

while iteration <= n:  # Ensure it runs for max_iterations
    print(f"================================ Iteration {iteration} ================================\n")
    
    # Generate the user input message only for the first iteration
    input_message = generate_user_prompt(Coulomb_input, feedback)

    # Process LLM response with the graph pipeline
    for step in graph.stream(
        {"messages": [{"role": "user", "content": input_message}]},
        stream_mode="values",
        config=config,
    ):
        step["messages"][-1].pretty_print()
    text = step["messages"][-1].content

    functions_filled = {}

    function_names = [
        "compute_real_energies",
        "compute_fourier_energies",
        "compute_intra_energies",
        "compute_self_energies"
    ]

    for function_name in function_names:
        extracted_function = extract_parent_with_children(text, function_name)
        if extracted_function:
            functions_filled[function_name] = extracted_function

    if not functions_filled:
        feedback = "No Python code found. Please review the code more clearly and provide a Python code after changes."
    else:
        for function_name, function_code in functions_filled.items():
            Coulomb_input = replace_function_by_index(Coulomb_input, function_code, function_name)
    
        print("executing the code")
        result = execute_code(Coulomb_input)

        print("patience updation")
        # Check if the output has repeated
        if llm_config.stop_on_repeat and last_output and result['stdout'] and last_output == result['stdout'][-1]:
            patience -= 1  # Reduce patience
            print(f"Output repeated. Remaining patience: {patience}")

            if patience <= 0:
                print("The output has repeated for several iterations. The loop has stopped due to patience reaching 0.")
                break  # Exit the loop
        else:
            patience = llm_config.patience  # Reset patience if output changes""

        print("feedback updation")
        # Handle feedback and determine next steps
        feedback, iteration, n, last_output = handle_feedback(result, iteration, n, Iteration, last_output)
    
    # Append data to DataFrame
    new_row = {
        'Run_ID': run_id,
        'Iteration': iteration,
        'Timestamp': datetime.now().strftime("%Y-%m-%d %H:%M:%S"),
        'model_name': llm_config.model_name,
        'reasoning_factor': llm_config.reasoning_factor,
        'context_type': llm_config.context_type,
        'Feedback': feedback,
        'AI_Message': text,
        'Updated_Coulomb_input': Coulomb_input
    }
    
    print("storing results")  
    results_logs = pd.concat([results_logs, pd.DataFrame([new_row])], ignore_index=True)

    if feedback == "LLM did a great job! Exiting from the loop":
        break

results_logs.to_csv(csv_file, index=False)
print(f"Results saved to {csv_file}")


================================ Iteration 1 ================================

================================ Human Message =================================

import numpy as np
import pandas as pd
from scipy.special import erfc,erf
import math 

# defining all variables
atom_properties = {
    'O': {'type': 'O', 'sigma': 3.165558, 'epsilon': 78.197431, 'charge': -0.8476, 'num_particles': 1},
    'H': {'type': 'H', 'sigma': 0.000000, 'epsilon': 0.000000, 'charge': 0.4238, 'num_particles': 2},
}

# Trying for small configurations first
file_paths = [
#        'spce_sample_config_periodic4.txt',
#        'spce_sample_config_periodic2.txt',
#        'spce_sample_config_periodic3.txt',
        '../data/spce_sample_config_periodic1_modified.txt'
    ]

NIST_SPC_E_Water = {
        'Configuration': [1, 2, 3, 4],
        'M (number of SPC/E molecules)': [100, 200, 300, 750],
        'Lx=Ly=Lz (Å)': [20.0, 20.0, 20.0, 30.0],
        'Edisp/kB (K)': [9.95387E+04, 1.93712E+05, 3.54344E+05, 4.4

In [ ]:
import config
import os
import csv
import pandas as pd

from user_prompt import generate_user_prompt
from feedback import handle_feedback
from datetime import datetime
from LLM_config import llm_config  # Import the LLMTestConfig class
from rag_pipeline import graph
from rag_pipeline import config  # assuming `graph` is properly imported from wherever you define it
from iteration_execution import execute_code  # assuming `execute_code` is defined elsewhere
from iteration_execution import extract_parent_with_children, replace_function_by_index  # assuming defined elsewhere

# File to store results
csv_file = '../outputs/results_logs_medium.csv'
## Load existing CSV or create a new DataFrame
if os.path.exists(csv_file):
    results_logs = pd.read_csv(csv_file)
else:
    header = ['Run_ID', 'Iteration', 'Timestamp', 'model_name', 'reasoning_factor','context_type','Feedback', 'AI_Message', 'Updated_Coulomb_input']
    results_logs = pd.DataFrame(columns=header)

# Function to get the next run_id
def get_next_run_id(result_logs):
    if not results_logs.empty and "Run_ID" in results_logs.columns:
        return int(results_logs["Run_ID"].max()) + 1 if pd.notna(results_logs["Run_ID"].max()) else 1
    return 1

# Initialize variables
run_id = get_next_run_id(results_logs)  # Get next run_id
Iteration = {}
feedback = ""
iteration = 1
n = llm_config.max_iterations
last_output = ""
patience = llm_config.patience  # Initialize patience from LLM config

# Check if the CSV file exists, if not, create it and write the header
try:
    with open(csv_file, mode='r', newline='', encoding='utf-8') as file:
        pass  # If file exists, do nothing
except FileNotFoundError:
    with open(csv_file, mode='w', newline='', encoding='utf-8') as file:
        writer = csv.writer(file)
        writer.writerow(header)  # Write the header if the file does not exist

# Read Coulomb_input for the first time
with open("Coulomb_input.py", "r") as file:
    Coulomb_input = file.read()

while iteration <= n:  # Ensure it runs for max_iterations
    print(f"================================ Iteration {iteration} ================================\n")
    
    # Generate the user input message only for the first iteration
    input_message = generate_user_prompt(Coulomb_input, feedback)

    # Process LLM response with the graph pipeline
    for step in graph.stream(
        {"messages": [{"role": "user", "content": input_message}]},
        stream_mode="values",
        config=config,
    ):
        step["messages"][-1].pretty_print()
    text = step["messages"][-1].content

    functions_filled = {}

    function_names = [
        "compute_real_energies",
        "compute_fourier_energies",
        "compute_intra_energies",
        "compute_self_energies"
    ]

    for function_name in function_names:
        extracted_function = extract_parent_with_children(text, function_name)
        if extracted_function:
            functions_filled[function_name] = extracted_function

    if not functions_filled:
        feedback = "No Python code found. Please review the code more clearly and provide a Python code after changes."
    else:
        for function_name, function_code in functions_filled.items():
            Coulomb_input = replace_function_by_index(Coulomb_input, function_code, function_name)
    
        print("executing the code")
        result = execute_code(Coulomb_input)

        print("patience updation")
        # Check if the output has repeated
        if llm_config.stop_on_repeat and last_output and result['stdout'] and last_output == result['stdout'][-1]:
            patience -= 1  # Reduce patience
            print(f"Output repeated. Remaining patience: {patience}")

            if patience <= 0:
                print("The output has repeated for several iterations. The loop has stopped due to patience reaching 0.")
                break  # Exit the loop
        else:
            patience = llm_config.patience  # Reset patience if output changes""

        print("feedback updation")
        # Handle feedback and determine next steps
        feedback, iteration, n, last_output = handle_feedback(result, iteration, n, Iteration, last_output)
    
    # Append data to DataFrame
    new_row = {
        'Run_ID': run_id,
        'Iteration': iteration,
        'Timestamp': datetime.now().strftime("%Y-%m-%d %H:%M:%S"),
        'model_name': llm_config.model_name,
        'reasoning_factor': llm_config.reasoning_factor,
        'context_type': llm_config.context_type,
        'Feedback': feedback,
        'AI_Message': text,
        'Updated_Coulomb_input': Coulomb_input
    }
    
    print("storing results")  
    results_logs = pd.concat([results_logs, pd.DataFrame([new_row])], ignore_index=True)

    if feedback == "LLM did a great job! Exiting from the loop":
        break

results_logs.to_csv(csv_file, index=False)
print(f"Results saved to {csv_file}")


================================ Iteration 1 ================================

================================ Human Message =================================

import numpy as np
import pandas as pd
from scipy.special import erfc,erf
import math 

# defining all variables
atom_properties = {
    'O': {'type': 'O', 'sigma': 3.165558, 'epsilon': 78.197431, 'charge': -0.8476, 'num_particles': 1},
    'H': {'type': 'H', 'sigma': 0.000000, 'epsilon': 0.000000, 'charge': 0.4238, 'num_particles': 2},
}

# Trying for small configurations first
file_paths = [
#        'spce_sample_config_periodic4.txt',
#        'spce_sample_config_periodic2.txt',
#        'spce_sample_config_periodic3.txt',
        '../data/spce_sample_config_periodic1_modified.txt'
    ]

NIST_SPC_E_Water = {
        'Configuration': [1, 2, 3, 4],
        'M (number of SPC/E molecules)': [100, 200, 300, 750],
        'Lx=Ly=Lz (Å)': [20.0, 20.0, 20.0, 30.0],
        'Edisp/kB (K)': [9.95387E+04, 1.93712E+05, 3.54344E+05, 4.4

In [ ]:
import config
import os
import csv
import pandas as pd

from user_prompt import generate_user_prompt
from feedback import handle_feedback
from datetime import datetime
from LLM_config import llm_config  # Import the LLMTestConfig class
from rag_pipeline import graph
from rag_pipeline import config  # assuming `graph` is properly imported from wherever you define it
from iteration_execution import execute_code  # assuming `execute_code` is defined elsewhere
from iteration_execution import extract_parent_with_children, replace_function_by_index  # assuming defined elsewhere

# File to store results
csv_file = '../outputs/results_logs_medium.csv'
## Load existing CSV or create a new DataFrame
if os.path.exists(csv_file):
    results_logs = pd.read_csv(csv_file)
else:
    header = ['Run_ID', 'Iteration', 'Timestamp', 'model_name', 'reasoning_factor','context_type','Feedback', 'AI_Message', 'Updated_Coulomb_input']
    results_logs = pd.DataFrame(columns=header)

# Function to get the next run_id
def get_next_run_id(result_logs):
    if not results_logs.empty and "Run_ID" in results_logs.columns:
        return int(results_logs["Run_ID"].max()) + 1 if pd.notna(results_logs["Run_ID"].max()) else 1
    return 1

# Initialize variables
run_id = get_next_run_id(results_logs)  # Get next run_id
Iteration = {}
feedback = ""
iteration = 1
n = llm_config.max_iterations
last_output = ""
patience = llm_config.patience  # Initialize patience from LLM config

# Check if the CSV file exists, if not, create it and write the header
try:
    with open(csv_file, mode='r', newline='', encoding='utf-8') as file:
        pass  # If file exists, do nothing
except FileNotFoundError:
    with open(csv_file, mode='w', newline='', encoding='utf-8') as file:
        writer = csv.writer(file)
        writer.writerow(header)  # Write the header if the file does not exist

# Read Coulomb_input for the first time
with open("Coulomb_input.py", "r") as file:
    Coulomb_input = file.read()

while iteration <= n:  # Ensure it runs for max_iterations
    print(f"================================ Iteration {iteration} ================================\n")
    
    # Generate the user input message only for the first iteration
    input_message = generate_user_prompt(Coulomb_input, feedback)

    # Process LLM response with the graph pipeline
    for step in graph.stream(
        {"messages": [{"role": "user", "content": input_message}]},
        stream_mode="values",
        config=config,
    ):
        step["messages"][-1].pretty_print()
    text = step["messages"][-1].content

    functions_filled = {}

    function_names = [
        "compute_real_energies",
        "compute_fourier_energies",
        "compute_intra_energies",
        "compute_self_energies"
    ]

    for function_name in function_names:
        extracted_function = extract_parent_with_children(text, function_name)
        if extracted_function:
            functions_filled[function_name] = extracted_function

    if not functions_filled:
        feedback = "No Python code found. Please review the code more clearly and provide a Python code after changes."
    else:
        for function_name, function_code in functions_filled.items():
            Coulomb_input = replace_function_by_index(Coulomb_input, function_code, function_name)
    
        print("executing the code")
        result = execute_code(Coulomb_input)

        print("patience updation")
        # Check if the output has repeated
        if llm_config.stop_on_repeat and last_output and result['stdout'] and last_output == result['stdout'][-1]:
            patience -= 1  # Reduce patience
            print(f"Output repeated. Remaining patience: {patience}")

            if patience <= 0:
                print("The output has repeated for several iterations. The loop has stopped due to patience reaching 0.")
                break  # Exit the loop
        else:
            patience = llm_config.patience  # Reset patience if output changes""

        print("feedback updation")
        # Handle feedback and determine next steps
        feedback, iteration, n, last_output = handle_feedback(result, iteration, n, Iteration, last_output)
    
    # Append data to DataFrame
    new_row = {
        'Run_ID': run_id,
        'Iteration': iteration,
        'Timestamp': datetime.now().strftime("%Y-%m-%d %H:%M:%S"),
        'model_name': llm_config.model_name,
        'reasoning_factor': llm_config.reasoning_factor,
        'context_type': llm_config.context_type,
        'Feedback': feedback,
        'AI_Message': text,
        'Updated_Coulomb_input': Coulomb_input
    }
    
    print("storing results")  
    results_logs = pd.concat([results_logs, pd.DataFrame([new_row])], ignore_index=True)

    if feedback == "LLM did a great job! Exiting from the loop":
        break

results_logs.to_csv(csv_file, index=False)
print(f"Results saved to {csv_file}")


================================ Iteration 1 ================================

================================ Human Message =================================

import numpy as np
import pandas as pd
from scipy.special import erfc,erf
import math 

# defining all variables
atom_properties = {
    'O': {'type': 'O', 'sigma': 3.165558, 'epsilon': 78.197431, 'charge': -0.8476, 'num_particles': 1},
    'H': {'type': 'H', 'sigma': 0.000000, 'epsilon': 0.000000, 'charge': 0.4238, 'num_particles': 2},
}

# Trying for small configurations first
file_paths = [
#        'spce_sample_config_periodic4.txt',
#        'spce_sample_config_periodic2.txt',
#        'spce_sample_config_periodic3.txt',
        '../data/spce_sample_config_periodic1_modified.txt'
    ]

NIST_SPC_E_Water = {
        'Configuration': [1, 2, 3, 4],
        'M (number of SPC/E molecules)': [100, 200, 300, 750],
        'Lx=Ly=Lz (Å)': [20.0, 20.0, 20.0, 30.0],
        'Edisp/kB (K)': [9.95387E+04, 1.93712E+05, 3.54344E+05, 4.4

Traceback (most recent call last):
  File "/Users/pulicharishma/anaconda3/lib/python3.11/site-packages/IPython/core/interactiveshell.py", line 3505, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "/var/folders/fg/2c9z7vpx4wsby7vtrdlf60zw0000gn/T/ipykernel_5904/3584077030.py", line 59, in <module>
    for step in graph.stream(
  File "/Users/pulicharishma/anaconda3/lib/python3.11/site-packages/langgraph/pregel/__init__.py", line 1724, in stream
    for _ in runner.tick(
  File "/Users/pulicharishma/anaconda3/lib/python3.11/site-packages/langgraph/pregel/runner.py", line 230, in tick
    run_with_retry(
  File "/Users/pulicharishma/anaconda3/lib/python3.11/site-packages/langgraph/pregel/retry.py", line 40, in run_with_retry
    return task.proc.invoke(task.input, config)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/Users/pulicharishma/anaconda3/lib/python3.11/site-packages/langgraph/utils/runnable.py", line 495, in invoke
    input = step.invoke(inpu

In [2]:
import numpy as np
import pandas as pd
from scipy.special import erfc,erf
import math 

# defining all variables
atom_properties = {
    'O': {'type': 'O', 'sigma': 3.165558, 'epsilon': 78.197431, 'charge': -0.8476, 'num_particles': 1},
    'H': {'type': 'H', 'sigma': 0.000000, 'epsilon': 0.000000, 'charge': 0.4238, 'num_particles': 2},
}

# Trying for small configurations first
file_paths = [
#        'spce_sample_config_periodic4.txt',
#        'spce_sample_config_periodic2.txt',
#        'spce_sample_config_periodic3.txt',
        '../data/spce_sample_config_periodic1_modified.txt'
    ]

NIST_SPC_E_Water = {
        'Configuration': [1, 2, 3, 4],
        'M (number of SPC/E molecules)': [100, 200, 300, 750],
        'Lx=Ly=Lz (Å)': [20.0, 20.0, 20.0, 30.0],
        'Edisp/kB (K)': [9.95387E+04, 1.93712E+05, 3.54344E+05, 4.48593E+05],
        'ELRC/kB (K)': [-8.23715E+02, -3.29486E+03, -7.41343E+03, -1.37286E+04],
        'Ereal/kB (K)': [-5.58889E+05, -1.19295E+06, -1.96297E+06, -3.57226E+06],
        'Efourier/kB (K)': [6.27009E+03, 6.03495E+03, 5.24461E+03, 7.58785E+03],
        'Eself/kB (K)': [-2.84469E+06, -5.68938E+06, -8.53407E+06, -1.42235E+07],
        'Eintra/kB (K)': [2.80999E+06, 5.61998E+06, 8.42998E+06, 1.41483E+07],
        'Etotal/kB (K)': [-4.88604E+05, -1.06590E+06, -1.71488E+06, -3.20501E+06]
    }

# Data processing
def extracting_positions(input_file):
        # Extract the positions from the .xyz file
    with open(input_file, "r") as file:
        lines = file.readlines()

    data_lines = lines[2:]

    data_list = []
    for line in data_lines:
        stripped_line = line.strip()
        parts = stripped_line.split()
        if len(parts) >= 5:  
            try:
                x, y, z = map(float, parts[1:4])
                atom_type = parts[4]
                data_list.append([x, y, z, atom_type])
            except ValueError:
                continue  

    # Create a DataFrame with all configurations
    columns = ["X", "Y", "Z", "Atom Type"]
    configuration = pd.DataFrame(data_list, columns=columns)

    configuration.index = range(1, len(configuration) + 1)

    configuration["Molecule"] = ((configuration.index - 1) // 3) + 1
    
    return configuration

# create the target dataframes
def creating_dataframes(file_paths, atom_properties,NIST_SPC_E_Water):
    
    # Create the NIST_SPC_E_Water dataframe
    NIST_SPC_E_Water = pd.DataFrame(NIST_SPC_E_Water)
    
    NIST_SPC_E_Water['Sum of energies'] = (NIST_SPC_E_Water['Edisp/kB (K)'] + NIST_SPC_E_Water['ELRC/kB (K)'] +
                             NIST_SPC_E_Water['Ereal/kB (K)'] + NIST_SPC_E_Water['Efourier/kB (K)'] +
                             NIST_SPC_E_Water['Eself/kB (K)'] + NIST_SPC_E_Water['Eintra/kB (K)'])

    #for col in NIST_SPC_E_Water.columns[3:]:  # Skip the first column (Configuration)
     #   NIST_SPC_E_Water[col] = NIST_SPC_E_Water[col].apply(lambda x: f"{x:.4E}")
        
    # Creating the force_field dataframe
    force_field = pd.DataFrame(atom_properties).from_dict(atom_properties, orient='index')

    # Create the system dataframe contaning some variables
    system = pd.DataFrame(file_paths, columns=["file_paths"])

    system['configuration #'] = system['file_paths'].str.extract(r'(\d+)').astype(int)

    system[["number of particles", "box length"]] = system["configuration #"].apply(
    lambda x: pd.Series({
        "number of particles": float(NIST_SPC_E_Water.loc[NIST_SPC_E_Water["Configuration"] == x, 
                                                          "M (number of SPC/E molecules)"].values[0]),
        "box length": float(NIST_SPC_E_Water.loc[NIST_SPC_E_Water["Configuration"] == x, 
                                                 "Lx=Ly=Lz (Å)"].values[0])}))

    system['cutoff'] = 10
    system['alpha'] = 5.6
    system['kmax'] = 5
    system['ε0'] = float(8.854187817E-12)
    system['kB'] = float(1.3806488E-23)
        
    return system, force_field, NIST_SPC_E_Water

def compute_real_energies(system_data, configuration, force_field):
    # Compute the real energy part of the Coulomb Ewald summation for the system.
    #
    # Parameters:
    # - system_data: DataFrame containing system-level information, including configuration, box length, and number of molecules.
    # - configuration: DataFrame containing atomic positions and atom types for the specific configuration.
    # - force_field: DataFrame containing the force field parameters (sigma, epsilon, charge, etc.) for the atoms.
    #
    # Returns:
    # - real_energy: Computed real energy part of the Coulomb Ewald summation (in K) as a float value.
    #
    # Description:
    # This function calculates the real energy contribution in the Coulomb Ewald summation,
    # which involves calculating the electrostatic interactions between atoms in the system
    # in real space. The real energy part represents the direct interactions between atoms
    # based on their positions and force field parameters.

    real_energy = 0

    #--- Complete this code ---#

    return real_energy

def compute_fourier_energies(system_data, configuration, force_field):
    # Compute the Fourier energy part of the Coulomb Ewald summation for the system.
    #
    # Parameters:
    # - system_data: DataFrame containing system-level information, including configuration, box length, and number of molecules.
    # - configuration: DataFrame containing atomic positions and atom types for the specific configuration.
    # - force_field: DataFrame containing the force field parameters (sigma, epsilon, charge, etc.) for the atoms.
    #
    # Returns:
    # - fourier_energy: Computed Fourier energy part of the Coulomb Ewald summation (in K) as a float value.
    #
    # Description:
    # This function calculates the Fourier energy contribution in the Coulomb Ewald summation,
    # which involves computing the reciprocal space interactions between atoms using
    # Fourier transforms. The Fourier energy part represents the interactions in reciprocal space
    # between the charges of the atoms.

    fourier_energy = 0

    #--- Complete this code ---#

    return fourier_energy


def compute_self_energies(system_data, configuration, force_field):
    # Compute the self-energy contribution for the system's atoms as part of the Coulomb Ewald summation.
    #
    # Parameters:
    # - system_data: DataFrame containing system-level information, including configuration, box length, and number of molecules.
    # - configuration: DataFrame containing atomic positions and atom types for the specific configuration.
    # - force_field: DataFrame containing the force field parameters (sigma, epsilon, charge, etc.) for the atoms.
    #
    # Returns:
    # - self_energy: Computed self-energy for each atom in the system (in K) as a float value.
    #
    # Description:
    # This function computes the self-energy term for each atom in the system, which represents
    # the interaction of an atom with itself as part of the Coulomb Ewald summation. The self-energy
    # term is typically related to the charge of the atom and its self-interaction in the electrostatic
    # calculations.

    self_energy = 0

    #--- Complete this code ---#

    return self_energy


def compute_intra_energies(system_data, configuration, force_field):
    # Compute the intra-molecular energy contribution for the system as part of the Coulomb Ewald summation.
    #
    # Parameters:
    # - system_data: DataFrame containing system-level information, including configuration, box length, and number of molecules.
    # - configuration: DataFrame containing atomic positions and atom types for the specific configuration.
    # - force_field: DataFrame containing the force field parameters (sigma, epsilon, charge, etc.) for the atoms.
    #
    # Returns:
    # - intra_energy: Computed intra-molecular energy (in K) as a float value.
    #
    # Description:
    # This function computes the intra-molecular energy, which represents the energy associated
    # with the interactions between atoms within the same molecule as part of the Coulomb Ewald summation.
    # This includes interactions like van der Waals forces and non-bonded interactions between atoms
    # that are in the same molecule.
    intra_energy = 0

    #--- Complete this code ---#

    return intra_energy


# DataFrame Descriptions:

# NIST_SPC_E_Water DataFrames:
# This dataset contains reference calculations for SPC/E water in non-cuboid cells,
# as provided by NIST (https://www.nist.gov/mml/csd/chemical-informatics-group/spce-water-reference-calculations-non-cuboid-cell-10a-cutoff).
# The data includes thermodynamic properties, force field parameters, system metadata, and atomic configurations.

# 1. NIST_SPC_E_Water DataFrame:
#    - Contains thermodynamic properties of SPC/E water configurations in monoclinic cells.
#    - Columns:
#        - 'Configuration' (int): Configuration ID (1-4).
#        - 'M (number of SPC/E molecules)' (int): Number of SPC/E molecules in the system.
#        - 'Cell Type' (str): Type of simulation cell (Monoclinic).
#        - 'Cell Side Lengths [a, b, c] (Å)' (tuple of floats): Simulation box dimensions in Ångströms.
#        - 'Cell Angles [α, β, γ] (degrees)' (tuple of floats): Cell angles defining the monoclinic structure.
#        - 'Number of Wave Vectors' (int): Number of wave vectors used in Ewald summation.
#        - 'Edisp/kB (K)' (float), 'ELRC/kB (K)' (float), 'Ereal/kB (K)' (float), 'Efourier/kB (K)' (float),
#          'Eself/kB (K)' (float), 'Eintra/kB (K)' (float), 'Etotal/kB (K)' (float): Various energy components in Kelvin.

# 2. force_field DataFrame:
#    - Contains force field parameters for SPC/E water, specifically for oxygen ('O') and hydrogen ('H').
#    - Columns:
#        - 'type' (str): Atom type ('O' or 'H').
#        - 'sigma' (float): Lennard-Jones parameter (Å).
#        - 'epsilon' (float): Lennard-Jones well depth (K).
#        - 'charge' (float): Partial charge (e).
#        - 'num_particles' (int): Number of particles per molecule.

# 3. system DataFrame:
#    - Contains metadata about each system configuration.
#    - Columns:
#        - 'file_paths' (str): File names containing atomic configurations.
#        - 'configuration #' (int): Extracted configuration number (1-4).
#        - 'number of particles' (int): Number of molecules (from 'NIST_SPC_E_Water').
#        - 'box length' (tuple of floats): Box dimensions (from 'NIST_SPC_E_Water').
#        - 'cutoff' (float): Fixed cutoff distance for interactions (10 Å).
#        - 'alpha' (float): Ewald summation parameter (5.6 / min(a, b, c)).
#        - 'kmax' (int): Maximum wave vector index (5); also, only include k for which k² < kmax² + 2, i.e., k² < 27.
#        - 'ε0' (float): Permittivity of Vacuum (8.854187817E-12 C²/(J m)).
#        - 'kB' (float): Boltzmann Constant (1.3806488E-23 J/K).

# 4. configuration DataFrame (from 'extracting_positions'):
#    - Created per file, containing atomic positions.
#    - Columns:
#        - 'X' (float): Atom coordinates in Ångströms.
#        - 'Y' (float): Atom coordinates in Ångströms.
#        - 'Z' (float): Atom coordinates in Ångströms.
#        - 'Atom Type' (str): Type of atom ('O' or 'H').
#        - 'Molecule' (int): Molecule index assigned based on position.

system, force_field, NIST_SPC_E_Water = creating_dataframes(file_paths, atom_properties,NIST_SPC_E_Water)

print(system.info())
print(force_field.info())
print(NIST_SPC_E_Water.info())
print(extracting_positions(file_paths[0]).info())

# Computing energies storing in results
results = pd.DataFrame()

results['Number of Particles'] = system['number of particles'].astype(int)

# Calculate pairwise energy for all system configurations
results['real_energies'] = system['file_paths'].apply(
    lambda file_path: compute_real_energies(
        system[system['file_paths'] == file_path].iloc[0],  # Ensure single row selection
        extracting_positions(file_path),
        force_field
    )
)

# Calculate pairwise energy for all system configurations
results['fourier_energies'] = system['file_paths'].apply(
    lambda file_path: compute_fourier_energies(
        system[system['file_paths'] == file_path].iloc[0],  # Ensure single row selection
        extracting_positions(file_path),
        force_field
    )
)

# Calculate pairwise energy for all system configurations
results['self_energies'] = system['file_paths'].apply(
    lambda file_path: compute_self_energies(
        system[system['file_paths'] == file_path].iloc[0],  # Ensure single row selection
        extracting_positions(file_path),
        force_field
    )
)

# Calculate pairwise energy for all system configurations
results['intra_energies'] = system['file_paths'].apply(
    lambda file_path: compute_intra_energies(
        system[system['file_paths'] == file_path].iloc[0],  # Ensure single row selection
        extracting_positions(file_path),
        force_field
    )
)

def compare_coulomb_energy(df1, df2, tolerance=1e-4):
    # Merge df1 and df2 based on the number of particles
    df_merged = df1.merge(df2, left_on='Number of Particles', right_on='M (number of SPC/E molecules)', how='left')

    # Initialize counters
    matched_real = matched_fourier = matched_self = matched_intra = 0
    not_matched_real = not_matched_fourier = not_matched_self = not_matched_intra = 0

    # Initialize output lists
    real_energy_output, fourier_energy_output = [], []
    self_energy_output, intra_energy_output = [], []

    # Iterate over merged DataFrame
    for idx, row in df_merged.iterrows():
        # Extract computed values from df1
        real_energy = row['real_energies']
        fourier_energy = row['fourier_energies']
        self_energy = row['self_energies']
        intra_energy = row['intra_energies']
        num_molecules = row['Number of Particles']

        # Extract reference values from df2
        if pd.isna(row['Ereal/kB (K)']):
            continue  # Skip if no match is found in df2

        nist_real_energy = float(row['Ereal/kB (K)'])
        nist_fourier_energy = float(row['Efourier/kB (K)'])
        nist_self_energy = float(row['Eself/kB (K)'])
        nist_intra_energy = float(row['Eintra/kB (K)'])

        # Perform numeric comparisons with a tolerance
        match_real = np.isclose(real_energy, nist_real_energy, atol=tolerance)
        match_fourier = np.isclose(fourier_energy, nist_fourier_energy, atol=tolerance)
        match_self = np.isclose(self_energy, nist_self_energy, atol=tolerance)
        match_intra = np.isclose(intra_energy, nist_intra_energy, atol=tolerance)

        matched_real += int(match_real)
        not_matched_real += int(not match_real)

        matched_fourier += int(match_fourier)
        not_matched_fourier += int(not match_fourier)

        matched_self += int(match_self)
        not_matched_self += int(not match_self)

        matched_intra += int(match_intra)
        not_matched_intra += int(not match_intra)

        # Store formatted outputs
        real_energy_output.append(f"Test {idx+1} ({num_molecules} molecules): Computed: {real_energy:.4E}, NIST: {nist_real_energy:.4E}, Match: {match_real}")
        fourier_energy_output.append(f"Test {idx+1} ({num_molecules} molecules): Computed: {fourier_energy:.4E}, NIST: {nist_fourier_energy:.4E}, Match: {match_fourier}")
        self_energy_output.append(f"Test {idx+1} ({num_molecules} molecules): Computed: {self_energy:.4E}, NIST: {nist_self_energy:.4E}, Match: {match_self}")
        intra_energy_output.append(f"Test {idx+1} ({num_molecules} molecules): Computed: {intra_energy:.4E}, NIST: {nist_intra_energy:.4E}, Match: {match_intra}")

    # Print final results
    print("Real Energy Comparison:")
    print(*real_energy_output, sep=chr(10))
    print("Fourier Energy Comparison:")
    print(*fourier_energy_output, sep=chr(10))
    print("Self Energy Comparison:")
    print(*self_energy_output, sep=chr(10))
    print("Intra Energy Comparison:")
    print(*intra_energy_output, sep=chr(10))
    print()
    print(f"Count of correct Real Energy answers: {matched_real}")
    print(f"Count of incorrect Real Energy answers: {not_matched_real}")
    print(f"Count of correct Fourier Energy answers: {matched_fourier}")
    print(f"Count of incorrect Fourier Energy answers: {not_matched_fourier}")
    print(f"Count of correct Self Energy answers: {matched_self}")
    print(f"Count of incorrect Self Energy answers: {not_matched_self}")
    print(f"Count of correct Intra Energy answers: {matched_intra}")
    print(f"Count of incorrect Intra Energy answers: {not_matched_intra}")
    print()

    total_correct = matched_real + matched_fourier + matched_self + matched_intra
    total_incorrect = not_matched_real + not_matched_fourier + not_matched_self + not_matched_intra

    print(f"Total correct answers: {total_correct}")
    print(f"Total incorrect answers: {total_incorrect}")


# calling compare_coulomb_energy function
compare_coulomb_energy(results, NIST_SPC_E_Water)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1 entries, 0 to 0
Data columns (total 9 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   file_paths           1 non-null      object 
 1   configuration #      1 non-null      int64  
 2   number of particles  1 non-null      float64
 3   box length           1 non-null      float64
 4   cutoff               1 non-null      int64  
 5   alpha                1 non-null      float64
 6   kmax                 1 non-null      int64  
 7   ε0                   1 non-null      float64
 8   kB                   1 non-null      float64
dtypes: float64(5), int64(3), object(1)
memory usage: 204.0+ bytes
None
<class 'pandas.core.frame.DataFrame'>
Index: 2 entries, O to H
Data columns (total 5 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   type           2 non-null      object 
 1   sigma          2 non-null      float64
 2   ep

In [21]:
class LLMTestConfig:
    def __init__(self, model_name: str, max_iterations: int, patience: int, context_type: str,
                 stop_on_repeat: bool = True, data_url_or_file_path: str = None, 
                 system_prompt: str = None, embedding_model_name: str = None, 
                 context_file: str = None, reasoning_factor: str = "medium"):
        self.model_name = model_name.lower()  # Name of the LLM (e.g., 'GPT-4')
        self.max_iterations = max_iterations  # Maximum number of iterations the LLM can take
        self.patience = patience  # Patience level (how many times the output should repeat before stopping)
        self.stop_on_repeat = stop_on_repeat  # Whether to stop if the output repeats
        self.data_url_or_file_path = data_url_or_file_path  # Data URL or file path for RAG
        self.embedding_model_name = embedding_model_name  # Optional, custom embedding model name
        self.reasoning_factor = reasoning_factor.lower()  # Normalize input to lowercase
        self.context_type = context_type.lower()

        # Validate reasoning factor
        if self.reasoning_factor not in ["high", "medium", "low"]:
            raise ValueError("reasoning_factor must be 'high', 'medium', or 'low'.")

        # Store context file name without opening
        self.context_file = context_file  

        # Adjust system prompt based on reasoning factor
        reasoning_prompts = {
            "high": "Provide a detailed, step-by-step explanation with rigorous justification.",
            "medium": "Give a balanced explanation with key reasoning points.",
            "low": "Provide a brief, high-level response with minimal details."
        }
        
        self.system_prompt = system_prompt or (
            f"You are an expert in molecular simulations in chemistry and energy calculations for molecules. "
            f"Use the retrieved context (if available) to answer the question.\n\n"
            f"{reasoning_prompts[self.reasoning_factor]}"
        )  # Default system prompt if not provided

# Example usage
llm_config = LLMTestConfig(
    model_name="gpt-4o",           # Model name
    max_iterations=10,              # Maximum number of iterations
    patience=10,                     # Patience level for stopping condition
    stop_on_repeat=False,            # Whether to stop if the output repeats
    data_url_or_file_path="https://www.nist.gov/mml/csd/chemical-informatics-group/spce-water-reference-calculations-10a-cutoff",  # Data URL or file path for RAG
    embedding_model_name="text-embedding-3-large",  # Custom embedding model (if desired)
    reasoning_factor="medium",        # Reasoning level: "high", "medium", or "low"
    context_type="short",           # Context mode: "short", "medium", or "large"
    context_file="../context/shorter_context.txt"  # Context file (only storing name, not opening)
)


In [14]:
llm_config.model_name

'gpt-4o'

In [23]:
from langchain import hub
from LLM_config import llm_config  # Import the LLMTestConfig class
from langgraph.graph import MessagesState, StateGraph
from langchain_core.tools import tool
from langchain_core.messages import SystemMessage
from langgraph.prebuilt import ToolNode
from langgraph.graph import END
from langgraph.prebuilt import ToolNode, tools_condition
from langgraph.checkpoint.memory import MemorySaver

from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_community.document_loaders import RecursiveUrlLoader
from langchain_openai import OpenAIEmbeddings
from langchain_openai import ChatOpenAI
from langchain_chroma import Chroma

# Use values from LLMTestConfig
llm_openAI = ChatOpenAI(model=llm_config.model_name)  # Use model_name from config

# Initialize embeddings
embeddings = OpenAIEmbeddings(model=llm_config.embedding_model_name)

# Initialize vector store
vector_store = Chroma(embedding_function=embeddings)

# Load and chunk contents of the blog
loader = RecursiveUrlLoader(llm_config.data_url_or_file_path)
docs = loader.load()

# Step 2: Chunk the text
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
all_splits = text_splitter.split_documents(docs)

_ = vector_store.add_documents(documents=all_splits)

# Define prompt for question-answering
prompt = hub.pull("rlm/rag-prompt")

graph_builder = StateGraph(MessagesState)

@tool(response_format="content_and_artifact")
def retrieve(query: str):
    """Retrieve information related to a query."""
    retrieved_docs = vector_store.similarity_search(query, k=2)
    serialized = "\n\n".join(
        (f"Source: {doc.metadata}\n" f"Content: {doc.page_content}")
        for doc in retrieved_docs
    )
    return serialized, retrieved_docs

# Step 1: Generate an AIMessage that may include a tool-call to be sent.
def query_or_respond(state: MessagesState):
    """Generate tool call for retrieval or respond."""
    llm_with_tools = llm_openAI.bind_tools([retrieve])
    response = llm_with_tools.invoke(state["messages"])
    # MessagesState appends messages to state instead of overwriting
    return {"messages": [response]}


# Step 2: Execute the retrieval.
tools = ToolNode([retrieve])


# Step 3: Generate a response using the retrieved content.
def generate(state: MessagesState):
    """Generate answer."""
    # Get generated ToolMessages
    recent_tool_messages = []
    for message in reversed(state["messages"]):
        if message.type == "tool":
            recent_tool_messages.append(message)
        else:
            break
    tool_messages = recent_tool_messages[::-1]

    # Format into prompt
    docs_content = "\n\n".join(doc.content for doc in tool_messages)
    system_message_content = (
        f"{llm_config.system_prompt}"
        "\n\n"
        f"{docs_content}"
    )
    conversation_messages = [
        message
        for message in state["messages"]
        if message.type in ("human", "system")
        or (message.type == "ai" and not message.tool_calls)
    ]
    prompt = [SystemMessage(system_message_content)] + conversation_messages

    # Run
    response = llm_openAI.invoke(prompt)
    return {"messages": [response]}

graph_builder.add_node(query_or_respond)
graph_builder.add_node(tools)
graph_builder.add_node(generate)

graph_builder.set_entry_point("query_or_respond")
graph_builder.add_conditional_edges(
    "query_or_respond",
    tools_condition,
    {END: END, "tools": "tools"},
)
graph_builder.add_edge("tools", "generate")
graph_builder.add_edge("generate", END)

graph = graph_builder.compile()

memory = MemorySaver()
graph = graph_builder.compile(checkpointer=memory)

# Specify an ID for the thread
config = {"configurable": {"thread_id": "abc123"}}

In [ ]:
results_logs

,Run_ID,Iteration,Timestamp,model_name,reasoning_factor,context_type,Feedback,AI_Message,Updated_Coulomb_input


In [ ]:
print(results_logs['AI_Message'][1])

Below is one possible complete solution, with the four functions implemented. (Make sure not to change any print statements or the function names/parameters.) In this solution, we compute:

1. The real-space term (Ereal) by summing over all unique pairs within the cutoff using erfc(αr).
2. The Fourier-space term (Efourier) by summing over allowed reciprocal lattice vectors (n = [nx, ny, nz] with n² < 27) and computing the structure factor S(n) (using exp(2π i n·r/L)).
3. The self-energy correction (Eself) by summing –(α/√π) * q²/(4πϵ₀) over all atoms.
4. The intramolecular energy (Eintra) term by summing over each unique pair in every molecule using erf(αr).

Below is the complete updated code:

--------------------------------------------------
#!/usr/bin/env python3
import numpy as np
import pandas as pd
from scipy.special import erfc, erf
import math 

# defining all variables
atom_properties = {
    'O': {'type': 'O', 'sigma': 3.165558, 'epsilon': 78.197431, 'charge': -0.8476, 'num

In [ ]:
with open("Coulomb_input.py", "r") as file:
    Coulomb_input = file.read()

In [ ]:
import re

def replace_function_by_index(Coulomb_input, function_body, function_name):
    pattern = r"(def " + re.escape(function_name) + r".*?return.*?)(?=\n|$)"
    updated_code = re.sub(pattern, function_body, Coulomb_input, flags=re.DOTALL)
    return updated_code


In [ ]:
import subprocess

def execute_code(code: str) -> dict:
    try:
        process = subprocess.Popen(
            ["/Users/pulicharishma/anaconda3/bin/python", "-c", code],
            text=True,
            stdout=subprocess.PIPE,
            stderr=subprocess.PIPE
        )
        output, errors = process.communicate()
        
        return {
            "stdout": output.strip().split("\n") if output else [],
            "stderr": errors.strip().split("\n") if errors else [],
            "return_code": process.returncode
        }
    except Exception as e:
        return {"error": str(e)}

In [82]:
results_logs['AI_Message']

Series([], Name: AI_Message, dtype: object)

In [81]:
function_names = [
        "compute_real_energies",
        "compute_fourier_energies",
        "compute_intra_energies",
        "compute_self_energies"
    ]

for function_name in function_names:
    extracted_function = extract_parent_with_children(results_logs['AI_Message'][2], function_name)
    if extracted_function:
        functions_filled[function_name] = extracted_function

KeyError: 2

In [ ]:
if not functions_filled:
    feedback = "No Python code found. Please review the code more clearly and provide a Python code after changes."
else:
    for function_name, function_code in functions_filled.items():
        Coulomb_input1 = replace_function_by_index(Coulomb_input, function_code, function_name)

In [79]:
for i in execute_code(Coulomb_input1)['stdout']:
    print(i)

Real Energy Comparison:
Test 1 (100.0 molecules): Computed: 0.0000E+00, NIST: -5.5889E+05, Match: False
Fourier Energy Comparison:
Test 1 (100.0 molecules): Computed: 0.0000E+00, NIST: 6.2701E+03, Match: False
Self Energy Comparison:
Test 1 (100.0 molecules): Computed: -2.2164E+35, NIST: -2.8447E+06, Match: False
Intra Energy Comparison:
Test 1 (100.0 molecules): Computed: 0.0000E+00, NIST: 2.8100E+06, Match: False

Count of correct Real Energy answers: 0
Count of incorrect Real Energy answers: 1
Count of correct Fourier Energy answers: 0
Count of incorrect Fourier Energy answers: 1
Count of correct Self Energy answers: 0
Count of incorrect Self Energy answers: 1
Count of correct Intra Energy answers: 0
Count of incorrect Intra Energy answers: 1

Total correct answers: 0
Total incorrect answers: 4


In [ ]:
import re

def extract_parent_with_children(code, function_name):
    parent_function_pattern = r"(def " + re.escape(function_name) + r".*?return.*?)(?=\n|$)"
    parent_match = re.search(parent_function_pattern, code, re.DOTALL)

    if parent_match:
        parent_function_body = parent_match.group(0)
        function_call_pattern = r"([a-zA-Z_][a-zA-Z0-9_]*\s?\(.*?\))"
        function_calls = re.findall(function_call_pattern, parent_function_body)

        related_functions = set()
        for call in function_calls:
            func_name = call.split('(')[0].strip()
            related_functions.add(func_name)

        related_functions_body = ''
        for func_name in related_functions:
            child_function_pattern = r"(def " + re.escape(func_name) + r".*?return.*?)(?=\n|$)"
            child_function_match = re.search(child_function_pattern, code, re.DOTALL)

            if child_function_match:
                related_functions_body += child_function_match.group(0) + "\n"

        return related_functions_body
    else:
        return None

In [ ]:
replace_function_by_index()

In [5]:
print(results_logs['AI_Message'][5])

Below is one complete solution. Make sure to insert the code only into the sections marked “#--- Complete this code ---#” without modifying any other part of the code (names, parameters, or print statements).

────────────────────────────
import numpy as np
import pandas as pd
from scipy.special import erfc, erf
import math 

# defining all variables
atom_properties = {
    'O': {'type': 'O', 'sigma': 3.165558, 'epsilon': 78.197431, 'charge': -0.8476, 'num_particles': 1},
    'H': {'type': 'H', 'sigma': 0.000000, 'epsilon': 0.000000, 'charge': 0.4238, 'num_particles': 2},
}

# Trying for small configurations first
file_paths = [
#        'spce_sample_config_periodic4.txt',
#        'spce_sample_config_periodic2.txt',
#        'spce_sample_config_periodic3.txt',
        '../data/spce_sample_config_periodic1_modified.txt'
    ]

NIST_SPC_E_Water = {
    'Configuration': [1, 2, 3, 4],
    'M (number of SPC/E molecules)': [100, 200, 300, 750],
    'Lx=Ly=Lz (Å)': [20.0, 20.0, 20.0, 30.0],


In [6]:
print(results_logs['Updated_Coulomb_input'][5])

import numpy as np
import pandas as pd
from scipy.special import erfc,erf
import math 

# defining all variables
atom_properties = {
    'O': {'type': 'O', 'sigma': 3.165558, 'epsilon': 78.197431, 'charge': -0.8476, 'num_particles': 1},
    'H': {'type': 'H', 'sigma': 0.000000, 'epsilon': 0.000000, 'charge': 0.4238, 'num_particles': 2},
}

# Trying for small configurations first
file_paths = [
#        'spce_sample_config_periodic4.txt',
#        'spce_sample_config_periodic2.txt',
#        'spce_sample_config_periodic3.txt',
        '../data/spce_sample_config_periodic1_modified.txt'
    ]

NIST_SPC_E_Water = {
        'Configuration': [1, 2, 3, 4],
        'M (number of SPC/E molecules)': [100, 200, 300, 750],
        'Lx=Ly=Lz (Å)': [20.0, 20.0, 20.0, 30.0],
        'Edisp/kB (K)': [9.95387E+04, 1.93712E+05, 3.54344E+05, 4.48593E+05],
        'ELRC/kB (K)': [-8.23715E+02, -3.29486E+03, -7.41343E+03, -1.37286E+04],
        'Ereal/kB (K)': [-5.58889E+05, -1.19295E+06, -1.96297E+06, -3

In [12]:
print(results_logs['Updated_Coulomb_input'][0].replace("\\\\", "\\"))


import numpy as np
import pandas as pd
from scipy.special import erfc,erf
import math 

# defining all variables
atom_properties = {
    'O': {'type': 'O', 'sigma': 3.165558, 'epsilon': 78.197431, 'charge': -0.8476, 'num_particles': 1},
    'H': {'type': 'H', 'sigma': 0.000000, 'epsilon': 0.000000, 'charge': 0.4238, 'num_particles': 2},
}

# Trying for small configurations first
file_paths = [
#        'spce_sample_config_periodic4.txt',
#        'spce_sample_config_periodic2.txt',
#        'spce_sample_config_periodic3.txt',
        '../data/spce_sample_config_periodic1_modified.txt'
    ]

NIST_SPC_E_Water = {
        'Configuration': [1, 2, 3, 4],
        'M (number of SPC/E molecules)': [100, 200, 300, 750],
        'Lx=Ly=Lz (Å)': [20.0, 20.0, 20.0, 30.0],
        'Edisp/kB (K)': [9.95387E+04, 1.93712E+05, 3.54344E+05, 4.48593E+05],
        'ELRC/kB (K)': [-8.23715E+02, -3.29486E+03, -7.41343E+03, -1.37286E+04],
        'Ereal/kB (K)': [-5.58889E+05, -1.19295E+06, -1.96297E+06, -3

In [4]:
import ast

import ast
import pandas as pd

def extract_functions_with_children(file_path):
    """
    Extracts functions from a Python script, distinguishing parent and child functions.

    Args:
        file_path (str): Path to the Python script.

    Returns:
        pd.DataFrame: DataFrame with 'function_name', 'func_code', and 'child_func_dict'.
    """
    with open(file_path, "r", encoding="utf-8") as f:
        source_code = f.read()

    tree = ast.parse(source_code)
    functions_data = []

    for node in tree.body:  # Iterate over top-level elements
        if isinstance(node, ast.FunctionDef):  # Only process functions
            function_name = node.name
            function_code = ast.get_source_segment(source_code, node)

            # Extract child functions inside the parent function
            child_func_dict = {}
            for child in node.body:
                if isinstance(child, ast.FunctionDef):  # Check if it's a nested function
                    child_name = child.name
                    child_code = ast.get_source_segment(source_code, child)
                    child_func_dict[child_name] = child_code

            # Append to list
            functions_data.append({
                "function_name": function_name,
                "func_code": function_code,
                "child_func_dict": child_func_dict if child_func_dict else None  # Store None if no child functions
            })

    # Convert to DataFrame
    df = pd.DataFrame(functions_data, columns=["function_name", "func_code", "child_func_dict"])
    return df

def extract_functions_from_file(file_path):
    """
    Extracts all function names and their corresponding code as a dictionary from a given Python file.
    
    Args:
        file_path (str): Path to the Python file.

    Returns:
        dict: A dictionary where keys are function names and values are the corresponding function code.
    """
    with open(file_path, "r", encoding="utf-8") as f:
        source_code = f.read()

    tree = ast.parse(source_code)
    functions = {}

    for node in ast.walk(tree):
        if isinstance(node, ast.FunctionDef):  # Extract function definitions
            function_name = node.name
            function_code = ast.get_source_segment(source_code, node)
            functions[function_name] = function_code

    return functions

# Example Usage:
file_path = "Coulomb_input.py"  # Replace with the actual file path
function_dict = extract_functions_from_file(file_path)

# Print extracted functions
for func_name, func_code in function_dict.items():
    print(f"Function: {func_name}\nCode:\n{func_code}\n{'-'*50}")

df_functions = extract_functions_with_children(file_path)

# Print DataFrame
print(df_functions)


Function: extracting_positions
Code:
def extracting_positions(input_file):
        # Extract the positions from the .xyz file
    with open(input_file, "r") as file:
        lines = file.readlines()

    data_lines = lines[2:]

    data_list = []
    for line in data_lines:
        stripped_line = line.strip()
        parts = stripped_line.split()
        if len(parts) >= 5:  
            try:
                x, y, z = map(float, parts[1:4])
                atom_type = parts[4]
                data_list.append([x, y, z, atom_type])
            except ValueError:
                continue  

    # Create a DataFrame with all configurations
    columns = ["X", "Y", "Z", "Atom Type"]
    configuration = pd.DataFrame(data_list, columns=columns)

    configuration.index = range(1, len(configuration) + 1)

    configuration["Molecule"] = ((configuration.index - 1) // 3) + 1
    
    return configuration
--------------------------------------------------
Function: creating_dataframes
Code:
def

In [14]:
print(df_functions.loc[df_functions['function_name'] == 'compute_real_energies']['func_code'])


3    def compute_real_energies(system_data, configu...
Name: func_code, dtype: object


In [17]:
print(df_functions['func_code'][3])

def compute_real_energies(system_data, configuration, force_field):
    # Compute the real energy part of the Coulomb Ewald summation for the system.
    #
    # Parameters:
    # - system_data: DataFrame containing system-level information, including configuration, box length, and number of molecules.
    # - configuration: DataFrame containing atomic positions and atom types for the specific configuration.
    # - force_field: DataFrame containing the force field parameters (sigma, epsilon, charge, etc.) for the atoms.
    #
    # Returns:
    # - real_energy: Computed real energy part of the Coulomb Ewald summation (in K) as a float value.
    #
    # Description:
    # This function calculates the real energy contribution in the Coulomb Ewald summation,
    # which involves calculating the electrostatic interactions between atoms in the system
    # in real space. The real energy part represents the direct interactions between atoms
    # based on their positions and force field pa

In [21]:
import ast
import pandas as pd

def extract_functions_with_dependencies(file_path):
    """
    Extracts functions from a Python script and identifies function calls inside each function.

    Args:
        file_path (str): Path to the Python script.

    Returns:
        pd.DataFrame: DataFrame with 'function_name', 'func_code', and 'called_functions_dict'.
    """
    with open(file_path, "r", encoding="utf-8") as f:
        source_code = f.read()

    tree = ast.parse(source_code)
    functions_data = {}
    
    # First, extract all function definitions
    function_codes = {}
    for node in tree.body:
        if isinstance(node, ast.FunctionDef):
            function_codes[node.name] = ast.get_source_segment(source_code, node)  # Store function code

    # Now, extract function calls within each function
    for node in tree.body:
        if isinstance(node, ast.FunctionDef):
            function_name = node.name
            function_code = function_codes[function_name]

            # Find all function calls inside this function
            called_functions_dict = {}
            for child in ast.walk(node):  # Traverse the function body
                if isinstance(child, ast.Call) and isinstance(child.func, ast.Name):
                    called_func_name = child.func.id  # Get called function name
                    if called_func_name in function_codes:  # Only store if it's a defined function
                        called_functions_dict[called_func_name] = function_codes[called_func_name]

            # Store function details
            functions_data[function_name] = {
                "func_code": function_code,
                "called_functions_dict": called_functions_dict if called_functions_dict else None
            }

    # Convert to DataFrame
    df = pd.DataFrame([
        {
            "function_name": key,
            "func_code": value["func_code"],
            "called_functions_dict": value["called_functions_dict"]
        }
        for key, value in functions_data.items()
    ])
    
    return df

# Example Usage:
file_path = "Coulomb_input.py"  # Replace with actual Python script path
df_functions = extract_functions_with_dependencies(file_path)

# Print DataFrame
print(df_functions)


              function_name  \
0      extracting_positions   
1       creating_dataframes   
2        minimum_imgde_dist   
3     compute_real_energies   
4  compute_fourier_energies   
5     compute_self_energies   
6    compute_intra_energies   
7    compare_coulomb_energy   

                                           func_code  \
0  def extracting_positions(input_file):\n       ...   
1  def creating_dataframes(file_paths, atom_prope...   
2  def minimum_imgde_dist(system_data):\n    retu...   
3  def compute_real_energies(system_data, configu...   
4  def compute_fourier_energies(system_data, conf...   
5  def compute_self_energies(system_data, configu...   
6  def compute_intra_energies(system_data, config...   
7  def compare_coulomb_energy(df1, df2, tolerance...   

                               called_functions_dict  
0                                               None  
1                                               None  
2                                               No

In [29]:
import ast
import pandas as pd

def extract_functions_with_dependencies(source_code):
    """
    Extracts functions from a Python script string and identifies function calls inside each function.

    Args:
        source_code (str): The Python script as a string.

    Returns:
        pd.DataFrame: DataFrame with 'function_name', 'func_code', and 'called_functions_dict'.
    """
    tree = ast.parse(source_code)
    functions_data = {}

    # First, extract all function definitions
    function_codes = {}
    for node in tree.body:
        if isinstance(node, ast.FunctionDef):
            function_codes[node.name] = ast.get_source_segment(source_code, node)  # Store function code

    # Now, extract function calls within each function
    for node in tree.body:
        if isinstance(node, ast.FunctionDef):
            function_name = node.name
            function_code = function_codes[function_name]

            # Find all function calls inside this function
            called_functions_dict = {}
            for child in ast.walk(node):  # Traverse the function body
                if isinstance(child, ast.Call) and isinstance(child.func, ast.Name):
                    called_func_name = child.func.id  # Get called function name
                    if called_func_name in function_codes:  # Only store if it's a defined function
                        called_functions_dict[called_func_name] = function_codes[called_func_name]

            # Store function details
            functions_data[function_name] = {
                "func_code": function_code,
                "called_functions_dict": called_functions_dict if called_functions_dict else None
            }

    # Convert to DataFrame
    df = pd.DataFrame([
        {
            "function_name": key,
            "func_code": value["func_code"],
            "called_functions_dict": value["called_functions_dict"]
        }
        for key, value in functions_data.items()
    ])
    
    return df

In [36]:
import ast
import pandas as pd
import re

def extract_functions_with_dependencies_from_text(text):
    """
    Extracts Python functions from a string containing Python code and other text.
    Identifies function calls inside each function and extracts docstrings if present.

    Args:
        text (str): The input text that contains Python code along with other text.

    Returns:
        pd.DataFrame: DataFrame with 'function_name', 'func_code', and 'called_functions_dict'.
    """
    # Use regex to extract Python code embedded in the text
    # Assuming Python code is enclosed in triple backticks (```), or you can modify the pattern based on your needs
    python_code_blocks = re.findall(r'```(.*?)```', text, re.DOTALL)

    # If no Python code block is found, raise an error
    if not python_code_blocks:
        raise ValueError("No Python code found in the provided text.")

    # Combine all extracted code blocks into one (if multiple)
    python_code = "\n".join(python_code_blocks)

    # Parse the Python code using AST
    try:
        tree = ast.parse(python_code)
    except SyntaxError as e:
        raise ValueError(f"Syntax error in extracted Python code: {e}")

    functions_data = {}

    # First, extract all function definitions
    function_codes = {}
    function_docstrings = {}

    for node in tree.body:
        if isinstance(node, ast.FunctionDef):
            function_codes[node.name] = ast.get_source_segment(python_code, node)
            
            # Extract docstring using AST's method
            docstring = ast.get_docstring(node)
            if docstring:
                function_docstrings[node.name] = docstring
            else:
                function_docstrings[node.name] = None

    # Now, extract function calls within each function
    for node in tree.body:
        if isinstance(node, ast.FunctionDef):
            function_name = node.name
            function_code = function_codes[function_name]
            function_docstring = function_docstrings[function_name]

            # Find all function calls inside this function
            called_functions_dict = {}
            for child in ast.walk(node):  # Traverse the function body
                if isinstance(child, ast.Call) and isinstance(child.func, ast.Name):
                    called_func_name = child.func.id  # Get called function name
                    if called_func_name in function_codes:  # Only store if it's a defined function
                        called_functions_dict[called_func_name] = function_codes[called_func_name]

            # Store function details
            functions_data[function_name] = {
                "func_code": function_code,
                "docstring": function_docstring,
                "called_functions_dict": called_functions_dict if called_functions_dict else None
            }

    # Convert to DataFrame
    df = pd.DataFrame([
        {
            "function_name": key,
            "func_code": value["func_code"],
            "docstring": value["docstring"],
            "called_functions_dict": value["called_functions_dict"]
        }
        for key, value in functions_data.items()
    ])
    
    return df

In [44]:
df_functions = extract_functions_with_dependencies_from_text(text)
df_functions

,function_name,func_code,docstring,called_functions_dict
0,minimum_image_distance,"def minimum_image_distance(r_ij_vector, box_le...",Compute the minimum image distance considering...,None
1,compute_real_energies,"def compute_real_energies(system_data, configu...",None,{'minimum_image_distance': 'def minimum_image_...


In [54]:
import ast
import pandas as pd

def replace_functions(source_code, df_functions):
    """
    Replaces function definitions in `source_code` with those in `df_functions`,
    ensuring that dependent (child) functions are also included immediately before their parent functions.
    If child functions are not present, they are added.

    Args:
        source_code (str): The original script as a string.
        df_functions (pd.DataFrame): DataFrame containing function names, code, and dependencies.

    Returns:
        str: The modified source code with updated functions.
    """
    tree = ast.parse(source_code)
    modified_code = source_code  # Start with the original code

    # A set of function names already in the source code (for adding missing functions)
    existing_functions = {node.name for node in tree.body if isinstance(node, ast.FunctionDef)}

    # First, process child functions
    added_functions = {}  # Keep track of where each function's code should be added

    for _, row in df_functions.iterrows():
        called_functions_dict = row["called_functions_dict"]  # Dictionary of dependencies
        function_name = row["function_name"]

        if called_functions_dict:
            for child_name, child_code in called_functions_dict.items():
                if child_name not in existing_functions and child_name not in added_functions:
                    # Add child function immediately before its parent function
                    added_functions[child_name] = child_code

    # Now, replace the functions with their updated versions
    for _, row in df_functions.iterrows():
        function_name = row["function_name"]
        updated_func_code = row["func_code"]
        called_functions_dict = row["called_functions_dict"]  # Dictionary of dependencies

        # Replace the function definition in the source code
        for node in tree.body:
            if isinstance(node, ast.FunctionDef) and node.name == function_name:
                old_func_code = ast.get_source_segment(source_code, node)
                modified_code = modified_code.replace(old_func_code, updated_func_code)

                # Also insert child functions immediately before the parent function
                if called_functions_dict:
                    for child_name, child_code in called_functions_dict.items():
                        if child_name in added_functions:
                            # Insert the child function code right before the parent function
                            modified_code = modified_code.replace(updated_func_code, added_functions[child_name] + "\n\n" + updated_func_code)

    return modified_code

In [55]:
# --- Read the input script ---
with open("Coulomb_input.py", "r", encoding="utf-8") as f:
    original_source_code = f.read()

# --- Process the source code ---
modified_source_code = replace_functions(original_source_code, df_functions)

print(modified_source_code)

import numpy as np
import pandas as pd
from scipy.special import erfc,erf
import math 

# defining all variables
atom_properties = {
    'O': {'type': 'O', 'sigma': 3.165558, 'epsilon': 78.197431, 'charge': -0.8476, 'num_particles': 1},
    'H': {'type': 'H', 'sigma': 0.000000, 'epsilon': 0.000000, 'charge': 0.4238, 'num_particles': 2},
}

# Trying for small configurations first
file_paths = [
#        'spce_sample_config_periodic4.txt',
#        'spce_sample_config_periodic2.txt',
#        'spce_sample_config_periodic3.txt',
        '../data/spce_sample_config_periodic1_modified.txt'
    ]

NIST_SPC_E_Water = {
        'Configuration': [1, 2, 3, 4],
        'M (number of SPC/E molecules)': [100, 200, 300, 750],
        'Lx=Ly=Lz (Å)': [20.0, 20.0, 20.0, 30.0],
        'Edisp/kB (K)': [9.95387E+04, 1.93712E+05, 3.54344E+05, 4.48593E+05],
        'ELRC/kB (K)': [-8.23715E+02, -3.29486E+03, -7.41343E+03, -1.37286E+04],
        'Ereal/kB (K)': [-5.58889E+05, -1.19295E+06, -1.96297E+06, -3

In [56]:
# --- Write the modified script ---
with open("modified_script.py", "w", encoding="utf-8") as f:
    f.write(modified_source_code)

print("Updated script saved to: modified_script.py")

Updated script saved to: modified_script.py


In [ ]:
import config
import os
import csv
import pandas as pd

from user_prompt import generate_user_prompt
from feedback import handle_feedback
from datetime import datetime
from LLM_config import llm_config  # Import the LLMTestConfig class
from rag_pipeline import graph
from rag_pipeline import config  # assuming `graph` is properly imported from wherever you define it
from iteration_execution import execute_code  # assuming `execute_code` is defined elsewhere
from iteration_execution import extract_functions_with_dependencies_from_text, replace_functions  # assuming defined elsewhere

# Dynamically create the file name to store results
if llm_config.is_reasoning:
    csv_file = f'../outputs/results_logs_{llm_config.model_name}_{llm_config.reasoning_factor}.csv'
else:
    csv_file = f'../outputs/results_logs_{llm_config.model_name}_{llm_config.temperature}.csv'
## Load existing CSV or create a new DataFrame
if os.path.exists(csv_file):
    results_logs = pd.read_csv(csv_file)
else:
    header = ['Run_ID', 'Iteration', 'Timestamp', 'model_name', 'reasoning_factor','context_type','Feedback', 'AI_Message', 'Updated_Coulomb_input']
    results_logs = pd.DataFrame(columns=header)

# Function to get the next run_id
def get_next_run_id(result_logs):
    if not results_logs.empty and "Run_ID" in results_logs.columns:
        return int(results_logs["Run_ID"].max()) + 1 if pd.notna(results_logs["Run_ID"].max()) else 1
    return 1

# Initialize variables
run_id = get_next_run_id(results_logs)  # Get next run_id
Iteration = {}
feedback = ""
iteration = 1
n = llm_config.max_iterations
last_output = ""
patience = llm_config.patience  # Initialize patience from LLM config

# Check if the CSV file exists, if not, create it and write the header
try:
    with open(csv_file, mode='r', newline='', encoding='utf-8') as file:
        pass  # If file exists, do nothing
except FileNotFoundError:
    with open(csv_file, mode='w', newline='', encoding='utf-8') as file:
        writer = csv.writer(file)
        writer.writerow(header)  # Write the header if the file does not exist

# Read Coulomb_input for the first time
with open("Coulomb_input.py", "r") as file:
    Coulomb_input = file.read()

while iteration <= n:  # Ensure it runs for max_iterations
    print(f"================================ Iteration {iteration} ================================\n")
    
    # Generate the user input message only for the first iteration
    input_message = generate_user_prompt(Coulomb_input, feedback)

    # Process LLM response with the graph pipeline
    for step in graph.stream(
        {"messages": [{"role": "user", "content": input_message}]},
        stream_mode="values",
        config=config,
    ):
        step["messages"][-1].pretty_print()
    # Assuming 'step' contains the necessary context and text
    text = step["messages"][-1].content

    # Extract functions and their dependencies from the text
    df_functions = extract_functions_with_dependencies_from_text(text)

    # Check if there are any function names extracted in the DataFrame
    if df_functions.empty or df_functions['function_name'].isnull().all():
        feedback = "No Python code found. Please review the code more clearly and provide a Python code after changes."
    else:
        # Replace functions in the Coulomb_input (or the input script)
        Coulomb_input = replace_functions(Coulomb_input, df_functions)
        print("Functions successfully replaced.")

        print("executing the code")
        result = execute_code(Coulomb_input)

        print("patience updation")
        # Check if the output has repeated
        if llm_config.stop_on_repeat and last_output and result['stdout'] and last_output == result['stdout'][-1]:
            patience -= 1  # Reduce patience
            print(f"Output repeated. Remaining patience: {patience}")

            if patience <= 0:
                print("The output has repeated for several iterations. The loop has stopped due to patience reaching 0.")
                break  # Exit the loop
        else:
            patience = llm_config.patience  # Reset patience if output changes""

        print("feedback updation")
        # Handle feedback and determine next steps
        feedback, iteration, n, last_output = handle_feedback(result, iteration, n, Iteration, last_output)
    
    # Append data to DataFrame
    new_row = {
        'Run_ID': run_id,
        'Iteration': iteration,
        'Timestamp': datetime.now().strftime("%Y-%m-%d %H:%M:%S"),
        'model_name': llm_config.model_name,
        'reasoning_factor': llm_config.reasoning_factor,
        'context_type': llm_config.context_type,
        'Feedback': feedback,
        'AI_Message': text,
        'Updated_Coulomb_input': Coulomb_input
    }
    
    print("storing results")  
    # Append data directly to the CSV file using csv.writer
    with open(csv_file, mode='a', newline='', encoding='utf-8') as file:
        writer = csv.writer(file)
        writer.writerow(new_row.values())  # Append the new row data

    if feedback == "LLM did a great job! Exiting from the loop":
        break

print(f"Results saved to {csv_file}")


================================ Iteration 1 ================================

================================ Human Message =================================

import numpy as np
import pandas as pd
from scipy.special import erfc,erf
import math 

# defining all variables
atom_properties = {
    'O': {'type': 'O', 'sigma': 3.165558, 'epsilon': 78.197431, 'charge': -0.8476, 'num_particles': 1},
    'H': {'type': 'H', 'sigma': 0.000000, 'epsilon': 0.000000, 'charge': 0.4238, 'num_particles': 2},
}

# Trying for small configurations first
file_paths = [
#        'spce_sample_config_periodic4.txt',
#        'spce_sample_config_periodic2.txt',
#        'spce_sample_config_periodic3.txt',
        '../data/spce_sample_config_periodic1_modified.txt'
    ]

NIST_SPC_E_Water = {
        'Configuration': [1, 2, 3, 4],
        'M (number of SPC/E molecules)': [100, 200, 300, 750],
        'Lx=Ly=Lz (Å)': [20.0, 20.0, 20.0, 30.0],
        'Edisp/kB (K)': [9.95387E+04, 1.93712E+05, 3.54344E+05, 4.4

In [ ]:
text = """Below is an example implementation where each missing function is filled in with a “traditional” Ewald summation–style calculation. Note that the code uses basic loops over atom pairs and a limited Fourier‐space sum. This example is kept at a high‐level and minimal detail so that it fits in the overall framework without altering other parts of the code.

--------------------------------------------------
# computing the real-space contribution
def compute_real_energies(system_data, configuration, force_field):
    real_energy = 0.0
    alpha = system_data['alpha']
    cutoff = system_data['cutoff']
    eps0 = system_data['ε0']
    
    positions = configuration[['X','Y','Z']].values
    charges = configuration['Atom Type'].apply(lambda at: force_field.loc[at,'charge']).values
    Natoms = len(positions)
    
    for i in range(Natoms-1):
        for j in range(i+1, Natoms):
            # compute minimum image distance (assuming cubic box)
            dx = positions[i,0] - positions[j,0]
            dy = positions[i,1] - positions[j,1]
            dz = positions[i,2] - positions[j,2]
            box = system_data['box length']
            dx -= box * round(dx/box)
            dy -= box * round(dy/box)
            dz -= box * round(dz/box)
            r = math.sqrt(dx*dx + dy*dy + dz*dz)
            if r < cutoff and r != 0:
                real_energy += (charges[i]*charges[j] / (4*math.pi*eps0)) * erfc(alpha*r)/r
    return real_energy

# computing the Fourier-space contribution
def compute_fourier_energies(system_data, configuration, force_field):
    fourier_energy = 0.0
    alpha = system_data['alpha']
    box = system_data['box length']
    eps0 = system_data['ε0']
    V = box**3
    
    positions = configuration[['X','Y','Z']].values
    charges = configuration['Atom Type'].apply(lambda at: force_field.loc[at,'charge']).values
    # Only include k-vectors with integers in range -kmax to kmax (excluding k=0)
    kmax = system_data['kmax']
    for nx in range(-kmax, kmax+1):
        for ny in range(-kmax, kmax+1):
            for nz in range(-kmax, kmax+1):
                if nx == ny == nz == 0:
                    continue
                kx = 2*math.pi*nx/box
                ky = 2*math.pi*ny/box
                kz = 2*math.pi*nz/box
                k2 = kx*kx+ky*ky+kz*kz
                # Check additional condition: k^2 < kmax^2+2 (i.e., k^2 < 27 for kmax=5)
                if k2 >= 27:
                    continue
                # structure factor
                sk_real = 0.0
                sk_imag = 0.0
                for j in range(len(positions)):
                    phase = kx*positions[j,0]+ky*positions[j,1]+kz*positions[j,2]
                    sk_real += charges[j]*math.cos(phase)
                    sk_imag += charges[j]*math.sin(phase)
                sk2 = sk_real*sk_real + sk_imag*sk_imag
                coef = math.exp(- (k2/(4*alpha*alpha)))/k2
                fourier_energy += coef * sk2
    fourier_energy *= (1/(2*V))*(1/(4*math.pi*eps0))
    return fourier_energy

# computing the self-energy correction
def compute_self_energies(system_data, configuration, force_field):
    self_energy = 0.0
    alpha = system_data['alpha']
    eps0 = system_data['ε0']
    # self energy for each particle
    for at in configuration['Atom Type']:
        q = force_field.loc[at,'charge']
        self_energy += q*q
    self_energy *= (-alpha/(math.sqrt(math.pi)))/(4*math.pi*eps0)
    return self_energy

# computing the intramolecular contribution
def compute_intra_energies(system_data, configuration, force_field):
    intra_energy = 0.0
    alpha = system_data['alpha']
    eps0 = system_data['ε0']
    # Each molecule contains three atoms (one oxygen and two hydrogens)
    molecules = configuration.groupby("Molecule")
    for m, group in molecules:
        indices = group.index.tolist()
        for i in range(len(indices)-1):
            for j in range(i+1, len(indices)):
                pos_i = group.iloc[i][['X','Y','Z']].values
                pos_j = group.iloc[j][['X','Y','Z']].values
                dx = pos_i[0]-pos_j[0]
                dy = pos_i[1]-pos_j[1]
                dz = pos_i[2]-pos_j[2]
                r = math.sqrt(dx*dx + dy*dy + dz*dz)
                if r != 0:
                    # using erf for intramolecular switching (removing nonbonded intramolecular)
                    qi = force_field.loc[group.iloc[i]['Atom Type'],'charge']
                    qj = force_field.loc[group.iloc[j]['Atom Type'],'charge']
                    intra_energy += (qi*qj/(4*math.pi*eps0)) * erf(alpha*r)/r
    return intra_energy

--------------------------------------------------

High-Level Explanation:
• In compute_real_energies the function loops over all unique atom pairs (using minimum image conventions) and computes their interactions using the complementary error function (erfc).
• In compute_fourier_energies the k-space contributions are summed over a restricted set of reciprocal lattice vectors (excluding k=0) using the structure factor’s real and imaginary parts.
• In compute_self_energies the self-interaction correction for each atom is computed.
• In compute_intra_energies the intramolecular interactions (such as oxygen–hydrogen in one molecule) are computed using the error function (erf)."""

In [39]:
import ast
import pandas as pd
import re

def extract_functions_with_dependencies_from_text(text):
    """
    Extracts Python functions from a string containing Python code and other text.
    Identifies function calls inside each function and extracts docstrings if present.

    Args:
        text (str): The input text that contains Python code along with other text.

    Returns:
        pd.DataFrame: DataFrame with 'function_name', 'func_code', and 'called_functions_dict'.
    """

    # Use regex to extract Python code embedded in the text
    function_pattern = r"(def [a-zA-Z_][a-zA-Z0-9_]*\s*\(.*?\):[\s\S]*?return.*?)(?=\n|$)"
    python_code_blocks = re.findall(function_pattern, text)

    # If no Python code block is found, raise an error
    if not python_code_blocks:
        raise ValueError("No Python code found in the provided text.")

    # Combine all extracted code blocks into one (if multiple)
    python_code = "\n".join(python_code_blocks)

    # Parse the Python code using AST
    try:
        tree = ast.parse(python_code)
    except SyntaxError as e:
        raise ValueError(f"Syntax error in extracted Python code: {e}")

    functions_data = {}

    # First, extract all function definitions
    function_codes = {}
    function_docstrings = {}

    for node in tree.body:
        if isinstance(node, ast.FunctionDef):
            function_codes[node.name] = ast.get_source_segment(python_code, node)
            
            # Extract docstring using AST's method
            docstring = ast.get_docstring(node)
            if docstring:
                function_docstrings[node.name] = docstring
            else:
                function_docstrings[node.name] = None

    # Now, extract function calls within each function
    for node in tree.body:
        if isinstance(node, ast.FunctionDef):
            function_name = node.name
            function_code = function_codes[function_name]
            function_docstring = function_docstrings[function_name]

            # Find all function calls inside this function
            called_functions_dict = {}
            for child in ast.walk(node):  # Traverse the function body
                if isinstance(child, ast.Call) and isinstance(child.func, ast.Name):
                    called_func_name = child.func.id  # Get called function name
                    if called_func_name in function_codes:  # Only store if it's a defined function
                        called_functions_dict[called_func_name] = function_codes[called_func_name]

            # Store function details
            functions_data[function_name] = {
                "func_code": function_code,
                "docstring": function_docstring,
                "called_functions_dict": called_functions_dict if called_functions_dict else None
            }

    # Convert to DataFrame
    df = pd.DataFrame([
        {
            "function_name": key,
            "func_code": value["func_code"],
            "docstring": value["docstring"],
            "called_functions_dict": value["called_functions_dict"]
        }
        for key, value in functions_data.items()
    ])
    
    return df


In [41]:
df_functions = extract_functions_with_dependencies_from_text(text)

In [46]:
print(Coulomb_input)

import numpy as np
import pandas as pd
from scipy.special import erfc,erf
import math 

# defining all variables
atom_properties = {
    'O': {'type': 'O', 'sigma': 3.165558, 'epsilon': 78.197431, 'charge': -0.8476, 'num_particles': 1},
    'H': {'type': 'H', 'sigma': 0.000000, 'epsilon': 0.000000, 'charge': 0.4238, 'num_particles': 2},
}

# Trying for small configurations first
file_paths = [
#        'spce_sample_config_periodic4.txt',
#        'spce_sample_config_periodic2.txt',
#        'spce_sample_config_periodic3.txt',
        '../data/spce_sample_config_periodic1_modified.txt'
    ]

NIST_SPC_E_Water = {
        'Configuration': [1, 2, 3, 4],
        'M (number of SPC/E molecules)': [100, 200, 300, 750],
        'Lx=Ly=Lz (Å)': [20.0, 20.0, 20.0, 30.0],
        'Edisp/kB (K)': [9.95387E+04, 1.93712E+05, 3.54344E+05, 4.48593E+05],
        'ELRC/kB (K)': [-8.23715E+02, -3.29486E+03, -7.41343E+03, -1.37286E+04],
        'Ereal/kB (K)': [-5.58889E+05, -1.19295E+06, -1.96297E+06, -3

In [45]:
extract_functions_with_dependencies_from_text(Coulomb_input)

,function_name,func_code,docstring,called_functions_dict
0,extracting_positions,def extracting_positions(input_file):\n ...,None,None
1,creating_dataframes,"def creating_dataframes(file_paths, atom_prope...",None,None
2,compute_real_energies,"def compute_real_energies(system_data, configu...",None,None
3,compute_fourier_energies,"def compute_fourier_energies(system_data, conf...",None,None
4,compute_self_energies,"def compute_self_energies(system_data, configu...",None,None
5,compute_intra_energies,"def compute_intra_energies(system_data, config...",None,None


In [42]:
def replace_functions(source_code, df_functions):
    """
    Replaces function definitions in `source_code` with those in `df_functions`,
    ensuring that dependent (child) functions are also included immediately before their parent functions.
    If child functions are not present, they are added.

    Args:
        source_code (str): The original script as a string.
        df_functions (pd.DataFrame): DataFrame containing function names, code, and dependencies.

    Returns:
        str: The modified source code with updated functions.
    """
    tree = ast.parse(source_code)
    modified_code = source_code  # Start with the original code

    # A set of function names already in the source code (for adding missing functions)
    existing_functions = {node.name for node in tree.body if isinstance(node, ast.FunctionDef)}

    # First, process child functions
    added_functions = {}  # Keep track of where each function's code should be added

    for _, row in df_functions.iterrows():
        called_functions_dict = row["called_functions_dict"]  # Dictionary of dependencies
        function_name = row["function_name"]

        if called_functions_dict:
            for child_name, child_code in called_functions_dict.items():
                if child_name not in existing_functions and child_name not in added_functions:
                    # Add child function immediately before its parent function
                    added_functions[child_name] = child_code

    # Now, replace the functions with their updated versions
    for _, row in df_functions.iterrows():
        function_name = row["function_name"]
        updated_func_code = row["func_code"]
        called_functions_dict = row["called_functions_dict"]  # Dictionary of dependencies

        # Replace the function definition in the source code
        for node in tree.body:
            if isinstance(node, ast.FunctionDef) and node.name == function_name:
                old_func_code = ast.get_source_segment(source_code, node)
                modified_code = modified_code.replace(old_func_code, updated_func_code)

                # Also insert child functions immediately before the parent function
                if called_functions_dict:
                    for child_name, child_code in called_functions_dict.items():
                        if child_name in added_functions:
                            # Insert the child function code right before the parent function
                            modified_code = modified_code.replace(updated_func_code, added_functions[child_name] + "\n\n" + updated_func_code)

    return modified_code

In [43]:
Coulomb_input

'import numpy as np\nimport pandas as pd\nfrom scipy.special import erfc,erf\nimport math \n\n# defining all variables\natom_properties = {\n    \'O\': {\'type\': \'O\', \'sigma\': 3.165558, \'epsilon\': 78.197431, \'charge\': -0.8476, \'num_particles\': 1},\n    \'H\': {\'type\': \'H\', \'sigma\': 0.000000, \'epsilon\': 0.000000, \'charge\': 0.4238, \'num_particles\': 2},\n}\n\n# Trying for small configurations first\nfile_paths = [\n#        \'spce_sample_config_periodic4.txt\',\n#        \'spce_sample_config_periodic2.txt\',\n#        \'spce_sample_config_periodic3.txt\',\n        \'../data/spce_sample_config_periodic1_modified.txt\'\n    ]\n\nNIST_SPC_E_Water = {\n        \'Configuration\': [1, 2, 3, 4],\n        \'M (number of SPC/E molecules)\': [100, 200, 300, 750],\n        \'Lx=Ly=Lz (Å)\': [20.0, 20.0, 20.0, 30.0],\n        \'Edisp/kB (K)\': [9.95387E+04, 1.93712E+05, 3.54344E+05, 4.48593E+05],\n        \'ELRC/kB (K)\': [-8.23715E+02, -3.29486E+03, -7.41343E+03, -1.37286E+04]

In [44]:
print(replace_functions(Coulomb_input,df_functions))

import numpy as np
import pandas as pd
from scipy.special import erfc,erf
import math 

# defining all variables
atom_properties = {
    'O': {'type': 'O', 'sigma': 3.165558, 'epsilon': 78.197431, 'charge': -0.8476, 'num_particles': 1},
    'H': {'type': 'H', 'sigma': 0.000000, 'epsilon': 0.000000, 'charge': 0.4238, 'num_particles': 2},
}

# Trying for small configurations first
file_paths = [
#        'spce_sample_config_periodic4.txt',
#        'spce_sample_config_periodic2.txt',
#        'spce_sample_config_periodic3.txt',
        '../data/spce_sample_config_periodic1_modified.txt'
    ]

NIST_SPC_E_Water = {
        'Configuration': [1, 2, 3, 4],
        'M (number of SPC/E molecules)': [100, 200, 300, 750],
        'Lx=Ly=Lz (Å)': [20.0, 20.0, 20.0, 30.0],
        'Edisp/kB (K)': [9.95387E+04, 1.93712E+05, 3.54344E+05, 4.48593E+05],
        'ELRC/kB (K)': [-8.23715E+02, -3.29486E+03, -7.41343E+03, -1.37286E+04],
        'Ereal/kB (K)': [-5.58889E+05, -1.19295E+06, -1.96297E+06, -3

In [ ]:
def replace_functions(source_code, df_functions):
    """
    Replaces function definitions in `source_code` with those in `df_functions`,
    ensuring that dependent (child) functions are also included immediately before their parent functions.
    If child functions are not present, they are added.

    Args:
        source_code (str): The original script as a string.
        df_functions (pd.DataFrame): DataFrame containing function names, code, and dependencies.

    Returns:
        str: The modified source code with updated functions.
    """
    tree = ast.parse(source_code)
    modified_code = source_code  # Start with the original code

    # A set of function names already in the source code (for adding missing functions)
    existing_functions = {node.name for node in tree.body if isinstance(node, ast.FunctionDef)}

    # First, process child functions
    added_functions = {}  # Keep track of where each function's code should be added

    for _, row in df_functions.iterrows():
        called_functions_dict = row["called_functions_dict"]  # Dictionary of dependencies
        function_name = row["function_name"]

        if called_functions_dict:
            for child_name, child_code in called_functions_dict.items():
                if child_name not in existing_functions and child_name not in added_functions:
                    # Add child function immediately before its parent function
                    added_functions[child_name] = child_code

    # Now, replace the functions with their updated versions
    for _, row in df_functions.iterrows():
        function_name = row["function_name"]
        updated_func_code = row["func_code"]
        called_functions_dict = row["called_functions_dict"]  # Dictionary of dependencies

        # Replace the function definition in the source code
        for node in tree.body:
            if isinstance(node, ast.FunctionDef) and node.name == function_name:
                old_func_code = ast.get_source_segment(source_code, node)
                modified_code = modified_code.replace(old_func_code, updated_func_code)

                # Also insert child functions immediately before the parent function
                if called_functions_dict:
                    for child_name, child_code in called_functions_dict.items():
                        if child_name in added_functions:
                            # Insert the child function code right before the parent function
                            modified_code = modified_code.replace(updated_func_code, added_functions[child_name] + "\n\n" + updated_func_code)

    return modified_code

In [80]:
import ast
import pandas as pd
import re

def extract_functions_with_global_variables_from_text(text):
    """
    Extracts Python functions and global variables from a string containing Python code and other text.
    Identifies function calls inside each function, extracts docstrings, and identifies global variables.

    Args:
        text (str): The input text that contains Python code along with other text.

    Returns:
        pd.DataFrame: DataFrame with 'function_name', 'func_code', 'docstring', 'global_variables', and 'called_functions_dict'.
    """
    # Use regex to extract Python code embedded in the text
    python_code = "\n".join(re.findall(r'```(.*?)```', text, re.DOTALL))

    # If no Python code block is found, raise an error
    if not python_code:
        raise ValueError("No Python code found in the provided text.")

    # Parse the Python code using AST
    tree = ast.parse(python_code)
    functions_data = {}

    # First, extract all function definitions
    function_codes = {}
    function_docstrings = {}

    for node in tree.body:
        if isinstance(node, ast.FunctionDef):
            function_codes[node.name] = ast.get_source_segment(python_code, node)
            
            # Extract docstring using AST's method
            docstring = ast.get_docstring(node)
            if docstring:
                function_docstrings[node.name] = docstring
            else:
                function_docstrings[node.name] = None

    # Extract global variables (variables in the global scope)
    global_variables = {}
    for node in tree.body:
        if isinstance(node, ast.Assign):  # Global variable assignment
            for target in node.targets:
                if isinstance(target, ast.Name):  # Only name targets are considered variables
                    # Extract the value of the global variable
                    value = ast.literal_eval(node.value) if isinstance(node.value, ast.Constant) else None
                    global_variables[target.id] = value

    # Now, extract function calls and variables within each function
    for node in tree.body:
        if isinstance(node, ast.FunctionDef):
            function_name = node.name
            function_code = function_codes[function_name]
            function_docstring = function_docstrings[function_name]

            # Find all function calls inside this function
            called_functions_dict = {}
            for child in ast.walk(node):  # Traverse the function body
                if isinstance(child, ast.Call) and isinstance(child.func, ast.Name):
                    called_func_name = child.func.id  # Get called function name
                    if called_func_name in function_codes:  # Only store if it's a defined function
                        called_functions_dict[called_func_name] = function_codes[called_func_name]

            # Store function details
            functions_data[function_name] = {
                "func_code": function_code,
                "docstring": function_docstring,
                "called_functions_dict": called_functions_dict if called_functions_dict else None
            }

    # Convert to DataFrame for functions
    df = pd.DataFrame([
        {
            "function_name": key,
            "func_code": value["func_code"],
            "docstring": value["docstring"],
            "called_functions_dict": value["called_functions_dict"]
        }
        for key, value in functions_data.items()
    ])

    return df, global_variables

In [90]:
import ast

def replace_functions(source_code, df_functions, global_variables):
    """
    Replaces function definitions in `source_code` with those in `df_functions`,
    ensuring that dependent (child) functions are also included immediately before their parent functions.
    The global variables are inserted once at the very top before all functions.

    Args:
        source_code (str): The original script as a string.
        df_functions (pd.DataFrame): DataFrame containing function names, code, and dependencies.
        global_variables (dict): Dictionary of global variables to be included before functions.

    Returns:
        str: The modified source code with updated functions and global variables.
    """
    # List of function names to be skipped from replacement
    fixed_functions = {"extracting_positions", "creating_dataframes", "compare_coulomb_energy"}

    tree = ast.parse(source_code)
    modified_code = source_code  # Start with the original code

    # A set of function names already in the source code (for adding missing functions)
    existing_functions = {node.name for node in tree.body if isinstance(node, ast.FunctionDef)}

    # First, process child functions
    added_functions = {}  # Keep track of where each function's code should be added

    for _, row in df_functions.iterrows():
        called_functions_dict = row["called_functions_dict"]  # Dictionary of dependencies
        function_name = row["function_name"]

        # Skip fixed functions
        if function_name in fixed_functions:
            continue

        if called_functions_dict:
            for child_name, child_code in called_functions_dict.items():
                if child_name not in existing_functions and child_name not in added_functions:
                    # Add child function immediately before its parent function
                    added_functions[child_name] = child_code

    # Now, replace the functions with their updated versions
    for _, row in df_functions.iterrows():
        function_name = row["function_name"]
        updated_func_code = row["func_code"]
        called_functions_dict = row["called_functions_dict"]  # Dictionary of dependencies

        # Skip fixed functions
        if function_name in fixed_functions:
            continue

        # Replace the function definition in the source code
        for node in tree.body:
            if isinstance(node, ast.FunctionDef) and node.name == function_name:
                old_func_code = ast.get_source_segment(source_code, node)

                # Insert the function code in place of the old one
                modified_code = modified_code.replace(old_func_code, updated_func_code)

                # Also insert child functions immediately before the parent function
                if called_functions_dict:
                    for child_name, child_code in called_functions_dict.items():
                        if child_name in added_functions:
                            # Insert the child function code right before the parent function
                            modified_code = modified_code.replace(updated_func_code, added_functions[child_name] + "\n\n" + updated_func_code)

    # Insert global variables once before all function definitions
    if isinstance(global_variables, dict):
        # Format global variables as code
        global_code = "\n".join(f"{var} = {repr(global_variables[var])}" for var in global_variables)
    else:
        raise ValueError("global_variables must be a dictionary.")

    # Find the first function definition and insert the global variables before it
    function_start_index = None
    for idx, line in enumerate(modified_code.splitlines()):
        if line.strip().startswith("def "):  # Check for the first function definition
            function_start_index = idx
            break

    if function_start_index is not None:
        # Insert the global variables just before the first function definition
        modified_code_lines = modified_code.splitlines()
        modified_code_lines.insert(function_start_index, global_code)
        modified_code = "\n".join(modified_code_lines)

    return modified_code


In [ ]:
def execute_code(code: str) -> dict:
    """
    Executes a Python code string and captures the output, errors, and return code.

    Args:
        code (str): The Python code to execute.

    Returns:
        dict: Dictionary with 'stdout', 'stderr', and 'return_code'.
    """
    try:
        process = subprocess.Popen(
            ["/Users/pulicharishma/anaconda3/bin/python", "-c", code],
            text=True,
            stdout=subprocess.PIPE,
            stderr=subprocess.PIPE
        )
        output, errors = process.communicate()
        
        return {
            "stdout": output.strip().split("\n") if output else [],
            "stderr": errors.strip().split("\n") if errors else [],
            "return_code": process.returncode
        }
    except Exception as e:
        return {"error": str(e)}


In [63]:
text = """"Below is the modified code block with the constant e_charge defined, along with the four functions. (Only the code in the sections marked "#--- Complete this code ---#" has been modified.)

```python
#--- Complete this code ---#
import numpy as np
import math

# Define the elementary charge in Coulombs
e_charge = 1.602176634e-19

def compute_real_energies(system_data, configuration, force_field):
    # Real-space energy: sum_{j<l} (q_j q_l/(4πε₀))*[erfc(α*r)/r] if r < cutoff
    real_energy = 0.0
    # Extract necessary system parameters and convert units
    cutoff_m = system_data['cutoff'] * 1e-10         # cutoff in meters
    box_length = system_data['box length']
    L_m = box_length * 1e-10                         # box length in meters
    alpha_m = system_data['alpha'] * 1e10            # convert from 1/Å to 1/m
    eps0 = system_data['ε0']                         # SI units: F/m (C²/(J m))
    kB = system_data['kB']
    
    # Get positions in meters
    pos = configuration[['X','Y','Z']].values.astype(float) * 1e-10  # shape (N,3) in m
    N_atoms = pos.shape[0]
    
    # Precompute charges for each atom (in Coulombs)
    charges = []
    for i in range(N_atoms):
        atom_type = configuration.iloc[i]['Atom Type']
        q = float(force_field.loc[atom_type, 'charge'])
        charges.append(q * e_charge)
    charges = np.array(charges)
    
    # Coulomb prefactor in SI units: 1/(4*pi*ε0)
    prefac = 1.0 / (4.0 * math.pi * eps0)
    
    # Loop over unique pairs j < l and sum contribution if distance within cutoff
    for j in range(N_atoms-1):
        for l in range(j+1, N_atoms):
            # Compute distance between atoms in meters
            rij = pos[j] - pos[l]
            r = np.linalg.norm(rij)
            if r <= cutoff_m and r > 0:
                term = (charges[j] * charges[l]) * prefac * math.erfc(alpha_m * r) / r
                real_energy += term
    # Return energy in Kelvin (Joules divided by kB)
    return real_energy / kB

def compute_fourier_energies(system_data, configuration, force_field):
    # Fourier-space energy:
    # E_fourier = (1/(2πV)) * Σ_{k≠0} [1/k² * exp(-k²/(4α²)) * (1/(4πε₀))* |Σ_j q_j exp(i k·r_j)|² ]
    fourier_energy = 0.0
    box_length = system_data['box length']
    L_m = box_length * 1e-10           # in meters
    V_m = L_m**3                       # volume in m^3
    eps0 = system_data['ε0']
    kB = system_data['kB']
    alpha_m = system_data['alpha'] * 1e10    # from 1/Å to 1/m
    
    # Get positions (in m) and charges (in Coulombs)
    pos = configuration[['X','Y','Z']].values.astype(float) * 1e-10  # (N,3)
    N_atoms = pos.shape[0]
    charges = []
    for i in range(N_atoms):
        atom_type = configuration.iloc[i]['Atom Type']
        q = float(force_field.loc[atom_type, 'charge'])
        charges.append(q * e_charge)
    charges = np.array(charges)
    
    # Prepare k-space summation.
    # Use system_data['kmax'] to build integer vector range.
    kmax_val = int(system_data['kmax'])
    # Include only k for which (k_x^2+k_y^2+k_z^2) < (kmax^2+2)
    ksq_limit = kmax_val**2 + 2
    
    # Loop over integer triplets in range. k-vector: k = 2π * n / L_m.
    for nx in range(-kmax_val, kmax_val+1):
        for ny in range(-kmax_val, kmax_val+1):
            for nz in range(-kmax_val, kmax_val+1):
                if nx == 0 and ny == 0 and nz == 0:
                    continue
                n2 = nx**2 + ny**2 + nz**2
                if n2 >= ksq_limit:
                    continue
                # k vector in reciprocal space (SI units: 1/m)
                k_vec = (2.0 * math.pi * np.array([nx, ny, nz])) / L_m
                k_sq = np.dot(k_vec, k_vec)
                if k_sq == 0:
                    continue
                # Compute the structure factor S(k) = Σ_j q_j exp(i k·r_j)
                S_k = 0.0 + 0.0j
                for j in range(N_atoms):
                    phase = np.dot(k_vec, pos[j])
                    S_k += charges[j] * np.exp(1j * phase)
                # Fourier term for this k-vector
                term = (1.0 / (2.0 * math.pi * V_m)) * (1.0 / k_sq) * \
                        np.exp(- k_sq / (4.0 * alpha_m**2)) * (1.0 / (4.0 * math.pi * eps0)) * (abs(S_k)**2)
                fourier_energy += term
    return fourier_energy / kB

def compute_self_energies(system_data, configuration, force_field):
    # Self energy: E_self = - (α/√π) * Σ_j (q_j^2/(4πε₀))
    self_energy = 0.0
    eps0 = system_data['ε0']
    kB = system_data['kB']
    alpha_m = system_data['alpha'] * 1e10   # convert to 1/m
    # Get charges in Coulombs
    N_atoms = configuration.shape[0]
    total = 0.0
    for i in range(N_atoms):
        atom_type = configuration.iloc[i]['Atom Type']
        q = float(force_field.loc[atom_type, 'charge'])
        total += (q * e_charge)**2
    self_energy = - alpha_m / math.sqrt(math.pi) * (total / (4.0 * math.pi * eps0))
    return self_energy / kB

def compute_intra_energies(system_data, configuration, force_field):
    # Intra-molecular energy: sum over molecules: for each molecule, sum over pairs (κ < λ)
    # E_intra = - Σ_mol Σ_{κ<λ} (q_κ * q_λ/(4πε₀)) * [erf(α*r_κλ)/r_κλ]
    intra_energy = 0.0
    eps0 = system_data['ε0']
    kB = system_data['kB']
    alpha_m = system_data['alpha'] * 1e10   # convert to 1/m
    
    # Group by molecule
    mol_groups = configuration.groupby("Molecule")
    for mol_id, group in mol_groups:
        atoms = group.reset_index(drop=True)
        N_atoms = atoms.shape[0]
        # Get positions (in m) and charges (in Coulombs) for atoms in this molecule
        pos = atoms[['X','Y','Z']].values.astype(float) * 1e-10
        charges = []
        for i in range(N_atoms):
            atom_type = atoms.iloc[i]['Atom Type']
            q = float(force_field.loc[atom_type, 'charge'])
            charges.append(q * e_charge)
        charges = np.array(charges)
        # Loop over unique pairs within the molecule
        for i in range(N_atoms-1):
            for j in range(i+1, N_atoms):
                rij = pos[i] - pos[j]
                r = np.linalg.norm(rij)
                if r > 0:
                    term = (charges[i] * charges[j]) / (4.0 * math.pi * eps0) * math.erf(alpha_m * r) / r
                    intra_energy -= term
    return intra_energy / kB
```

Explanation:
1. The constant e_charge is now defined at the start of the block.
2. For each energy function the units are converted appropriately: Å to meters for distances, and 1/Å to 1/m for alpha.
3. Partial charges are converted into Coulombs using e_charge.
4. The contributions are summed and converted from Joules to Kelvin by dividing by kB.
5. The print statements in the compare_coulomb_energy function and other parts of the code remain unchanged.

Please run the code to verify that the error is resolved."""

In [85]:
df_functions, global_variables = extract_functions_with_global_variables_from_text(text)
global_variables

{'e_charge': 1.602176634e-19}

In [86]:
df_functions

,function_name,func_code,docstring,called_functions_dict
0,compute_real_energies,"def compute_real_energies(system_data, configu...",None,None
1,compute_fourier_energies,"def compute_fourier_energies(system_data, conf...",None,None
2,compute_self_energies,"def compute_self_energies(system_data, configu...",None,None
3,compute_intra_energies,"def compute_intra_energies(system_data, config...",None,None


In [91]:
print(replace_functions(Coulomb_input,df_functions,dict(global_variables)))

import numpy as np
import pandas as pd
from scipy.special import erfc,erf
import math 

# defining all variables
atom_properties = {
    'O': {'type': 'O', 'sigma': 3.165558, 'epsilon': 78.197431, 'charge': -0.8476, 'num_particles': 1},
    'H': {'type': 'H', 'sigma': 0.000000, 'epsilon': 0.000000, 'charge': 0.4238, 'num_particles': 2},
}

# Trying for small configurations first
file_paths = [
#        'spce_sample_config_periodic4.txt',
#        'spce_sample_config_periodic2.txt',
#        'spce_sample_config_periodic3.txt',
        '../data/spce_sample_config_periodic1_modified.txt'
    ]

NIST_SPC_E_Water = {
        'Configuration': [1, 2, 3, 4],
        'M (number of SPC/E molecules)': [100, 200, 300, 750],
        'Lx=Ly=Lz (Å)': [20.0, 20.0, 20.0, 30.0],
        'Edisp/kB (K)': [9.95387E+04, 1.93712E+05, 3.54344E+05, 4.48593E+05],
        'ELRC/kB (K)': [-8.23715E+02, -3.29486E+03, -7.41343E+03, -1.37286E+04],
        'Ereal/kB (K)': [-5.58889E+05, -1.19295E+06, -1.96297E+06, -3

In [92]:
execute_code(replace_functions(Coulomb_input,df_functions,global_variables))

{'stdout': ['Real Energy Comparison:',
  'Test 1 (100.0 molecules): Computed: -2.8606E-08, NIST: -5.5889E+05, Match: False',
  'Fourier Energy Comparison:',
  'Test 1 (100.0 molecules): Computed: 3.1577E+03, NIST: 6.2701E+03, Match: False',
  'Self Energy Comparison:',
  'Test 1 (100.0 molecules): Computed: -5.6894E+07, NIST: -2.8447E+06, Match: False',
  'Intra Energy Comparison:',
  'Test 1 (100.0 molecules): Computed: 1.0167E+07, NIST: 2.8100E+06, Match: False',
  '',
  'Count of correct Real Energy answers: 0',
  'Count of incorrect Real Energy answers: 1',
  'Count of correct Fourier Energy answers: 0',
  'Count of incorrect Fourier Energy answers: 1',
  'Count of correct Self Energy answers: 0',
  'Count of incorrect Self Energy answers: 1',
  'Count of correct Intra Energy answers: 0',
  'Count of incorrect Intra Energy answers: 1',
  '',
  'Total correct answers: 0',
  'Total incorrect answers: 4'],
 'stderr': [],
 'return_code': 0}

In [152]:
import ast
import pandas as pd
import re
import subprocess

import ast

def replace_functions(source_code, df_functions, global_variables):
    """
    Replaces function definitions in `source_code` with those in `df_functions`,
    ensuring that dependent (child) functions are also included immediately before their parent functions.
    The global variables are inserted once at the very top before all functions, but only those not already present.

    Args:
        source_code (str): The original script as a string.
        df_functions (pd.DataFrame): DataFrame containing function names, code, and dependencies.
        global_variables (dict): Dictionary of global variables to be included before functions.

    Returns:
        str: The modified source code with updated functions and global variables.
    """
    # List of function names to be skipped from replacement
    fixed_functions = {"extracting_positions", "create_dataframes", "compare_coulomb_energy"}

    tree = ast.parse(source_code)
    modified_code = source_code  # Start with the original code

    # A set of function names already in the source code (for adding missing functions)
    existing_functions = {node.name for node in tree.body if isinstance(node, ast.FunctionDef)}

    # Extract the existing global variables from the source code
    existing_globals = set()
    for node in tree.body:
        if isinstance(node, ast.Assign):  # Look for assignments (global variables are usually assigned outside functions)
            for target in node.targets:
                if isinstance(target, ast.Name):
                    existing_globals.add(target.id)

    # First, process child functions
    added_functions = {}  # Keep track of where each function's code should be added

    for _, row in df_functions.iterrows():
        called_functions_dict = row["called_functions_dict"]  # Dictionary of dependencies
        function_name = row["function_name"]

        # Skip fixed functions
        if function_name in fixed_functions:
            continue

        if called_functions_dict:
            for child_name, child_code in called_functions_dict.items():
                if child_name not in existing_functions and child_name not in added_functions:
                    # Add child function immediately before its parent function
                    added_functions[child_name] = child_code

    # Now, replace the functions with their updated versions
    for _, row in df_functions.iterrows():
        function_name = row["function_name"]
        updated_func_code = row["func_code"]
        called_functions_dict = row["called_functions_dict"]  # Dictionary of dependencies

        # Skip fixed functions
        if function_name in fixed_functions:
            continue

        # Replace the function definition in the source code
        for node in tree.body:
            if isinstance(node, ast.FunctionDef) and node.name == function_name:
                old_func_code = ast.get_source_segment(source_code, node)

                # Insert the function code in place of the old one
                modified_code = modified_code.replace(old_func_code, updated_func_code)

                # Also insert child functions immediately before the parent function
                if called_functions_dict:
                    for child_name, child_code in called_functions_dict.items():
                        if child_name in added_functions:
                            # Insert the child function code right before the parent function
                            modified_code = modified_code.replace(updated_func_code, added_functions[child_name] + "\n\n" + updated_func_code)

    # Insert global variables once before all function definitions, but only if they are not already present
    if isinstance(global_variables, dict):
        # Format global variables as code, only adding those not already in the source code
        global_code = "\n".join(f"{var} = {repr(global_variables[var])}" for var in global_variables if var not in existing_globals)
    else:
        raise ValueError("global_variables must be a dictionary.")

    # Find the first function definition and insert the global variables before it
    function_start_index = None
    for idx, line in enumerate(modified_code.splitlines()):
        if line.strip().startswith("def "):  # Check for the first function definition
            function_start_index = idx
            break

    if function_start_index is not None:
        # Insert the global variables just before the first function definition
        modified_code_lines = modified_code.splitlines()
        modified_code_lines.insert(function_start_index, global_code)
        modified_code = "\n".join(modified_code_lines)

    return modified_code



import ast
import pandas as pd
import re

def extract_functions_with_dependencies_from_text(text):
    """
    Extracts Python functions and global variables from a string containing Python code and other text.
    Identifies function calls inside each function, extracts docstrings, and identifies global variables.

    Args:
        text (str): The input text that contains Python code along with other text.

    Returns:
        pd.DataFrame: DataFrame with 'function_name', 'func_code', 'docstring', 'global_variables', and 'called_functions_dict'.
    """
    try:
        # First, try extracting Python code between ```...``` blocks
        python_code1 = "\n".join(re.findall(r'```(.*?)```', text, re.DOTALL))

        # Then, try extracting Python code between ----...---- blocks
        python_code2 = "\n".join(re.findall(r'[-]{2,}(.*?)\n[-]{2,}', text, re.DOTALL))

        # Then, try extracting Python code between '''...''' blocks
        python_code3 = "\n".join(re.findall(r"'''(.*?)'''", text, re.DOTALL)).strip()

        # If either python_code1 or python_code2 is found, select the first non-empty result
        if python_code2 or python_code1 or python_code3:
            python_code = python_code1 or python_code2
        else:
            raise ValueError("No Python code found in the provided text.")
        
    except Exception as e:
        print("Error:", e)
        return None, None
    
    print(python_code)
    
    # Parse the Python code using AST
    tree = ast.parse(python_code)
    functions_data = {}

    # First, extract all function definitions
    function_codes = {}
    function_docstrings = {}

    for node in tree.body:
        if isinstance(node, ast.FunctionDef):
            function_codes[node.name] = ast.get_source_segment(python_code, node)
            
            # Extract docstring using AST's method
            docstring = ast.get_docstring(node)
            if docstring:
                function_docstrings[node.name] = docstring
            else:
                function_docstrings[node.name] = None

    # Extract global variables (variables in the global scope)
    global_variables = {}
    for node in tree.body:
        if isinstance(node, ast.Assign):  # Global variable assignment
            for target in node.targets:
                if isinstance(target, ast.Name):  # Only name targets are considered variables
                    # Extract the value of the global variable
                    value = ast.literal_eval(node.value) if isinstance(node.value, ast.Constant) else None
                    global_variables[target.id] = value

    # Now, extract function calls and variables within each function
    for node in tree.body:
        if isinstance(node, ast.FunctionDef):
            function_name = node.name
            function_code = function_codes[function_name]
            function_docstring = function_docstrings[function_name]

            # Find all function calls inside this function
            called_functions_dict = {}
            for child in ast.walk(node):  # Traverse the function body
                if isinstance(child, ast.Call) and isinstance(child.func, ast.Name):
                    called_func_name = child.func.id  # Get called function name
                    if called_func_name in function_codes:  # Only store if it's a defined function
                        called_functions_dict[called_func_name] = function_codes[called_func_name]

            # Store function details
            functions_data[function_name] = {
                "func_code": function_code,
                "docstring": function_docstring,
                "called_functions_dict": called_functions_dict if called_functions_dict else None
            }

    # Convert to DataFrame for functions
    df = pd.DataFrame([
        {
            "function_name": key,
            "func_code": value["func_code"],
            "docstring": value["docstring"],
            "called_functions_dict": value["called_functions_dict"]
        }
        for key, value in functions_data.items()
    ])

    return df, global_variables

def execute_code(code: str) -> dict:
    """
    Executes a Python code string and captures the output, errors, and return code.

    Args:
        code (str): The Python code to execute.

    Returns:
        dict: Dictionary with 'stdout', 'stderr', and 'return_code'.
    """
    try:
        process = subprocess.Popen(
            ["/Users/pulicharishma/anaconda3/bin/python", "-c", code],
            text=True,
            stdout=subprocess.PIPE,
            stderr=subprocess.PIPE
        )
        output, errors = process.communicate()
        
        return {
            "stdout": output.strip().split("\n") if output else [],
            "stderr": errors.strip().split("\n") if errors else [],
            "return_code": process.returncode
        }
    except Exception as e:
        return {"error": str(e)}


In [ ]:
text = """Below is the modified code with the four missing functions filled in. In each function the calculations follow the standard Ewald summation formulation (using the provided formula) and then convert the energy to Kelvin by dividing by kB. You can run one iteration at a time. Note that only the sections marked “#--- Complete this code ---#” have been modified.

------------------------------------------------------------
# (Begin modified code)
import numpy as np
import pandas as pd
from scipy.special import erfc, erf
import math 

# defining all variables
atom_properties = {
    'O': {'type': 'O', 'sigma': 3.165558, 'epsilon': 78.197431, 'charge': -0.8476, 'num_particles': 1},
    'H': {'type': 'H', 'sigma': 0.000000, 'epsilon': 0.000000, 'charge': 0.4238, 'num_particles': 2},
}

# Trying for small configurations first
file_paths = [
#        'spce_sample_config_periodic4.txt',
#        'spce_sample_config_periodic2.txt',
#        'spce_sample_config_periodic3.txt',
        '../data/spce_sample_config_periodic1_modified.txt'
    ]

NIST_SPC_E_Water = {
        'Configuration': [1, 2, 3, 4],
        'M (number of SPC/E molecules)': [100, 200, 300, 750],
        'Lx=Ly=Lz (Å)': [20.0, 20.0, 20.0, 30.0],
        'Edisp/kB (K)': [9.95387E+04, 1.93712E+05, 3.54344E+05, 4.48593E+05],
        'ELRC/kB (K)': [-8.23715E+02, -3.29486E+03, -7.41343E+03, -1.37286E+04],
        'Ereal/kB (K)': [-5.58889E+05, -1.19295E+06, -1.96297E+06, -3.57226E+06],
        'Efourier/kB (K)': [6.27009E+03, 6.03495E+03, 5.24461E+03, 7.58785E+03],
        'Eself/kB (K)': [-2.84469E+06, -5.68938E+06, -8.53407E+06, -1.42235E+07],
        'Eintra/kB (K)': [2.80999E+06, 5.61998E+06, 8.42998E+06, 1.41483E+07],
        'Etotal/kB (K)': [-4.88604E+05, -1.06590E+06, -1.71488E+06, -3.20501E+06]
    }

# Data processing
def extracting_positions(input_file):
    # Extract the positions from the .xyz file
    with open(input_file, "r") as file:
        lines = file.readlines()

    data_lines = lines[2:]

    data_list = []
    for line in data_lines:
        stripped_line = line.strip()
        parts = stripped_line.split()
        if len(parts) >= 5:  
            try:
                x, y, z = map(float, parts[1:4])
                atom_type = parts[4]
                data_list.append([x, y, z, atom_type])
            except ValueError:
                continue  

    # Create a DataFrame with all configurations
    columns = ["X", "Y", "Z", "Atom Type"]
    configuration = pd.DataFrame(data_list, columns=columns)

    configuration.index = range(1, len(configuration) + 1)

    configuration["Molecule"] = ((configuration.index - 1) // 3) + 1
    
    return configuration

# create the target dataframes
def creating_dataframes(file_paths, atom_properties,NIST_SPC_E_Water):
    
    # Create the NIST_SPC_E_Water dataframe
    NIST_SPC_E_Water = pd.DataFrame(NIST_SPC_E_Water)
    
    NIST_SPC_E_Water['Sum of energies'] = (NIST_SPC_E_Water['Edisp/kB (K)'] + NIST_SPC_E_Water['ELRC/kB (K)'] +
                             NIST_SPC_E_Water['Ereal/kB (K)'] + NIST_SPC_E_Water['Efourier/kB (K)'] +
                             NIST_SPC_E_Water['Eself/kB (K)'] + NIST_SPC_E_Water['Eintra/kB (K)'])

    # Creating the force_field dataframe
    force_field = pd.DataFrame(atom_properties).from_dict(atom_properties, orient='index')

    # Create the system dataframe contaning some variables
    system = pd.DataFrame(file_paths, columns=["file_paths"])

    system['configuration #'] = system['file_paths'].str.extract(r'(\d+)').astype(int)

    system[["number of particles", "box length"]] = system["configuration #"].apply(
    lambda x: pd.Series({
        "number of particles": float(NIST_SPC_E_Water.loc[NIST_SPC_E_Water["Configuration"] == x, 
                                                          "M (number of SPC/E molecules)"].values[0]),
        "box length": float(NIST_SPC_E_Water.loc[NIST_SPC_E_Water["Configuration"] == x, 
                                                 "Lx=Ly=Lz (Å)"].values[0])}))

    system['cutoff'] = 10
    system['alpha'] = 5.6
    system['kmax'] = 5
    system['ε0'] = float(8.854187817E-12)
    system['kB'] = float(1.3806488E-23)
        
    return system, force_field, NIST_SPC_E_Water

def compute_real_energies(system_data, configuration, force_field):
    # Compute the real space energy of Coulomb interactions
    real_energy = 0.0

    #--- Complete this code ---#
    cutoff = system_data['cutoff']
    alpha = system_data['alpha']
    epsilon0 = system_data['ε0']
    kB = system_data['kB']
    
    # Extract atomic positions and corresponding charges
    positions = configuration[['X', 'Y', 'Z']].to_numpy()
    # Map each atom type to its charge from the force_field DataFrame
    charges = configuration['Atom Type'].apply(lambda at: force_field.loc[at, 'charge']).to_numpy()

    num_atoms = len(charges)
    # Sum over unique pairs (j < l)
    for j in range(num_atoms):
        for l in range(j+1, num_atoms):
            rij_vec = positions[j] - positions[l]
            rij = np.linalg.norm(rij_vec)
            if rij < cutoff and rij != 0:
                # real-space potential contribution from pair (j,l)
                pair_term = (charges[j]*charges[l])/(4*math.pi*epsilon0) * erfc(alpha*rij)/rij
                real_energy += pair_term
    
    # Convert to Kelvin (energy divided by Boltzmann constant)
    real_energy = real_energy / kB
    return real_energy

def compute_fourier_energies(system_data, configuration, force_field):
    # Compute the Fourier space (reciprocal space) energy contribution
    fourier_energy = 0.0

    #--- Complete this code ---#
    alpha = system_data['alpha']
    epsilon0 = system_data['ε0']
    kB = system_data['kB']
    kmax = int(system_data['kmax'])
    box_length = system_data['box length']
    volume = box_length**3
    
    # Extract positions and charges
    positions = configuration[['X', 'Y', 'Z']].to_numpy()
    charges = configuration['Atom Type'].apply(lambda at: force_field.loc[at, 'charge']).to_numpy()

    # Loop over reciprocal lattice vectors. Here we consider integer vectors (kx,ky,kz)
    # and include only those with k^2 < 27 (as given in the description).
    for kx in range(-kmax, kmax+1):
        for ky in range(-kmax, kmax+1):
            for kz in range(-kmax, kmax+1):
                # Skip the zero vector
                if kx == 0 and ky == 0 and kz == 0:
                    continue
                # Compute the squared magnitude of the k-vector (using integer indices)
                k_squared_int = kx**2 + ky**2 + kz**2
                if k_squared_int >= 27:
                    continue
                # Note: In standard Ewald, the physical k-vector is scaled by (2*pi/box_length). 
                # However, the provided formula uses the integer k directly in the exponential.
                # We follow the provided formula:
                k_mod = math.sqrt(k_squared_int)
                exp_term = math.exp(- (math.pi * k_mod/alpha)**2)
                # Compute the structure factor S(k)
                real_sum = 0.0
                imag_sum = 0.0
                for j in range(len(charges)):
                    # Using 2pi * k dot r_j / box_length as argument
                    arg = 2*math.pi*(kx*positions[j,0] + ky*positions[j,1] + kz*positions[j,2])/box_length
                    real_sum += charges[j] * math.cos(arg)
                    imag_sum += charges[j] * math.sin(arg)
                S_k_sq = real_sum**2 + imag_sum**2
                # Accumulate Fourier energy: the formula uses 1/k^2 factor (with k^2 = k_squared_int)
                fourier_energy += (1.0/(k_squared_int))*exp_term * S_k_sq

    # Multiply prefactors as given in the formula: 1/(2*pi*V) * 1/(4*pi*epsilon0)
    fourier_energy *= (1.0/(2*math.pi*volume))*(1.0/(4*math.pi*epsilon0))
    
    # Convert result to Kelvin
    fourier_energy = fourier_energy / kB
    return fourier_energy

def compute_self_energies(system_data, configuration, force_field):
    # Compute the self-energy correction (each atom interacts with its own smeared charge)
    self_energy = 0.0

    #--- Complete this code ---#
    alpha = system_data['alpha']
    epsilon0 = system_data['ε0']
    kB = system_data['kB']
    
    charges = configuration['Atom Type'].apply(lambda at: force_field.loc[at, 'charge']).to_numpy()
    
    for q in charges:
        self_energy += (q**2)
    # Self energy factor: -alpha/sqrt(pi)/(4*pi*epsilon0)
    self_energy = - (alpha/math.sqrt(math.pi))/(4*math.pi*epsilon0) * self_energy
    self_energy = self_energy / kB
    return self_energy


def compute_intra_energies(system_data, configuration, force_field):
    # Compute the intramolecular energy correction (exclude self interactions within the same molecule)
    intra_energy = 0.0

    #--- Complete this code ---#
    epsilon0 = system_data['ε0']
    kB = system_data['kB']
    alpha = system_data['alpha']
    
    # Group the configuration by molecule index.
    grouped = configuration.groupby("Molecule")
    # For each molecule, calculate pair interactions between atoms within the molecule.
    for molecule, group in grouped:
        positions = group[['X', 'Y', 'Z']].to_numpy()
        # Get charges for the atoms in this molecule
        charges = group['Atom Type'].apply(lambda at: force_field.loc[at, 'charge']).to_numpy()
        num_atoms = len(charges)
        # Sum over unique pairs (κ < λ)
        for i in range(num_atoms):
            for j in range(i+1, num_atoms):
                rij_vec = positions[i] - positions[j]
                rij = np.linalg.norm(rij_vec)
                if rij != 0:
                    # Use the error function contribution for intramolecular interactions.
                    pair_term = (charges[i]*charges[j])/(4*math.pi*epsilon0) * erf(alpha*rij)/rij
                    intra_energy += pair_term
    # The intramolecular contribution subtracts the interactions already accounted for,
    # so we take a minus sign.
    intra_energy = - intra_energy
    intra_energy = intra_energy / kB
    return intra_energy

# DataFrame Descriptions:
# 1. NIST_SPC_E_Water DataFrame: ...
# 2. force_field DataFrame: ...
# 3. system DataFrame: ...
# 4. configuration DataFrame: ...

system, force_field, NIST_SPC_E_Water = creating_dataframes(file_paths, atom_properties,NIST_SPC_E_Water)

# Computing energies storing in results
results = pd.DataFrame()

results['Number of Particles'] = system['number of particles'].astype(int)

# Calculate pairwise energy for all system configurations
results['real_energies'] = system['file_paths'].apply(
    lambda file_path: compute_real_energies(
        system[system['file_paths'] == file_path].iloc[0],  # Ensure single row selection
        extracting_positions(file_path),
        force_field
    )
)

# Calculate pairwise energy for all system configurations
results['fourier_energies'] = system['file_paths'].apply(
    lambda file_path: compute_fourier_energies(
        system[system['file_paths'] == file_path].iloc[0],  # Ensure single row selection
        extracting_positions(file_path),
        force_field
    )
)

# Calculate pairwise energy for all system configurations
results['self_energies'] = system['file_paths'].apply(
    lambda file_path: compute_self_energies(
        system[system['file_paths'] == file_path].iloc[0],  # Ensure single row selection
        extracting_positions(file_path),
        force_field
    )
)

# Calculate pairwise energy for all system configurations
results['intra_energies'] = system['file_paths'].apply(
    lambda file_path: compute_intra_energies(
        system[system['file_paths'] == file_path].iloc[0],  # Ensure single row selection
        extracting_positions(file_path),
        force_field
    )
)

def compare_coulomb_energy(df1, df2, tolerance=1e-4):
    # Merge df1 and df2 based on the number of particles
    df_merged = df1.merge(df2, left_on='Number of Particles', right_on='M (number of SPC/E molecules)', how='left')

    # Initialize counters
    matched_real = matched_fourier = matched_self = matched_intra = 0
    not_matched_real = not_matched_fourier = not_matched_self = not_matched_intra = 0

    # Initialize output lists
    real_energy_output, fourier_energy_output = [], []
    self_energy_output, intra_energy_output = [], []

    # Iterate over merged DataFrame
    for idx, row in df_merged.iterrows():
        # Extract computed values from df1
        real_energy = row['real_energies']
        fourier_energy = row['fourier_energies']
        self_energy = row['self_energies']
        intra_energy = row['intra_energies']
        num_molecules = row['Number of Particles']

        # Extract reference values from df2
        if pd.isna(row['Ereal/kB (K)']):
            continue  # Skip if no match is found in df2

        nist_real_energy = float(row['Ereal/kB (K)'])
        nist_fourier_energy = float(row['Efourier/kB (K)'])
        nist_self_energy = float(row['Eself/kB (K)'])
        nist_intra_energy = float(row['Eintra/kB (K)'])

        # Perform numeric comparisons with a tolerance
        match_real = np.isclose(real_energy, nist_real_energy, atol=tolerance)
        match_fourier = np.isclose(fourier_energy, nist_fourier_energy, atol=tolerance)
        match_self = np.isclose(self_energy, nist_self_energy, atol=tolerance)
        match_intra = np.isclose(intra_energy, nist_intra_energy, atol=tolerance)

        matched_real += int(match_real)
        not_matched_real += int(not match_real)

        matched_fourier += int(match_fourier)
        not_matched_fourier += int(not match_fourier)

        matched_self += int(match_self)
        not_matched_self += int(not match_self)

        matched_intra += int(match_intra)
        not_matched_intra += int(not match_intra)

        # Store formatted outputs
        real_energy_output.append(f"Test {idx+1} ({num_molecules} molecules): Computed: {real_energy:.4E}, NIST: {nist_real_energy:.4E}, Match: {match_real}")
        fourier_energy_output.append(f"Test {idx+1} ({num_molecules} molecules): Computed: {fourier_energy:.4E}, NIST: {nist_fourier_energy:.4E}, Match: {match_fourier}")
        self_energy_output.append(f"Test {idx+1} ({num_molecules} molecules): Computed: {self_energy:.4E}, NIST: {nist_self_energy:.4E}, Match: {match_self}")
        intra_energy_output.append(f"Test {idx+1} ({num_molecules} molecules): Computed: {intra_energy:.4E}, NIST: {nist_intra_energy:.4E}, Match: {match_intra}")

    # Print final results
    print("Real Energy Comparison:")
    print(*real_energy_output, sep=chr(10))
    print("Fourier Energy Comparison:")
    print(*fourier_energy_output, sep=chr(10))
    print("Self Energy Comparison:")
    print(*self_energy_output, sep=chr(10))
    print("Intra Energy Comparison:")
    print(*intra_energy_output, sep=chr(10))
    print()
    print(f"Count of correct Real Energy answers: {matched_real}")
    print(f"Count of incorrect Real Energy answers: {not_matched_real}")
    print(f"Count of correct Fourier Energy answers: {matched_fourier}")
    print(f"Count of incorrect Fourier Energy answers: {not_matched_fourier}")
    print(f"Count of correct Self Energy answers: {matched_self}")
    print(f"Count of incorrect Self Energy answers: {not_matched_self}")
    print(f"Count of correct Intra Energy answers: {matched_intra}")
    print(f"Count of incorrect Intra Energy answers: {not_matched_intra}")
    print()

    total_correct = matched_real + matched_fourier + matched_self + matched_intra
    total_incorrect = not_matched_real + not_matched_fourier + not_matched_self + not_matched_intra

    print(f"Total correct answers: {total_correct}")
    print(f"Total incorrect answers: {total_incorrect}")

# calling compare_coulomb_energy function
compare_coulomb_energy(results, NIST_SPC_E_Water)
# (End modified code)
------------------------------------------------------------

Key reasoning and details:
1. In compute_real_energies, we sum over distinct pairs and include only interactions with separation less than the cutoff. The erfc function is used per the formula.
2. In compute_fourier_energies, we loop over reciprocal space vectors (using integer indices) and include only k‐vectors satisfying k² < 27. The structure factor is computed via a sum over atoms using sine and cosine functions.
3. In compute_self_energies, the self-correction term is computed directly from the sum of q² for each atom.
4. In compute_intra_energies, the intramolecular contributions within each molecule are computed and subtracted."""

In [147]:
text1 = """'''
import numpy as np
import pandas as pd
from scipy.special import erfc, erf
import math

# Coulomb constant factor in units of Kelvin·Å: (1/(4πε0))*e^2/kB
# Using: 1/(4πε0) ~ 8.987551787e9 [N·m²/C²], e = 1.60217662e-19 C, kB = 1.3806488e-23 J/K.
# Converting distance from meters to Å (1 m = 1e10 Å):
# factor = (8.987551787e9 * (1.60217662e-19)**2 / 1.3806488e-23) / 1e-10 
# This yields approximately:
coulomb_constant = 1.67e5  # in Kelvin·Å

def compute_real_energies(system_data, configuration, force_field):
    # Compute the real-space term Ereal of the Coulomb Ewald summation.
    # Only include interactions between atoms in different molecules.
    #
    # E_real = sum_{i<j, inter-molecular} (q_i*q_j * coulomb_constant * erfc(alpha*r_ij))/r_ij,
    # with r_ij in Å, and only for pairs with r_ij < cutoff.
    
    real_energy = 0.0
    alpha = system_data['alpha']  # in 1/Å
    cutoff = system_data['cutoff']  # in Å
    
    # Reset index for iteration over configuration rows
    config = configuration.reset_index(drop=True)
    num_atoms = len(config)
    
    # Loop over atom pairs (i < j)
    for i in range(num_atoms - 1):
        xi, yi, zi, atom_type_i = config.loc[i, ["X", "Y", "Z", "Atom Type"]]
        qi = force_field.loc[atom_type_i, "charge"]
        for j in range(i+1, num_atoms):
            xj, yj, zj, atom_type_j = config.loc[j, ["X", "Y", "Z", "Atom Type"]]
            qj = force_field.loc[atom_type_j, "charge"]
            # Only consider inter-molecular interactions in real term
            mol_i = config.loc[i, "Molecule"]
            mol_j = config.loc[j, "Molecule"]
            if mol_i == mol_j:
                continue  # skip intramolecular pairs; they will be handled in Eintra
            
            dx = xi - xj
            dy = yi - yj
            dz = zi - zj
            r = math.sqrt(dx*dx + dy*dy + dz*dz)
            if r > cutoff or r == 0:
                continue
            contribution = (qi * qj * coulomb_constant * erfc(alpha * r)) / r
            real_energy += contribution
    return real_energy

def compute_fourier_energies(system_data, configuration, force_field):
    # Compute the Fourier-space term Efourier of the Coulomb Ewald summation.
    #
    # We use the following discrete summation:
    # E_fourier = 1/(2π V) * sum_{k≠0, with integer k indices satisfying n^2 < kmax^2 +2} 
    #    [1/k^2 * exp( - (π*|k|/alpha)^2) * coulomb_constant * |S(k)|^2],
    # where S(k) = sum_{j=1}^N q_j * exp(2π i k·r_j / L),
    # V is the volume = L^3 with L in Å.
    #
    # Note: k (wavevector) is constructed from integer triplets (n_x,n_y,n_z) with
    # n_x, n_y, n_z in [-kmax, kmax] (kmax given in system_data). The condition n^2 < kmax^2+2 is applied.
    
    fourier_energy = 0.0
    L = system_data['box length']  # in Å
    volume = L**3  # in Å^3
    alpha = system_data['alpha']  # in 1/Å
    kmax_int = int(system_data['kmax'])
    
    # Prepare positions and charges from configuration
    config = configuration.reset_index(drop=True)
    positions = config[['X', 'Y', 'Z']].to_numpy()  # in Å
    charges = np.array([force_field.loc[atom_type, "charge"] for atom_type in config["Atom Type"]])
    
    # Loop over integer triplets for wavevector indices
    for nx in range(-kmax_int, kmax_int+1):
        for ny in range(-kmax_int, kmax_int+1):
            for nz in range(-kmax_int, kmax_int+1):
                # Skip the zero vector
                if nx == 0 and ny == 0 and nz == 0:
                    continue
                # Only include those with (n_x^2 + n_y^2 + n_z^2) < kmax^2 + 2
                if (nx**2 + ny**2 + nz**2) >= (kmax_int**2 + 2):
                    continue
                # Construct k-vector (in 1/Å) using the standard convention: k = 2π * n / L
                k_vector = (2 * np.pi / L) * np.array([nx, ny, nz])
                ksq = np.dot(k_vector, k_vector)
                # Structure factor S(k)
                # Use phase = 2π*(n_x*x + n_y*y + n_z*z)/L = dot(k_vector, r) i.e. k_vector already contains 2π/L factor.
                phases = np.dot(positions, k_vector)
                S_k = np.sum(charges * np.exp(1j * phases))
                # Exponential damping factor: exp[-(π |k| / alpha)^2]
                exp_factor = np.exp(- (np.pi * np.linalg.norm(k_vector) / alpha)**2)
                term = (1.0 / (2 * np.pi * volume)) * (1.0 / ksq) * exp_factor * coulomb_constant * (np.abs(S_k)**2)
                fourier_energy += term
    return fourier_energy

def compute_self_energies(system_data, configuration, force_field):
    # Compute the self-energy correction term Eself.
    #
    # E_self = - (alpha/√π) * coulomb_constant * sum_{j} (q_j^2)
    #
    # Here we sum over all atoms in the configuration.
    
    self_energy = 0.0
    alpha = system_data['alpha']
    config = configuration.reset_index(drop=True)
    
    for idx, row in config.iterrows():
        atom_type = row["Atom Type"]
        q = force_field.loc[atom_type, "charge"]
        self_energy += q*q
    self_energy *= - (alpha / math.sqrt(np.pi)) * coulomb_constant
    return self_energy

def compute_intra_energies(system_data, configuration, force_field):
    # Compute the intramolecular energy correction term Eintra.
    #
    # Eintra = - sum_{molecule} sum_{i<j in molecule} (q_i * q_j * coulomb_constant * erf(alpha*r_ij))/r_ij,
    # where r_ij is the distance between atoms i and j within the same molecule.
    
    intra_energy = 0.0
    alpha = system_data['alpha']
    
    # Group configuration by molecule.
    grouped = configuration.groupby("Molecule")
    
    for mol, group in grouped:
        group = group.reset_index(drop=True)
        num_atoms = len(group)
        for i in range(num_atoms - 1):
            xi, yi, zi, atom_type_i = group.loc[i, ["X", "Y", "Z", "Atom Type"]]
            qi = force_field.loc[atom_type_i, "charge"]
            for j in range(i+1, num_atoms):
                xj, yj, zj, atom_type_j = group.loc[j, ["X", "Y", "Z", "Atom Type"]]
                qj = force_field.loc[atom_type_j, "charge"]
                dx = xi - xj
                dy = yi - yj
                dz = zi - zj
                r = math.sqrt(dx*dx + dy*dy + dz*dz)
                # Avoid division by zero
                if r == 0:
                    continue
                contribution = (qi * qj * coulomb_constant * erf(alpha * r)) / r
                # Note the negative sign in the formula
                intra_energy -= contribution
    return intra_energy
''' 
# (The rest of the code remains unchanged.)
"""

In [146]:
text2 = """================================== Ai Message ==================================

'''
import numpy as np
import pandas as pd
from scipy.special import erfc, erf
import math 

# defining all variables
atom_properties = {
    'O': {'type': 'O', 'sigma': 3.165558, 'epsilon': 78.197431, 'charge': -0.8476, 'num_particles': 1},
    'H': {'type': 'H', 'sigma': 0.000000, 'epsilon': 0.000000, 'charge': 0.4238, 'num_particles': 2},
}

# Trying for small configurations first
file_paths = [
#        'spce_sample_config_periodic4.txt',
#        'spce_sample_config_periodic2.txt',
#        'spce_sample_config_periodic3.txt',
        '../data/spce_sample_config_periodic1_modified.txt'
]

NIST_SPC_E_Water = {
    'Configuration': [1, 2, 3, 4],
    'M (number of SPC/E molecules)': [100, 200, 300, 750],
    'Lx=Ly=Lz (Å)': [20.0, 20.0, 20.0, 30.0],
    'Edisp/kB (K)': [9.95387E+04, 1.93712E+05, 3.54344E+05, 4.48593E+05],
    'ELRC/kB (K)': [-8.23715E+02, -3.29486E+03, -7.41343E+03, -1.37286E+04],
    'Ereal/kB (K)': [-5.58889E+05, -1.19295E+06, -1.96297E+06, -3.57226E+06],
    'Efourier/kB (K)': [6.27009E+03, 6.03495E+03, 5.24461E+03, 7.58785E+03],
    'Eself/kB (K)': [-2.84469E+06, -5.68938E+06, -8.53407E+06, -1.42235E+07],
    'Eintra/kB (K)': [2.80999E+06, 5.61998E+06, 8.42998E+06, 1.41483E+07],
    'Etotal/kB (K)': [-4.88604E+05, -1.06590E+06, -1.71488E+06, -3.20501E+06]
}

# Data processing
def extracting_positions(input_file):
    # Extract the positions from the .xyz file
    with open(input_file, "r") as file:
        lines = file.readlines()
    data_lines = lines[2:]
    data_list = []
    for line in data_lines:
        stripped_line = line.strip()
        parts = stripped_line.split()
        if len(parts) >= 5:
            try:
                x, y, z = map(float, parts[1:4])
                atom_type = parts[4]
                data_list.append([x, y, z, atom_type])
            except ValueError:
                continue  
    columns = ["X", "Y", "Z", "Atom Type"]
    configuration = pd.DataFrame(data_list, columns=columns)
    configuration.index = range(1, len(configuration) + 1)
    configuration["Molecule"] = ((configuration.index - 1) // 3) + 1
    return configuration

# create the target dataframes
def creating_dataframes(file_paths, atom_properties, NIST_SPC_E_Water):
    NIST_SPC_E_Water = pd.DataFrame(NIST_SPC_E_Water)
    NIST_SPC_E_Water['Sum of energies'] = (
        NIST_SPC_E_Water['Edisp/kB (K)'] +
        NIST_SPC_E_Water['ELRC/kB (K)'] +
        NIST_SPC_E_Water['Ereal/kB (K)'] +
        NIST_SPC_E_Water['Efourier/kB (K)'] +
        NIST_SPC_E_Water['Eself/kB (K)'] +
        NIST_SPC_E_Water['Eintra/kB (K)']
    )
    force_field = pd.DataFrame(atom_properties).from_dict(atom_properties, orient='index')
    system = pd.DataFrame(file_paths, columns=["file_paths"])
    system['configuration #'] = system['file_paths'].str.extract(r'(\d+)').astype(int)
    system[["number of particles", "box length"]] = system["configuration #"].apply(
        lambda x: pd.Series({
            "number of particles": float(NIST_SPC_E_Water.loc[NIST_SPC_E_Water["Configuration"] == x, "M (number of SPC/E molecules)"].values[0]),
            "box length": float(NIST_SPC_E_Water.loc[NIST_SPC_E_Water["Configuration"] == x, "Lx=Ly=Lz (Å)"].values[0])
        })
    )
    system['cutoff'] = 10
    system['alpha'] = 5.6
    system['kmax'] = 5
    system['ε0'] = float(8.854187817E-12)
    system['kB'] = float(1.3806488E-23)
    return system, force_field, NIST_SPC_E_Water

def compute_real_energies(system_data, configuration, force_field):
    # Compute the real energy contribution in the Coulomb Ewald summation.
    # Here we sum over unique pairs of atoms that belong to different molecules and are within the cutoff.
    real_energy = 0.0
    cutoff_ang = system_data['cutoff']
    cutoff = cutoff_ang * 1e-10          # convert Å to meters
    alpha = system_data['alpha'] * 1e10    # convert 1/Å to 1/m
    eps0 = system_data['ε0']
    kB = system_data['kB']
    e_charge = 1.602176634e-19           # elementary charge in Coulombs
    
    positions = configuration[['X','Y','Z']].to_numpy() * 1e-10
    charges = []
    for atype in configuration['Atom Type']:
        q = force_field.loc[atype, 'charge']
        charges.append(q * e_charge)
    charges = np.array(charges)
    
    N = len(positions)
    for i in range(N-1):
        for j in range(i+1, N):
            # include only if atoms are in different molecules
            if configuration.iloc[i]['Molecule'] == configuration.iloc[j]['Molecule']:
                continue
            r_vec = positions[j] - positions[i]
            r = np.linalg.norm(r_vec)
            if r < cutoff and r > 0:
                contribution = (charges[i]*charges[j])/(4*math.pi*eps0) * erfc(alpha*r)/r
                real_energy += contribution
    return real_energy/kB

def compute_fourier_energies(system_data, configuration, force_field):
    # Compute the Fourier space energy contribution in the Coulomb Ewald summation.
    fourier_energy = 0.0
    kB = system_data['kB']
    eps0 = system_data['ε0']
    e_charge = 1.602176634e-19
    kmax = int(system_data['kmax'])
    box_length_ang = system_data['box length']
    box_length = box_length_ang * 1e-10   # convert Å to meters
    V = box_length**3
    alpha = system_data['alpha'] * 1e10    # convert 1/Å to 1/m
    
    positions = configuration[['X','Y','Z']].to_numpy() * 1e-10
    charges = []
    for atype in configuration['Atom Type']:
        q = force_field.loc[atype, 'charge']
        charges.append(q*e_charge)
    charges = np.array(charges)
    
    for n1 in range(-kmax, kmax+1):
        for n2 in range(-kmax, kmax+1):
            for n3 in range(-kmax, kmax+1):
                if n1==0 and n2==0 and n3==0:
                    continue
                n_sq = n1**2 + n2**2 + n3**2
                if n_sq >= (kmax**2 + 2):
                    continue
                n_vec = np.array([n1,n2,n3])
                k_vec = (2*math.pi*n_vec)/box_length
                k_sq = np.dot(k_vec,k_vec)
                S_k = 0.0 + 0.0j
                for idx, r in enumerate(positions):
                    phase = np.dot(k_vec, r)
                    S_k += charges[idx]*np.exp(1j*phase)
                damping = np.exp(-k_sq/(4*alpha**2))
                term = (1/k_sq)*damping*(abs(S_k)**2)
                fourier_energy += term
    prefactor = 1/(2*math.pi*V) * 1/(4*math.pi*eps0)
    fourier_energy *= prefactor
    return fourier_energy/kB

def compute_self_energies(system_data, configuration, force_field):
    # Compute the self-energy correction term of the Coulomb Ewald summation.
    self_energy = 0.0
    eps0 = system_data['ε0']
    kB = system_data['kB']
    e_charge = 1.602176634e-19
    alpha = system_data['alpha'] * 1e10   # convert 1/Å to 1/m

    for atype in configuration['Atom Type']:
        q = force_field.loc[atype, 'charge'] * e_charge
        self_energy += q**2
    self_energy = - (alpha/math.sqrt(math.pi))/(4*math.pi*eps0) * self_energy
    return self_energy/kB

def compute_intra_energies(system_data, configuration, force_field):
    # Compute the intramolecular energy contribution in the Coulomb Ewald summation.
    intra_energy = 0.0
    eps0 = system_data['ε0']
    kB = system_data['kB']
    e_charge = 1.602176634e-19
    alpha = system_data['alpha'] * 1e10  # convert 1/Å to 1/m

    positions = configuration[['X','Y','Z']].to_numpy() * 1e-10
    charges = []
    for atype in configuration['Atom Type']:
        q = force_field.loc[atype, 'charge'] * e_charge
        charges.append(q)
    charges = np.array(charges)
    
    molecule_ids = configuration['Molecule'].unique()
    for mol in molecule_ids:
        indices = configuration.index[configuration['Molecule'] == mol].tolist()
        n_atoms = len(indices)
        for i in range(n_atoms-1):
            for j in range(i+1, n_atoms):
                idx_i = indices[i]-1  # adjust for zero-indexing
                idx_j = indices[j]-1
                r_vec = positions[idx_j] - positions[idx_i]
                r = np.linalg.norm(r_vec)
                if r > 0:
                    term = - (charges[idx_i]*charges[idx_j])/(4*math.pi*eps0) * erf(alpha*r)/r
                    intra_energy += term
    return intra_energy/kB

# Create dataframes from input files and parameters
system, force_field, NIST_SPC_E_Water = creating_dataframes(file_paths, atom_properties, NIST_SPC_E_Water)

# Compute energies and store results
results = pd.DataFrame()
results['Number of Particles'] = system['number of particles'].astype(int)

results['real_energies'] = system['file_paths'].apply(
    lambda file_path: compute_real_energies(
        system[system['file_paths'] == file_path].iloc[0],
        extracting_positions(file_path),
        force_field
    )
)

results['fourier_energies'] = system['file_paths'].apply(
    lambda file_path: compute_fourier_energies(
        system[system['file_paths'] == file_path].iloc[0],
        extracting_positions(file_path),
        force_field
    )
)

results['self_energies'] = system['file_paths'].apply(
    lambda file_path: compute_self_energies(
        system[system['file_paths'] == file_path].iloc[0],
        extracting_positions(file_path),
        force_field
    )
)

results['intra_energies'] = system['file_paths'].apply(
    lambda file_path: compute_intra_energies(
        system[system['file_paths'] == file_path].iloc[0],
        extracting_positions(file_path),
        force_field
    )
)

def compare_coulomb_energy(df1, df2, tolerance=1e-4):
    df_merged = df1.merge(df2, left_on='Number of Particles', right_on='M (number of SPC/E molecules)', how='left')
    matched_real = matched_fourier = matched_self = matched_intra = 0
    not_matched_real = not_matched_fourier = not_matched_self = not_matched_intra = 0
    real_energy_output, fourier_energy_output = [], []
    self_energy_output, intra_energy_output = [], []
    
    for idx, row in df_merged.iterrows():
        real_energy = row['real_energies']
        fourier_energy = row['fourier_energies']
        self_energy = row['self_energies']
        intra_energy = row['intra_energies']
        num_molecules = row['Number of Particles']
        
        if pd.isna(row['Ereal/kB (K)']):
            continue
        
        nist_real_energy = float(row['Ereal/kB (K)'])
        nist_fourier_energy = float(row['Efourier/kB (K)'])
        nist_self_energy = float(row['Eself/kB (K)'])
        nist_intra_energy = float(row['Eintra/kB (K)'])
        
        match_real = np.isclose(real_energy, nist_real_energy, atol=tolerance)
        match_fourier = np.isclose(fourier_energy, nist_fourier_energy, atol=tolerance)
        match_self = np.isclose(self_energy, nist_self_energy, atol=tolerance)
        match_intra = np.isclose(intra_energy, nist_intra_energy, atol=tolerance)
        
        matched_real += int(match_real)
        not_matched_real += int(not match_real)
        matched_fourier += int(match_fourier)
        not_matched_fourier += int(not match_fourier)
        matched_self += int(match_self)
        not_matched_self += int(not match_self)
        matched_intra += int(match_intra)
        not_matched_intra += int(not match_intra)
        
        real_energy_output.append(f"Test {idx+1} ({num_molecules} molecules): Computed: {real_energy:.4E}, NIST: {nist_real_energy:.4E}, Match: {match_real}")
        fourier_energy_output.append(f"Test {idx+1} ({num_molecules} molecules): Computed: {fourier_energy:.4E}, NIST: {nist_fourier_energy:.4E}, Match: {match_fourier}")
        self_energy_output.append(f"Test {idx+1} ({num_molecules} molecules): Computed: {self_energy:.4E}, NIST: {nist_self_energy:.4E}, Match: {match_self}")
        intra_energy_output.append(f"Test {idx+1} ({num_molecules} molecules): Computed: {intra_energy:.4E}, NIST: {nist_intra_energy:.4E}, Match: {match_intra}")
    
    print("Real Energy Comparison:")
    print(*real_energy_output, sep="\n")
    print("Fourier Energy Comparison:")
    print(*fourier_energy_output, sep="\n")
    print("Self Energy Comparison:")
    print(*self_energy_output, sep="\n")
    print("Intra Energy Comparison:")
    print(*intra_energy_output, sep="\n")
    print()
    print(f"Count of correct Real Energy answers: {matched_real}")
    print(f"Count of incorrect Real Energy answers: {not_matched_real}")
    print(f"Count of correct Fourier Energy answers: {matched_fourier}")
    print(f"Count of incorrect Fourier Energy answers: {not_matched_fourier}")
    print(f"Count of correct Self Energy answers: {matched_self}")
    print(f"Count of incorrect Self Energy answers: {not_matched_self}")
    print(f"Count of correct Intra Energy answers: {matched_intra}")
    print(f"Count of incorrect Intra Energy answers: {not_matched_intra}")
    print()
    total_correct = matched_real + matched_fourier + matched_self + matched_intra
    total_incorrect = not_matched_real + not_matched_fourier + not_matched_self + not_matched_intra
    print(f"Total correct answers: {total_correct}")
    print(f"Total incorrect answers: {total_incorrect}")

# calling compare_coulomb_energy function
compare_coulomb_energy(results, NIST_SPC_E_Water)
'''
storing results
================================ Iteration 1 ================================

================================ Human Message =================================

import numpy as n"""

In [172]:
import ast
import pandas as pd
import re
import subprocess

import ast

def replace_functions(source_code, df_functions, global_variables):
    """
    Replaces function definitions in `source_code` with those in `df_functions`,
    ensuring that dependent (child) functions are also included immediately before their parent functions.
    The global variables are inserted once at the very top before all functions, but only those not already present.

    Args:
        source_code (str): The original script as a string.
        df_functions (pd.DataFrame): DataFrame containing function names, code, and dependencies.
        global_variables (dict): Dictionary of global variables to be included before functions.

    Returns:
        str: The modified source code with updated functions and global variables.
    """
    # List of function names to be skipped from replacement
    fixed_functions = {"extracting_positions", "create_dataframes", "compare_coulomb_energy"}

    tree = ast.parse(source_code)
    modified_code = source_code  # Start with the original code

    # A set of function names already in the source code (for adding missing functions)
    existing_functions = {node.name for node in tree.body if isinstance(node, ast.FunctionDef)}

    # Extract the existing global variables from the source code
    existing_globals = set()
    for node in tree.body:
        if isinstance(node, ast.Assign):  # Look for assignments (global variables are usually assigned outside functions)
            for target in node.targets:
                if isinstance(target, ast.Name):
                    existing_globals.add(target.id)

    # First, process child functions
    added_functions = {}  # Keep track of where each function's code should be added

    for _, row in df_functions.iterrows():
        called_functions_dict = row["called_functions_dict"]  # Dictionary of dependencies
        function_name = row["function_name"]

        # Skip fixed functions
        if function_name in fixed_functions:
            continue

        if called_functions_dict:
            for child_name, child_code in called_functions_dict.items():
                if child_name not in existing_functions and child_name not in added_functions:
                    # Add child function immediately before its parent function
                    added_functions[child_name] = child_code

    # Now, replace the functions with their updated versions
    for _, row in df_functions.iterrows():
        function_name = row["function_name"]
        updated_func_code = row["func_code"]
        called_functions_dict = row["called_functions_dict"]  # Dictionary of dependencies

        # Skip fixed functions
        if function_name in fixed_functions:
            continue

        # Replace the function definition in the source code
        for node in tree.body:
            if isinstance(node, ast.FunctionDef) and node.name == function_name:
                old_func_code = ast.get_source_segment(source_code, node)

                # Insert the function code in place of the old one
                modified_code = modified_code.replace(old_func_code, updated_func_code)

                # Also insert child functions immediately before the parent function
                if called_functions_dict:
                    for child_name, child_code in called_functions_dict.items():
                        if child_name in added_functions:
                            # Insert the child function code right before the parent function
                            modified_code = modified_code.replace(updated_func_code, added_functions[child_name] + "\n\n" + updated_func_code)

    # Insert global variables once before all function definitions, but only if they are not already present
    if isinstance(global_variables, dict):
        # Format global variables as code, only adding those not already in the source code
        global_code = "\n".join(f"{var} = {repr(global_variables[var])}" for var in global_variables if var not in existing_globals)
    else:
        raise ValueError("global_variables must be a dictionary.")

    # Find the first function definition and insert the global variables before it
    function_start_index = None
    for idx, line in enumerate(modified_code.splitlines()):
        if line.strip().startswith("def "):  # Check for the first function definition
            function_start_index = idx
            break

    if function_start_index is not None:
        # Insert the global variables just before the first function definition
        modified_code_lines = modified_code.splitlines()
        modified_code_lines.insert(function_start_index, global_code)
        modified_code = "\n".join(modified_code_lines)

    return modified_code



import ast
import pandas as pd
import re

def extract_functions_with_dependencies_from_text(text):
    """
    Extracts Python functions and global variables from a string containing Python code and other text.
    Identifies function calls inside each function, extracts docstrings, and identifies global variables.

    Args:
        text (str): The input text that contains Python code along with other text.

    Returns:
        pd.DataFrame: DataFrame with 'function_name', 'func_code', 'docstring', 'global_variables', and 'called_functions_dict'.
    """
    print("text given")
    try:
        # First, try extracting Python code between ```...``` blocks
        python_code1 = "\n".join(re.findall(r'```(.*?)```', text, re.DOTALL))

        # Then, try extracting Python code between ----...---- blocks
        python_code2 = "\n".join(re.findall(r'[-]{2,}(.*?)\n[-]{2,}', text, re.DOTALL))

        # Then, try extracting Python code between '''...''' blocks
        python_code3 = "\n".join(re.findall(r"'''(.*?)'''", text, re.DOTALL)).strip()

        # If either python_code1 or python_code2 is found, select the first non-empty result
        if python_code2 or python_code1 or python_code3:
            python_code = python_code1 or python_code2 or python_code3
        else:
            raise ValueError("No Python code found in the provided text.")
    
    except Exception as e:
        print("Error:", e)
        return None, None
    
    print("text extracted")
    
    # Replace sep="\n" with sep=chr(10) to avoid syntax issues
    python_code = re.sub(r'sep=\s*"\n"', 'sep=chr(10)', python_code)

    print(python_code)
    # Parse the Python code using AST
    tree = ast.parse(python_code)
    functions_data = {}

    # First, extract all function definitions
    function_codes = {}
    function_docstrings = {}

    for node in tree.body:
        if isinstance(node, ast.FunctionDef):
            function_codes[node.name] = ast.get_source_segment(python_code, node)
            
            # Extract docstring using AST's method
            docstring = ast.get_docstring(node)
            if docstring:
                function_docstrings[node.name] = docstring
            else:
                function_docstrings[node.name] = None

    # Extract global variables (variables in the global scope)
    global_variables = {}
    for node in tree.body:
        if isinstance(node, ast.Assign):  # Global variable assignment
            for target in node.targets:
                if isinstance(target, ast.Name):  # Only name targets are considered variables
                    # Extract the value of the global variable
                    value = ast.literal_eval(node.value) if isinstance(node.value, ast.Constant) else None
                    global_variables[target.id] = value

    # Now, extract function calls and variables within each function
    for node in tree.body:
        if isinstance(node, ast.FunctionDef):
            function_name = node.name
            function_code = function_codes[function_name]
            function_docstring = function_docstrings[function_name]

            # Find all function calls inside this function
            called_functions_dict = {}
            for child in ast.walk(node):  # Traverse the function body
                if isinstance(child, ast.Call) and isinstance(child.func, ast.Name):
                    called_func_name = child.func.id  # Get called function name
                    if called_func_name in function_codes:  # Only store if it's a defined function
                        called_functions_dict[called_func_name] = function_codes[called_func_name]

            # Store function details
            functions_data[function_name] = {
                "func_code": function_code,
                "docstring": function_docstring,
                "called_functions_dict": called_functions_dict if called_functions_dict else None
            }

    # Convert to DataFrame for functions
    df = pd.DataFrame([
        {
            "function_name": key,
            "func_code": value["func_code"],
            "docstring": value["docstring"],
            "called_functions_dict": value["called_functions_dict"]
        }
        for key, value in functions_data.items()
    ])

    return df, global_variables

def execute_code(code: str) -> dict:
    """
    Executes a Python code string and captures the output, errors, and return code.

    Args:
        code (str): The Python code to execute.

    Returns:
        dict: Dictionary with 'stdout', 'stderr', and 'return_code'.
    """
    try:
        process = subprocess.Popen(
            ["/Users/pulicharishma/anaconda3/bin/python", "-c", code],
            text=True,
            stdout=subprocess.PIPE,
            stderr=subprocess.PIPE
        )
        output, errors = process.communicate()
        
        return {
            "stdout": output.strip().split("\n") if output else [],
            "stderr": errors.strip().split("\n") if errors else [],
            "return_code": process.returncode
        }
    except Exception as e:
        return {"error": str(e)}


In [165]:
Coulomb_input = Coulomb_input.replace('#--- Complete this code ---#','### Complete this code ###')

In [166]:
df_functions

""


In [167]:
global_variables

{}

In [168]:
text = text.replace('#--- Complete this code ---#','### Complete this code ###')

In [173]:
# Extract functions and their dependencies from the text
df_functions, global_variables  = extract_functions_with_dependencies_from_text(text2)

# Check if there are any function names extracted in the DataFrame
if df_functions.empty or df_functions['function_name'].isnull().all():
    feedback = "No Python code found. Please review the code more clearly and provide a Python code after changes."
else:
    # Replace functions in the Coulomb_input (or the input script)
    modified_code = replace_functions(Coulomb_input, df_functions, global_variables)
    

text given
text extracted
import numpy as np
import pandas as pd
from scipy.special import erfc, erf
import math 

# defining all variables
atom_properties = {
    'O': {'type': 'O', 'sigma': 3.165558, 'epsilon': 78.197431, 'charge': -0.8476, 'num_particles': 1},
    'H': {'type': 'H', 'sigma': 0.000000, 'epsilon': 0.000000, 'charge': 0.4238, 'num_particles': 2},
}

# Trying for small configurations first
file_paths = [
#        'spce_sample_config_periodic4.txt',
#        'spce_sample_config_periodic2.txt',
#        'spce_sample_config_periodic3.txt',
        '../data/spce_sample_config_periodic1_modified.txt'
]

NIST_SPC_E_Water = {
    'Configuration': [1, 2, 3, 4],
    'M (number of SPC/E molecules)': [100, 200, 300, 750],
    'Lx=Ly=Lz (Å)': [20.0, 20.0, 20.0, 30.0],
    'Edisp/kB (K)': [9.95387E+04, 1.93712E+05, 3.54344E+05, 4.48593E+05],
    'ELRC/kB (K)': [-8.23715E+02, -3.29486E+03, -7.41343E+03, -1.37286E+04],
    'Ereal/kB (K)': [-5.58889E+05, -1.19295E+06, -1.96297E+06, -3.

In [174]:
df_functions

,function_name,func_code,docstring,called_functions_dict
0,extracting_positions,def extracting_positions(input_file):\n # E...,None,None
1,creating_dataframes,"def creating_dataframes(file_paths, atom_prope...",None,None
2,compute_real_energies,"def compute_real_energies(system_data, configu...",None,None
3,compute_fourier_energies,"def compute_fourier_energies(system_data, conf...",None,None
4,compute_self_energies,"def compute_self_energies(system_data, configu...",None,None
5,compute_intra_energies,"def compute_intra_energies(system_data, config...",None,None
6,compare_coulomb_energy,"def compare_coulomb_energy(df1, df2, tolerance...",None,None


In [ ]:
modified_code.replace()

In [193]:
print(modified_code)

import numpy as np
import pandas as pd
from scipy.special import erfc,erf
import math 

# defining all variables
atom_properties = {
    'O': {'type': 'O', 'sigma': 3.165558, 'epsilon': 78.197431, 'charge': -0.8476, 'num_particles': 1},
    'H': {'type': 'H', 'sigma': 0.000000, 'epsilon': 0.000000, 'charge': 0.4238, 'num_particles': 2},
}

# Trying for small configurations first
file_paths = [
#        'spce_sample_config_periodic4.txt',
#        'spce_sample_config_periodic2.txt',
#        'spce_sample_config_periodic3.txt',
        '../data/spce_sample_config_periodic1_modified.txt'
    ]

NIST_SPC_E_Water = {
        'Configuration': [1, 2, 3, 4],
        'M (number of SPC/E molecules)': [100, 200, 300, 750],
        'Lx=Ly=Lz (Å)': [20.0, 20.0, 20.0, 30.0],
        'Edisp/kB (K)': [9.95387E+04, 1.93712E+05, 3.54344E+05, 4.48593E+05],
        'ELRC/kB (K)': [-8.23715E+02, -3.29486E+03, -7.41343E+03, -1.37286E+04],
        'Ereal/kB (K)': [-5.58889E+05, -1.19295E+06, -1.96297E+06, -3

In [143]:
df_functions

""


In [196]:
execute_code(modified_code.replace("isclose","isscloase"))

{'stdout': [],
 'stderr': ['Traceback (most recent call last):',
  '  File "<string>", line 379, in <module>',
  '  File "<string>", line 328, in compare_coulomb_energy',
  '  File "/Users/pulicharishma/anaconda3/lib/python3.11/site-packages/numpy/__init__.py", line 311, in __getattr__',
  '    raise AttributeError("module {!r} has no attribute "',
  "AttributeError: module 'numpy' has no attribute 'isscloase'. Did you mean: 'isclose'?"],
 'return_code': 1}

In [ ]:
result = execute_code(modified_code.replace("isclose",""))

In [186]:
result['stdout']

['Real Energy Comparison:',
 'Test 1 (100.0 molecules): Computed: -2.4877E-30, NIST: -5.5889E+05, Match: False',
 'Fourier Energy Comparison:',
 'Test 1 (100.0 molecules): Computed: 3.1577E+03, NIST: 6.2701E+03, Match: False',
 'Self Energy Comparison:',
 'Test 1 (100.0 molecules): Computed: -5.6894E+07, NIST: -2.8447E+06, Match: False',
 'Intra Energy Comparison:',
 'Test 1 (100.0 molecules): Computed: 1.0167E+07, NIST: 2.8100E+06, Match: False',
 '',
 'Count of correct Real Energy answers: 0',
 'Count of incorrect Real Energy answers: 1',
 'Count of correct Fourier Energy answers: 0',
 'Count of incorrect Fourier Energy answers: 1',
 'Count of correct Self Energy answers: 0',
 'Count of incorrect Self Energy answers: 1',
 'Count of correct Intra Energy answers: 0',
 'Count of incorrect Intra Energy answers: 1',
 '',
 'Total correct answers: 0',
 'Total incorrect answers: 4']

In [176]:
execute_code(Coulomb_input)

{'stdout': ['Real Energy Comparison:',
  'Test 1 (100.0 molecules): Computed: 0.0000E+00, NIST: -5.5889E+05, Match: False',
  'Fourier Energy Comparison:',
  'Test 1 (100.0 molecules): Computed: 0.0000E+00, NIST: 6.2701E+03, Match: False',
  'Self Energy Comparison:',
  'Test 1 (100.0 molecules): Computed: -5.6894E-03, NIST: -2.8447E+06, Match: False',
  'Intra Energy Comparison:',
  'Test 1 (100.0 molecules): Computed: -5.6894E-03, NIST: 2.8100E+06, Match: False',
  '',
  'Count of correct Real Energy answers: 0',
  'Count of incorrect Real Energy answers: 1',
  'Count of correct Fourier Energy answers: 0',
  'Count of incorrect Fourier Energy answers: 1',
  'Count of correct Self Energy answers: 0',
  'Count of incorrect Self Energy answers: 1',
  'Count of correct Intra Energy answers: 0',
  'Count of incorrect Intra Energy answers: 1',
  '',
  'Total correct answers: 0',
  'Total incorrect answers: 4'],
 'stderr': [],
 'return_code': 0}

In [115]:
global_variables

{'atom_properties': None,
 'file_paths': None,
 'NIST_SPC_E_Water': None,
 'results': None}

In [ ]:
import re

def handle_feedback(result, i, n, Iteration, last_output):
    # Check if stdout exists and has enough entries
    print("STDOUT Output:", result['stdout'])  # Debugging print statement
    if result['stdout'] and len(result['stdout']) >= 2:
        try:
            incorrect_match = re.search(r"\d+", result['stdout'][-1])
            correct_match = re.search(r"\d+", result['stdout'][-2])

            incorrect = int(incorrect_match.group(0)) if incorrect_match else 0
            correct = int(correct_match.group(0)) if correct_match else 0
        except (IndexError, AttributeError, ValueError) as e:
            feedback = "Error extracting test results from output. Please check the format of the stdout."
            Iteration[i] = feedback
            return feedback, i + 1, n, last_output  # Move to next iteration safely
    else:
        # Mask file paths in stderr
        stderr_masked = []
        for line in result['stderr']:
            # Regex pattern to find file paths (e.g., "/path/to/file" or "C:\path\to\file")
            masked_line = re.sub(r'([A-Za-z]:[\\/].*?|\b(?:\/|\\)(?:[\w\.-]+[\\/])*[\w\.-]+\.[\w]+)', '[FILE_PATH_MASKED]', line)
            stderr_masked.append(masked_line)
            
        feedback = (
            "Your code resulted in the following error:\n"
            "Error\n"
            + '\n'.join(result['stderr'])
            + "\nReview the specific part or function causing the error in the next iteration and correct it.\n"
        )
        Iteration[i] = feedback
        
        # Handle additional iteration request
        if i + 1 == n - 1:
            try:
                additional_iterations = int(input("How many iterations do you want to add? "))
                n += additional_iterations  # Extend the loop
            except ValueError:
                print("Invalid input! Please enter an integer.")
        
        return feedback, i + 1, n, last_output  # Move to next iteration safely

    # Check if all tests passed
    if correct == 4:
        print(f"In Iteration {i}, LLM did a great job! Exiting from the loop")
        feedback = "LLM did a great job! Exiting from the loop"
        return feedback, i, n, last_output

    # Update feedback message for progress
    feedback = (
        "Your code hasn't yet passed all of the NIST benchmark tests. "
        "Here's the current progress:\n" + '\n'.join(result['stdout']) + "\n "
        f"Out of the tests, {correct} answers have been correct, with {incorrect} remaining. "
        "Keep going – you're getting closer! Continue refining the code step by step until it passes all the tests. "
        "Complete each function sequentially, incorporating feedback to optimize efficiency and align with NIST benchmarks. "
        "Aim to match the benchmark values as closely as possible with each attempt, ensuring visible numerical improvements. "
        "Refer to the documentation and revise accordingly.\n"
    )
    Iteration[i] = feedback

    # Handle additional iteration request
    if i + 1 == n - 1:
        try:
            additional_iterations = int(input("How many iterations do you want to add? "))
            n += additional_iterations  # Extend the loop
        except ValueError:
            print("Invalid input! Please enter an integer.")
    
    return feedback, i + 1, n, last_output  # Increment iteration safely


In [178]:
error = """Error
Traceback (most recent call last):
  File "<string>", line 365, in <module>
  File "<string>", line 314, in compare_coulomb_energy
  File "<__array_function__ internals>", line 180, in isclose
  File "/Users/pulicharishma/anaconda3/lib/python3.11/site-packages/numpy/core/numeric.py", line 2372, in isclose
    xfin = isfinite(x)
           ^^^^^^^^^^^"""

In [197]:
result = execute_code(modified_code.replace("isclose","isscloase"))

In [182]:
stderr_masked = []
for line in error:
    masked_line = re.sub(r'([A-Za-z]:[\\/].*?|\b(?:\/|\\)(?:[\w\.-]+[\\/])*[\w\.-]+\.[\w]+)', '[FILE_PATH_MASKED]', line)
    stderr_masked.append(masked_line)
print(''.join(stderr_masked))

Error
Traceback (most recent call last):
  File "<string>", line 365, in <module>
  File "<string>", line 314, in compare_coulomb_energy
  File "<__array_function__ internals>", line 180, in isclose
  File "/Users/pulicharishma/anaconda3/lib/python3.11/site-packages/numpy/core/numeric.py", line 2372, in isclose
    xfin = isfinite(x)
           ^^^^^^^^^^^


In [198]:
feedback = (
            "Your code resulted in the following error:\n"
            "Error\n"
            + '\n'.join([re.sub(r'/\S+', '[MASKED_PATH]', line) for line in result['stderr']])  # Masking file paths
            + "\nReview the specific part or function causing the error in the next iteration and correct it.\n"
        )
print(feedback)

Your code resulted in the following error:
Error
Traceback (most recent call last):
  File "<string>", line 379, in <module>
  File "<string>", line 328, in compare_coulomb_energy
  File "[MASKED_PATH] line 311, in __getattr__
    raise AttributeError("module {!r} has no attribute "
AttributeError: module 'numpy' has no attribute 'isscloase'. Did you mean: 'isclose'?
Review the specific part or function causing the error in the next iteration and correct it.



In [200]:
correct = "2"
incorrect = "10"

In [203]:
feedback += (
    "Your code hasn't yet passed all of the NIST benchmark tests. "
    "Here's the current progress:\n" + '\n'.join(result['stdout']) + "\n"
    f"Out of the tests, {correct} answers have been correct, with {incorrect} remaining. " +
    # Check if there are no incorrect answers 
    ("Focus on getting the code to align with the benchmark for at least one part! Continue refining the code step by step until it passes all the tests."
     if incorrect == 0 else
     "Keep going – you're getting closer! Continue refining the code step by step until it passes all the tests.") 
    + " Complete each function sequentially, incorporating feedback to optimize efficiency and align with NIST benchmarks. "
    "Aim to match the benchmark values as closely as possible with each attempt, ensuring visible numerical improvements. "
    "Refer to the context and revise accordingly.\n")
print(feedback)

Your code resulted in the following error:
Error
Traceback (most recent call last):
  File "<string>", line 379, in <module>
  File "<string>", line 328, in compare_coulomb_energy
  File "[MASKED_PATH] line 311, in __getattr__
    raise AttributeError("module {!r} has no attribute "
AttributeError: module 'numpy' has no attribute 'isscloase'. Did you mean: 'isclose'?
Review the specific part or function causing the error in the next iteration and correct it.
Your code hasn't yet passed all of the NIST benchmark tests. Here's the current progress:

Out of the tests, 2 answers have been correct, with 10 remaining. Keep going – you're getting closer! Continue refining the code step by step until it passes all the tests. Complete each function sequentially, incorporating feedback to optimize efficiency and align with NIST benchmarks. Aim to match the benchmark values as closely as possible with each attempt, ensuring visible numerical improvements. Refer to the context and revise according

In [8]:
import ast
import pandas as pd
import re
import subprocess

import ast

def replace_functions(source_code, df_functions, global_variables):
    """
    Replaces function definitions in `source_code` with those in `df_functions`,
    ensuring that dependent (child) functions are also included immediately before their parent functions.
    The global variables are inserted once at the very top before all functions, but only those not already present.

    Args:
        source_code (str): The original script as a string.
        df_functions (pd.DataFrame): DataFrame containing function names, code, and dependencies.
        global_variables (dict): Dictionary of global variables to be included before functions.

    Returns:
        str: The modified source code with updated functions and global variables.
    """
    # List of function names to be skipped from replacement
    fixed_functions = {"extracting_positions", "create_dataframes", "compare_coulomb_energy"}

    tree = ast.parse(source_code)
    modified_code = source_code  # Start with the original code

    # A set of function names already in the source code (for adding missing functions)
    existing_functions = {node.name for node in tree.body if isinstance(node, ast.FunctionDef)}

    # Extract the existing global variables from the source code
    existing_globals = set()
    for node in tree.body:
        if isinstance(node, ast.Assign):  # Look for assignments (global variables are usually assigned outside functions)
            for target in node.targets:
                if isinstance(target, ast.Name):
                    existing_globals.add(target.id)

    # First, process child functions
    added_functions = {}  # Keep track of where each function's code should be added

    for _, row in df_functions.iterrows():
        called_functions_dict = row["called_functions_dict"]  # Dictionary of dependencies
        function_name = row["function_name"]

        # Skip fixed functions
        if function_name in fixed_functions:
            continue

        if called_functions_dict:
            for child_name, child_code in called_functions_dict.items():
                if child_name not in existing_functions and child_name not in added_functions:
                    # Add child function immediately before its parent function
                    added_functions[child_name] = child_code

    # Now, replace the functions with their updated versions
    for _, row in df_functions.iterrows():
        function_name = row["function_name"]
        updated_func_code = row["func_code"]
        called_functions_dict = row["called_functions_dict"]  # Dictionary of dependencies

        # Skip fixed functions
        if function_name in fixed_functions:
            continue

        # Replace the function definition in the source code
        for node in tree.body:
            if isinstance(node, ast.FunctionDef) and node.name == function_name:
                old_func_code = ast.get_source_segment(source_code, node)

                # Insert the function code in place of the old one
                modified_code = modified_code.replace(old_func_code, updated_func_code)

                # Also insert child functions immediately before the parent function
                if called_functions_dict:
                    for child_name, child_code in called_functions_dict.items():
                        if child_name in added_functions:
                            # Insert the child function code right before the parent function
                            modified_code = modified_code.replace(updated_func_code, added_functions[child_name] + "\n\n" + updated_func_code)

    # Insert global variables once before all function definitions, but only if they are not already present
    if isinstance(global_variables, dict):
        # Format global variables as code, only adding those not already in the source code
        global_code = "\n".join(f"{var} = {repr(global_variables[var])}" for var in global_variables if var not in existing_globals)
    else:
        raise ValueError("global_variables must be a dictionary.")

    # Find the first function definition and insert the global variables before it
    function_start_index = None
    for idx, line in enumerate(modified_code.splitlines()):
        if line.strip().startswith("def "):  # Check for the first function definition
            function_start_index = idx
            break

    if function_start_index is not None:
        # Insert the global variables just before the first function definition
        modified_code_lines = modified_code.splitlines()
        modified_code_lines.insert(function_start_index, global_code)
        modified_code = "\n".join(modified_code_lines)

    return modified_code



import ast
import pandas as pd
import re

def extract_functions_with_dependencies_from_text(text):
    """
    Extracts Python functions and global variables from a string containing Python code and other text.
    Identifies function calls inside each function, extracts docstrings, and identifies global variables.

    Args:
        text (str): The input text that contains Python code along with other text.

    Returns:
        pd.DataFrame: DataFrame with 'function_name', 'func_code', 'docstring', 'global_variables', and 'called_functions_dict'.
    """
    try:
        # First, try extracting Python code between ```...``` blocks
        python_code1 = "\n".join(re.findall(r'```(.*?)```', text, re.DOTALL))

        # Then, try extracting Python code between ----...---- blocks
        python_code2 = "\n".join(re.findall(r'[-]{2,}(.*?)\n[-]{2,}', text, re.DOTALL))

        # Then, try extracting Python code between '''...''' blocks
        python_code3 = "\n".join(re.findall(r"'''(.*?)'''", text, re.DOTALL)).strip()

        # If either python_code1 or python_code2 is found, select the first non-empty result
        if python_code2 or python_code1 or python_code3:
            python_code = python_code1 or python_code2 or python_code3
        else:
            raise ValueError("No Python code found in the provided text.")
    
    except Exception as e:
        print("Error:", e)
        return None, None
        
    # Replace sep="\n" with sep=chr(10) to avoid syntax issues
    python_code = re.sub(r'sep=\s*"\n"', 'sep=chr(10)', python_code)

    # Parse the Python code using AST
    tree = ast.parse(python_code)
    functions_data = {}

    # First, extract all function definitions
    function_codes = {}
    function_docstrings = {}

    for node in tree.body:
        if isinstance(node, ast.FunctionDef):
            function_codes[node.name] = ast.get_source_segment(python_code, node)
            
            # Extract docstring using AST's method
            docstring = ast.get_docstring(node)
            if docstring:
                function_docstrings[node.name] = docstring
            else:
                function_docstrings[node.name] = None

    # Extract global variables (variables in the global scope)
    global_variables = {}
    for node in tree.body:
        if isinstance(node, ast.Assign):  # Global variable assignment
            for target in node.targets:
                if isinstance(target, ast.Name):  # Only name targets are considered variables
                    # Extract the value of the global variable
                    value = ast.literal_eval(node.value) if isinstance(node.value, ast.Constant) else None
                    global_variables[target.id] = value

    # Now, extract function calls and variables within each function
    for node in tree.body:
        if isinstance(node, ast.FunctionDef):
            function_name = node.name
            function_code = function_codes[function_name]
            function_docstring = function_docstrings[function_name]

            # Find all function calls inside this function
            called_functions_dict = {}
            for child in ast.walk(node):  # Traverse the function body
                if isinstance(child, ast.Call) and isinstance(child.func, ast.Name):
                    called_func_name = child.func.id  # Get called function name
                    if called_func_name in function_codes:  # Only store if it's a defined function
                        called_functions_dict[called_func_name] = function_codes[called_func_name]

            # Store function details
            functions_data[function_name] = {
                "func_code": function_code,
                "docstring": function_docstring,
                "called_functions_dict": called_functions_dict if called_functions_dict else None
            }

    # Convert to DataFrame for functions
    df = pd.DataFrame([
        {
            "function_name": key,
            "func_code": value["func_code"],
            "docstring": value["docstring"],
            "called_functions_dict": value["called_functions_dict"]
        }
        for key, value in functions_data.items()
    ])

    return df, global_variables

def execute_code(code: str) -> dict:
    """
    Executes a Python code string and captures the output, errors, and return code.

    Args:
        code (str): The Python code to execute.

    Returns:
        dict: Dictionary with 'stdout', 'stderr', and 'return_code'.
    """
    try:
        process = subprocess.Popen(
            ["/Users/pulicharishma/anaconda3/bin/python", "-c", code],
            text=True,
            stdout=subprocess.PIPE,
            stderr=subprocess.PIPE
        )
        output, errors = process.communicate()
        
        return {
            "stdout": output.strip().split("\n") if output else [],
            "stderr": errors.strip().split("\n") if errors else [],
            "return_code": process.returncode
        }
    except Exception as e:
        return {"error": str(e)}


In [3]:

with open("Coulomb_input.py", "r") as file:
    Coulomb_input = file.read()

In [14]:
text = """
To solve this problem, we'll need to break down the Ewald Summation formula into its component parts and implement each calculation separately. Let's start with the `compute_real_energies` function. 

### Step-by-Step Explanation

The real-space term \(E_{\text{real}}\) involves the direct interaction between charges that are within a certain cutoff distance. The formula for the real-space contribution is:

\[ E_{\text{real}} = \sum\limits_{j} \sum\limits_{l>j} \dfrac{q_j q_l}{4 \pi \epsilon_0} \dfrac{\text{erfc}(\alpha \cdot \left| \mathbf{r}_{jl} \right|)}{\left| \mathbf{r}_{jl} \right|} \Theta(r_{\text{cut}} - \left|\mathbf{r}_{jl}\right|) \]

where:
- \(q_j\) and \(q_l\) are the charges of the atoms,
- \(\mathbf{r}_{jl}\) is the distance vector between atoms \(j\) and \(l\),
- \(\epsilon_0\) is the permittivity of free space,
- \(\alpha\) is the Ewald parameter,
- \(r_{\text{cut}}\) is the cutoff radius for the interaction; if the distance between two particles exceeds this value, their interaction is not considered.

Let's implement this calculation in the `compute_real_energies` function:

```python
def compute_real_energies(system_data, configuration, force_field):
    # Extract necessary parameters from the system data
    box_length = system_data['box length']
    num_particles = system_data['number of particles']
    alpha = system_data['alpha']
    epsilon_0 = system_data['ε0']
    cutoff = system_data['cutoff']

    # Initialize the real energy accumulator
    real_energy = 0.0

    # Iterate through all pairs of atoms
    for i in range(len(configuration)):
        for j in range(i + 1, len(configuration)):
            # Extract data for atoms i and j
            atom_i = configuration.iloc[i]
            atom_j = configuration.iloc[j]

            # Compute the distance between the atoms, considering periodic boundary conditions
            dist_vector = np.array([atom_j['X'] - atom_i['X'],
                                    atom_j['Y'] - atom_i['Y'],
                                    atom_j['Z'] - atom_i['Z']])
            dist_vector -= box_length * np.round(dist_vector / box_length)
            distance = np.linalg.norm(dist_vector)

            # Skip pairs that are outside the cutoff radius
            if distance > cutoff:
                continue

            # Retrieve charges
            charge_i = force_field.loc[atom_i['Atom Type'], 'charge']
            charge_j = force_field.loc[atom_j['Atom Type'], 'charge']

            # Compute the real-space potential energy contribution
            real_energy += (charge_i * charge_j / (4 * np.pi * epsilon_0)) * erfc(alpha * distance) / distance

    # Convert energy to Kelvin using the conversion factor (Joules to Kelvin)
    real_energy *= (1.60219e-19 / system_data['kB'])

    return real_energy
```

### Explanation of the Code

- **Distance Calculation with Periodic Boundary Conditions**: 
  We compute the distance considering that the system is periodic. This wraps around distances that exceed half the box length to ensure atoms link with the nearest periodic image.

- **Direct Interaction Calculation**:
  Only pairs whose distance is less than the cutoff contribute to the real-space energy. We use the complementary error function `erfc` to smooth the transition across the real-space part.

- **Energy Conversion**: 
  The calculated real-space energy is converted from electrostatic energy units (eV or Joules) to Kelvin using the Boltzmann constant.

```python
def compute_realdsddsad_energies(system_data, configuration, force_field):
    # Extract necessary parameters from the system data
    box_length = system_data['box length']
    num_particles = system_data['number of particles']
    alpha = system_data['alpha']
    epsilon_0 = system_data['ε0']
    cutoff = system_data['cutoff']

    # Initialize the real energy accumulator
    real_energy = 0.0

    # Iterate through all pairs of atoms
    for i in range(len(configuration)):
        for j in range(i + 1, len(configuration)):
            # Extract data for atoms i and j
            atom_i = configuration.iloc[i]
            atom_j = configuration.iloc[j]

            # Compute the distance between the atoms, considering periodic boundary conditions
            dist_vector = np.array([atom_j['X'] - atom_i['X'],
                                    atom_j['Y'] - atom_i['Y'],
                                    atom_j['Z'] - atom_i['Z']])
            dist_vector -= box_length * np.round(dist_vector / box_length)
            distance = np.linalg.norm(dist_vector)

            # Skip pairs that are outside the cutoff radius
            if distance > cutoff:
                continue

            # Retrieve charges
            charge_i = force_field.loc[atom_i['Atom Type'], 'charge']
            charge_j = force_field.loc[atom_j['Atom Type'], 'charge']

            # Compute the real-space potential energy contribution
            real_energy += (charge_i * charge_j / (4 * np.pi * epsilon_0)) * erfc(alpha * distance) / distance

    # Convert energy to Kelvin using the conversion factor (Joules to Kelvin)
    real_energy *= (1.60219e-19 / system_data['kB'])

    return real_energy
```

### Next Steps

Run the above code and check the updates in the `compare_coulomb_energy` function output. If it yields correct matches for `Ereal`, we can proceed to similarly fill the other function templates (`compute_fourier_energies`, `compute_self_energies`, and `compute_intra_energies`) to complete the Ewald summation.

Please implement and run these changes, then let's review the results and further details for the `Efourier`, `Eself`, and `Eintra` calculations if needed.
"""

In [16]:
df_functions, global_variables = extract_functions_with_dependencies_from_text(text)

In [17]:
replace_functions(Coulomb_input,df_functions, global_variables)

'import numpy as np\nimport pandas as pd\nfrom scipy.special import erfc,erf\nimport math \n\n# defining all variables\natom_properties = {\n    \'O\': {\'type\': \'O\', \'sigma\': 3.165558, \'epsilon\': 78.197431, \'charge\': -0.8476, \'num_particles\': 1},\n    \'H\': {\'type\': \'H\', \'sigma\': 0.000000, \'epsilon\': 0.000000, \'charge\': 0.4238, \'num_particles\': 2},\n}\n\n# Trying for small configurations first\nfile_paths = [\n#        \'spce_sample_config_periodic4.txt\',\n#        \'spce_sample_config_periodic2.txt\',\n#        \'spce_sample_config_periodic3.txt\',\n        \'../data/spce_sample_config_periodic1_modified.txt\'\n    ]\n\nNIST_SPC_E_Water = {\n        \'Configuration\': [1, 2, 3, 4],\n        \'M (number of SPC/E molecules)\': [100, 200, 300, 750],\n        \'Lx=Ly=Lz (Å)\': [20.0, 20.0, 20.0, 30.0],\n        \'Edisp/kB (K)\': [9.95387E+04, 1.93712E+05, 3.54344E+05, 4.48593E+05],\n        \'ELRC/kB (K)\': [-8.23715E+02, -3.29486E+03, -7.41343E+03, -1.37286E+04]

In [42]:
with open("LJ_Coulomb_outputs.txt","r") as file:
    content = file.read()
content

'{\n"Reasoning": "To handle triclinic or monoclinic cells, we must (1) parse the cell lengths and angles from the input files, (2) construct both the direct (box) and inverse box matrices, and (3) apply the minimum-image convention in fractional space. For the Fourier-space computations, we similarly construct the reciprocal box. Below, the \'extracting_positions\' function is modified to read the cell parameters from the first three lines of the input file and store them as attributes (including the box matrix), while the \'apply_minimum_image\' function detects if the cell is non-cubic so it can use the fractional-wrapping approach (via \'minimum_image_distance_triclinic\'). The Ewald summation formulas then rely on this logic for real-space and dispersion pairwise distance calculations in non-cubic geometries. No changes are made to \'compare_LJ_coulomb_energy\'.",\n"Code": "import numpy as np\\\\nimport pandas as pd\\\\nfrom scipy.special import erfc, erf\\\\nimport math\\\\n\\\\n#

TypeError: string indices must be integers, not 'str'

In [8]:
import ast
dict = ast.literal_eval(context)

In [20]:
import json
import re
import ast
import pandas as pd
import subprocess
import textwrap

In [54]:
def extract_functions_with_dependencies_from_text(content):
    """
    Extracts Python functions and global variables from a string containing Python code and other text.
    Identifies function calls inside each function, extracts docstrings, and identifies global variables.

    Args:
        text (str): The input text that contains Python code along with other text.

    Returns:
        pd.DataFrame: DataFrame with 'function_name', 'func_code', 'docstring', 'global_variables', and 'called_functions_dict'.
    """

    data_json = json.loads(content)
    extracted_code = data_json.get("Code", "").replace("\\n", "\n")
    extracted_code = re.sub(r"sep='\n'", r"sep=chr(10)", extracted_code)

    # Parse the Python code using AST
    tree = ast.parse(extracted_code)
    functions_data = {}

    # First, extract all function definitions
    function_codes = {}
    function_docstrings = {}

    for node in tree.body:
        if isinstance(node, ast.FunctionDef):
            function_codes[node.name] = ast.get_source_segment(extracted_code, node)
            
            # Extract docstring using AST's method
            docstring = ast.get_docstring(node)
            if docstring:
                function_docstrings[node.name] = docstring
            else:
                function_docstrings[node.name] = None

    # Extract global variables (variables in the global scope)
    global_variables = {}
    for node in tree.body:
        if isinstance(node, ast.Assign):  # Global variable assignment
            for target in node.targets:
                if isinstance(target, ast.Name):  # Only name targets are considered variables
                    # Extract the value of the global variable
                    value = ast.literal_eval(node.value) if isinstance(node.value, ast.Constant) else None
                    global_variables[target.id] = value

    # Now, extract function calls and variables within each function
    for node in tree.body:
        if isinstance(node, ast.FunctionDef):
            function_name = node.name
            function_code = function_codes[function_name]
            function_docstring = function_docstrings[function_name]

            # Find all function calls inside this function
            called_functions_dict = {}
            for child in ast.walk(node):  # Traverse the function body
                if isinstance(child, ast.Call) and isinstance(child.func, ast.Name):
                    called_func_name = child.func.id  # Get called function name
                    if called_func_name in function_codes:  # Only store if it's a defined function
                        called_functions_dict[called_func_name] = function_codes[called_func_name]

            # Store function details
            functions_data[function_name] = {
                "func_code": function_code,
                "docstring": function_docstring,
                "called_functions_dict": called_functions_dict if called_functions_dict else None
            }

    # Convert to DataFrame for functions
    df = pd.DataFrame([
        {
            "function_name": key,
            "func_code": value["func_code"],
            "docstring": value["docstring"],
            "called_functions_dict": value["called_functions_dict"]
        }
        for key, value in functions_data.items()
    ])

    return df, global_variables

In [36]:
data_json = json.loads(content)
data_json

{'Reasoning': "To handle triclinic or monoclinic cells, we must (1) parse the cell lengths and angles from the input files, (2) construct both the direct (box) and inverse box matrices, and (3) apply the minimum-image convention in fractional space. For the Fourier-space computations, we similarly construct the reciprocal box. Below, the 'extracting_positions' function is modified to read the cell parameters from the first three lines of the input file and store them as attributes (including the box matrix), while the 'apply_minimum_image' function detects if the cell is non-cubic so it can use the fractional-wrapping approach (via 'minimum_image_distance_triclinic'). The Ewald summation formulas then rely on this logic for real-space and dispersion pairwise distance calculations in non-cubic geometries. No changes are made to 'compare_LJ_coulomb_energy'.",
 'Code': 'import numpy as np\\nimport pandas as pd\\nfrom scipy.special import erfc, erf\\nimport math\\n\\n######################

In [38]:
extracted_code = data_json.get("Code", "").replace("\\n", "\n")

In [39]:
# Save to a Python file (optional)
with open("extracted_code.py", "w") as py_file:
    py_file.write(extracted_code)

In [48]:
data_json = json.loads(content)
extracted_code = data_json.get("Code", "").replace("\\n", "\n")
updated_text = re.sub(r"sep='\n'", r"sep=chr(10)", extracted_code)


In [57]:
global_variables

{'atom_properties': None,
 'file_paths': None,
 'NIST_TRICLINIC_SPC_E_Water': None,
 'results': None}

In [56]:
df, global_variables = extract_functions_with_dependencies_from_text(content)
df

,function_name,func_code,docstring,called_functions_dict
0,build_triclinic_box_matrix,"def build_triclinic_box_matrix(a, b, c, alpha_...",Constructs a 3x3 box matrix and its inverse fo...,None
1,minimum_image_distance_triclinic,"def minimum_image_distance_triclinic(r_ij, box...",Applies the minimum image convention by conver...,None
2,apply_minimum_image,"def apply_minimum_image(r_ij, configuration, s...",Wrapper that decides whether to apply a standa...,{'minimum_image_distance_triclinic': 'def mini...
3,extracting_positions,def extracting_positions(input_file):\n wit...,None,{'build_triclinic_box_matrix': 'def build_tric...
4,creating_dataframes,"def creating_dataframes(file_paths, atom_prope...",None,None
5,pair_dispersion_energy,"def pair_dispersion_energy(system_data, config...",None,{'apply_minimum_image': 'def apply_minimum_ima...
6,compute_lrc_energy,"def compute_lrc_energy(system_row, force_field...",None,None
7,compute_real_energies,"def compute_real_energies(system_data, configu...",None,{'apply_minimum_image': 'def apply_minimum_ima...
8,compute_fourier_energies,"def compute_fourier_energies(system_data, conf...",None,None
9,compute_self_energies,"def compute_self_energies(system_data, configu...",None,None


In [ ]:
dict['Code']

'import numpy as np\\nimport pandas as pd\\nfrom scipy.special import erfc, erf\\nimport math\\n\\n###############################################\\n# New or Modified Functions for Triclinic/Monoclinic\\n###############################################\\ndef build_triclinic_box_matrix(a, b, c, alpha_deg, beta_deg, gamma_deg):\\n    \'\'\'\\n    Constructs a 3x3 box matrix and its inverse for a triclinic (or monoclinic) cell.\\n    a, b, c  : side lengths (float)\\n    alpha, beta, gamma : angles in degrees (float)\\n    Returns (box_matrix, inv_box_matrix) as NumPy arrays.\\n    \'\'\'\\n    alpha = math.radians(alpha_deg)\\n    beta = math.radians(beta_deg)\\n    gamma = math.radians(gamma_deg)\\n\\n    Ax = a\\n    Ay = 0.0\\n    Az = 0.0\\n\\n    Bx = b * math.cos(gamma)\\n    By = b * math.sin(gamma)\\n    Bz = 0.0\\n\\n    Cx = c * math.cos(beta)\\n    # Safeguard for the formula inside the sqrt:\\n    # Also note the standard expression for the third vector\\n    Cy = c * ( ( math

In [27]:
print(dict['Code'])

import numpy as np\nimport pandas as pd\nfrom scipy.special import erfc, erf\nimport math\n\n###############################################\n# New or Modified Functions for Triclinic/Monoclinic\n###############################################\ndef build_triclinic_box_matrix(a, b, c, alpha_deg, beta_deg, gamma_deg):\n    '''\n    Constructs a 3x3 box matrix and its inverse for a triclinic (or monoclinic) cell.\n    a, b, c  : side lengths (float)\n    alpha, beta, gamma : angles in degrees (float)\n    Returns (box_matrix, inv_box_matrix) as NumPy arrays.\n    '''\n    alpha = math.radians(alpha_deg)\n    beta = math.radians(beta_deg)\n    gamma = math.radians(gamma_deg)\n\n    Ax = a\n    Ay = 0.0\n    Az = 0.0\n\n    Bx = b * math.cos(gamma)\n    By = b * math.sin(gamma)\n    Bz = 0.0\n\n    Cx = c * math.cos(beta)\n    # Safeguard for the formula inside the sqrt:\n    # Also note the standard expression for the third vector\n    Cy = c * ( ( math.cos(alpha) - math.cos(beta)*math.cos

In [24]:
content.find('Constructs a 3x3 box matrix and its inverse for a triclinic (or monoclinic) cell')

1222

In [32]:
print(repr(dict['Code']))

'import numpy as np\\nimport pandas as pd\\nfrom scipy.special import erfc, erf\\nimport math\\n\\n###############################################\\n# New or Modified Functions for Triclinic/Monoclinic\\n###############################################\\ndef build_triclinic_box_matrix(a, b, c, alpha_deg, beta_deg, gamma_deg):\\n    \'\'\'\\n    Constructs a 3x3 box matrix and its inverse for a triclinic (or monoclinic) cell.\\n    a, b, c  : side lengths (float)\\n    alpha, beta, gamma : angles in degrees (float)\\n    Returns (box_matrix, inv_box_matrix) as NumPy arrays.\\n    \'\'\'\\n    alpha = math.radians(alpha_deg)\\n    beta = math.radians(beta_deg)\\n    gamma = math.radians(gamma_deg)\\n\\n    Ax = a\\n    Ay = 0.0\\n    Az = 0.0\\n\\n    Bx = b * math.cos(gamma)\\n    By = b * math.sin(gamma)\\n    Bz = 0.0\\n\\n    Cx = c * math.cos(beta)\\n    # Safeguard for the formula inside the sqrt:\\n    # Also note the standard expression for the third vector\\n    Cy = c * ( ( math

In [28]:
import textwrap


IndentationError: unexpected indent (<unknown>, line 2)

In [14]:
json.loads(dict['Code'])

JSONDecodeError: Expecting value: line 1 column 1 (char 0)

'import numpy as np\\nimport pandas as pd\\nfrom scipy.special import erfc, erf\\nimport math\\n\\n###############################################\\n# New or Modified Functions for Triclinic/Monoclinic\\n###############################################\\ndef build_triclinic_box_matrix(a, b, c, alpha_deg, beta_deg, gamma_deg):\\n    \'\'\'\\n    Constructs a 3x3 box matrix and its inverse for a triclinic (or monoclinic) cell.\\n    a, b, c  : side lengths (float)\\n    alpha, beta, gamma : angles in degrees (float)\\n    Returns (box_matrix, inv_box_matrix) as NumPy arrays.\\n    \'\'\'\\n    alpha = math.radians(alpha_deg)\\n    beta = math.radians(beta_deg)\\n    gamma = math.radians(gamma_deg)\\n\\n    Ax = a\\n    Ay = 0.0\\n    Az = 0.0\\n\\n    Bx = b * math.cos(gamma)\\n    By = b * math.sin(gamma)\\n    Bz = 0.0\\n\\n    Cx = c * math.cos(beta)\\n    # Safeguard for the formula inside the sqrt:\\n    # Also note the standard expression for the third vector\\n    Cy = c * ( ( math.cos(alpha) - math.cos(beta)*math.cos(gamma) ) / ( math.sin(gamma) + 1e-30 ) )\\n    tmp = 1.0 + 2.0 * math.cos(alpha) * math.cos(beta) * math.cos(gamma) \\\n          - ( math.cos(alpha)**2 + math.cos(beta)**2 + math.cos(gamma)**2 )\\n    if tmp < 0.0:\\n        tmp = 0.0  # numerical safeguard\\n    Cz = c * math.sqrt(tmp) / (math.sin(gamma) + 1e-30)\\n\\n    box_matrix = np.array([[Ax, Bx, Cx],\\n                           [Ay, By, Cy],\\n                           [Az, Bz, Cz]], dtype=float)\\n    inv_box_matrix = np.linalg.inv(box_matrix)\\n    return box_matrix, inv_box_matrix\\n\\ndef minimum_image_distance_triclinic(r_ij, box_matrix, inv_box_matrix):\\n    \'\'\'\\n    Applies the minimum image convention by converting a real-space displacement (r_ij)\\n    to fractional coordinates, wrapping into [-0.5, 0.5) by rounding, and converting back.\\n    \'\'\'\\n    frac = inv_box_matrix.dot(r_ij)\\n    frac -= np.round(frac)\\n    r_mic = box_matrix.dot(frac)\\n    return r_mic\\n\\ndef apply_minimum_image(r_ij, configuration, system_data):\\n    \'\'\'\\n    Wrapper that decides whether to apply a standard cubic minimum-image\\n    or a triclinic/monoclinic approach based on stored box matrix.\\n    \'\'\'\\n    cutoff = system_data[\'cutoff\']\\n    # If box_matrix is present in configuration.attrs, we do the triclinic version\\n    if \'box_matrix\' in configuration.attrs and \'inv_box_matrix\' in configuration.attrs:\\n        box_matrix = configuration.attrs[\'box_matrix\']\\n        inv_box_matrix = configuration.attrs[\'inv_box_matrix\']\\n        r_ij_mic = minimum_image_distance_triclinic(r_ij, box_matrix, inv_box_matrix)\\n        return r_ij_mic\\n    else:\\n        # Fallback to cubic\\n        box_length = system_data[\'box length\']\\n        return r_ij - box_length * np.round(r_ij / box_length)\\n\\n###############################################\\n# Updated extracting_positions for reading cell vectors\\n###############################################\\ndef extracting_positions(input_file):\\n    with open(input_file, \'r\') as file:\\n        lines = file.readlines()\\n\\n    # Parse cell side lengths from the first line\\n    line0 = lines[0].split()\\n    a = float(line0[0])\\n    b = float(line0[1])\\n    c = float(line0[2])\\n\\n    # Parse angles from the second line\\n    line1 = lines[1].split()\\n    alpha_deg = float(line1[0])\\n    beta_deg = float(line1[1])\\n    gamma_deg = float(line1[2])\\n\\n    # Parse number of molecules from the third line\\n    line2 = lines[2].split()\\n    n_mol = int(line2[0])  # not strictly required, but keep for sanity check\\n\\n    # Build the box matrix and the inverse box matrix\\n    box_matrix, inv_box_matrix = build_triclinic_box_matrix(a, b, c, alpha_deg, beta_deg, gamma_deg)\\n\\n    # Remaining lines contain atomic data\\n    data_lines = lines[3:]\\n\\n    data_list = []\\n    for line in data_lines:\\n        stripped_line = line.strip()\\n        parts = stripped_line.split()\\n        if len(parts) >= 5:\\n            try:\\n                x, y, z = map(float, parts[1:4])\\n                atom_type = parts[4]\\n                data_list.append([x, y, z, atom_type])\\n            except ValueError:\\n                continue\\n\\n    columns = [\'X\', \'Y\', \'Z\', \'Atom Type\']\\n    configuration = pd.DataFrame(data_list, columns=columns)\\n\\n    # The standard SPC/E water has 3 atoms per molecule.\\n    configuration.index = range(1, len(configuration) + 1)\\n    configuration[\'Molecule\'] = ((configuration.index - 1) // 3) + 1\\n\\n    # Store cell info in DataFrame attributes\\n    configuration.attrs[\'a\'] = a\\n    configuration.attrs[\'b\'] = b\\n    configuration.attrs[\'c\'] = c\\n    configuration.attrs[\'alpha\'] = alpha_deg\\n    configuration.attrs[\'beta\'] = beta_deg\\n    configuration.attrs[\'gamma\'] = gamma_deg\\n    configuration.attrs[\'box_matrix\'] = box_matrix\\n    configuration.attrs[\'inv_box_matrix\'] = inv_box_matrix\\n\\n    return configuration\\n\\n###############################################\\n# Original dictionaries, data, and creation of DataFrames\\n###############################################\\natom_properties = {\\n    \'O\': {\'type\': \'O\', \'sigma\': 3.165558, \'epsilon\': 78.197431, \'charge\': -0.8476, \'num_particles\': 1},\\n    \'H\': {\'type\': \'H\', \'sigma\': 0.000000, \'epsilon\': 0.000000, \'charge\': 0.4238, \'num_particles\': 2},\\n}\\n\\nfile_paths = [\\n    \'../data/spce-sample-non-cuboid-configurations/spce_triclinic_sample_periodic3.txt\'\\n]\\n\\nNIST_TRICLINIC_SPC_E_Water = {\\n    \'Configuration\': [1, 2, 3, 4],\\n    \'M (number of SPC/E molecules)\': [400, 300, 200, 100],\\n    \'Cell Type\': [\'Triclinic\', \'Monoclinic\', \'Triclinic\', \'Monoclinic\'],\\n    \'Cell Side Lengths [a, b, c] (Å)\': [\'[30 Å, 30 Å, 30 Å]\',\\n                                       \'[27 Å, 30 Å, 36 Å]\',\\n                                       \'[30 Å, 30 Å, 30 Å]\',\\n                                       \'[36 Å, 36 Å, 36 Å]\'],\\n    \'Cell Angles [α, β, γ] (degrees)\': [\'[100°, 95°, 75°]\',\\n                                        \'[90°, 75°, 90°]\',\\n                                        \'[85°, 75°, 80°]\',\\n                                        \'[90°, 60°, 90°]\'],\\n    \'Number of Wave Vectors\': [831, 1068, 838, 1028],\\n    \'Edisp/kB (K)\': [111992.0, 43286.0, 14403.3, 25025.1],\\n    \'ELRC/kB (K)\': [-4109.19, -2105.61, -1027.3, -163.091],\\n    \'Ereal/kB (K)\': [-727219.0, -476902.0, -297129.0, -171462.0],\\n    \'Efourier/kB (K)\': [44677.0, 44409.4, 28897.4, 22337.2],\\n    \'Eself/kB (K)\': [-11582000.0, -8686470.0, -5790980.0, -2895490.0],\\n    \'Eintra/kB (K)\': [11435400.0, 8576520.0, 5717680.0, 2858840.0],\\n    \'Etotal/kB (K)\': [-721254.0, -501259.0, -328153.0, -160912.0]\\n}\\n\\ndef creating_dataframes(file_paths, atom_properties, NIST_SPC_E_Water):\\n    NIST_SPC_E_Water = pd.DataFrame(NIST_SPC_E_Water)\\n    NIST_SPC_E_Water[\'Sum of energies\'] = (\\n        NIST_SPC_E_Water[\'Edisp/kB (K)\'] + NIST_SPC_E_Water[\'ELRC/kB (K)\'] +\\n        NIST_SPC_E_Water[\'Ereal/kB (K)\'] + NIST_SPC_E_Water[\'Efourier/kB (K)\'] +\\n        NIST_SPC_E_Water[\'Eself/kB (K)\'] + NIST_SPC_E_Water[\'Eintra/kB (K)\']\\n    )\\n\\n    force_field = pd.DataFrame(atom_properties).from_dict(atom_properties, orient=\'index\')\\n\\n    system = pd.DataFrame(file_paths, columns=[\'file_paths\'])\\n    system[\'configuration #\'] = (\\n        system[\'file_paths\'].str.extract(r\'(\\\\d+)\', expand=False).fillna(\'0\').astype(int)\\n    )\\n\\n    # (Note: if needed, we could parse the side lengths & angles from NIST_SPC_E_Water.\\n    #  For now, we keep a single \'box length\' for demonstration. You could store a, b, c, alpha, beta, gamma similarly.)\\n\\n    system[["number of particles", "box length"]] = system["configuration #"].apply(\\n        lambda x: pd.Series({\\n            "number of particles": float(\\n                NIST_SPC_E_Water.loc[\\n                    NIST_SPC_E_Water["Configuration"] == x,\\n                    "M (number of SPC/E molecules)"\\n                ].values[0]\\n            ) if x in NIST_SPC_E_Water["Configuration"].values else 0.0,\\n            "box length": 30.0  # For demonstration, override with 30.0.\\n        })\\n    )\\n\\n    system[\'cutoff\'] = 10\\n    system[\'alpha\'] = system.apply(\\n        lambda row: 5.6 / row[\'box length\'] if row[\'box length\'] != 0 else 0.28,\\n        axis=1\\n    )\\n    system[\'kmax\'] = 5\\n    system[\'ε0\'] = float(8.854187817E-12)\\n    system[\'kB\'] = float(1.3806488E-23)\\n\\n    return system, force_field, NIST_SPC_E_Water\\n\\n###############################################\\n# Pairwise Dispersion Function (unchanged except using apply_minimum_image)\\n###############################################\\ndef pair_dispersion_energy(system_data, configuration, force_field):\\n    positions = configuration[[\'X\', \'Y\', \'Z\']].values\\n    atom_types = configuration[\'Atom Type\'].values\\n    cutoff = system_data[\'cutoff\']\\n    num_atoms = len(positions)\\n\\n    total_dispersion_energy = 0.0\\n\\n    for i in range(num_atoms):\\n        for j in range(i + 1, num_atoms):\\n            r_ij = positions[i] - positions[j]\\n            # Use the new minimum-image approach\\n            r_ij = apply_minimum_image(r_ij, configuration, system_data)\\n            distance = np.linalg.norm(r_ij)\\n            if 0 < distance < cutoff:\\n                type_i, type_j = atom_types[i], atom_types[j]\\n                if type_i not in force_field.index or type_j not in force_field.index:\\n                    continue\\n                epsilon_i = force_field.loc[type_i, \'epsilon\']\\n                epsilon_j = force_field.loc[type_j, \'epsilon\']\\n                sigma_i = force_field.loc[type_i, \'sigma\']\\n                sigma_j = force_field.loc[type_j, \'sigma\']\\n                epsilon_ij = np.sqrt(epsilon_i * epsilon_j)\\n                sigma_ij = (sigma_i + sigma_j) / 2.0\\n                s_over_r = sigma_ij / distance\\n                potential_energy = 4 * epsilon_ij * (s_over_r**12 - s_over_r**6)\\n                total_dispersion_energy += potential_energy\\n\\n    return total_dispersion_energy\\n\\n###############################################\\n# Compute LRC energy for LJ (unchanged)\\n###############################################\\ndef compute_lrc_energy(system_row, force_field):\\n    U_lrc_total = 0.0\\n    for atom_type, atom_data in force_field.iterrows():\\n        num_particles = system_row[\'number of particles\'] * atom_data[\'num_particles\']\\n        volume = system_row[\'box length\']**3\\n        rho = num_particles / volume\\n        sigma = atom_data[\'sigma\']\\n        epsilon = atom_data[\'epsilon\']\\n        cutoff = system_row[\'cutoff\']\\n        sigma_by_cutoff_3 = (sigma / cutoff)**3\\n        sigma_by_cutoff_9 = sigma_by_cutoff_3**3\\n        U_lrc_per_particle = (8 / 3) * np.pi * rho * epsilon * sigma**3 * (sigma_by_cutoff_9 / 3 - sigma_by_cutoff_3)\\n        U_lrc_per_particle *= num_particles\\n        U_lrc_total += U_lrc_per_particle\\n    return U_lrc_total\\n\\n###############################################\\n# Real-space Coulomb Energy (replacing min_image with apply_minimum_image)\\n###############################################\\ndef compute_real_energies(system_data, configuration, force_field):\\n    e_charge = 1.602176634e-19\\n    coulomb_factor = 8.9875517923e9 / 1.3806488e-23\\n\\n    alpha = system_data[\'alpha\']\\n    cutoff = system_data[\'cutoff\']\\n\\n    positions = configuration[[\'X\', \'Y\', \'Z\']].values\\n    atom_types = configuration[\'Atom Type\'].values\\n    mol_ids = configuration[\'Molecule\'].values\\n    charges = np.array([force_field.loc[t, \'charge\'] for t in atom_types])\\n\\n    n_atoms = len(positions)\\n    real_energy = 0.0\\n\\n    for j in range(n_atoms - 1):\\n        for l in range(j + 1, n_atoms):\\n            if mol_ids[j] == mol_ids[l]:\\n                continue\\n            r_ij = positions[l] - positions[j]\\n            r_ij = apply_minimum_image(r_ij, configuration, system_data)\\n            r = np.linalg.norm(r_ij)\\n            if r < cutoff and r > 1e-14:\\n                q_j = charges[j] * e_charge\\n                q_l = charges[l] * e_charge\\n                r_m = r * 1e-10\\n                real_energy += coulomb_factor * (q_j * q_l / r_m) * erfc(alpha * r)\\n\\n    return real_energy\\n\\n###############################################\\n# Fourier-space Coulomb Energy (expanded for triclinic by building reciprocal)\\n###############################################\\ndef compute_fourier_energies(system_data, configuration, force_field):\\n    e_charge = 1.602176634e-19\\n    coulomb_factor = 8.9875517923e9 / 1.3806488e-23\\n\\n    alpha = system_data[\'alpha\']\\n    kmax = system_data[\'kmax\']\\n    positions = configuration[[\'X\', \'Y\', \'Z\']].values\\n    atom_types = configuration[\'Atom Type\'].values\\n    charges = np.array([force_field.loc[t, \'charge\'] for t in atom_types])\\n    charges_c = charges * e_charge\\n    positions_m = positions * 1e-10\\n\\n    # Build reciprocal box if available\\n    if \'inv_box_matrix\' in configuration.attrs:\\n        box_matrix = configuration.attrs[\'box_matrix\']\\n        inv_box_matrix = configuration.attrs[\'inv_box_matrix\']\\n        # Reciprocal box is 2*pi * (inv_box_matrix^T)\\n        recip_box = 2.0 * np.pi * inv_box_matrix.T\\n        # Volume in m^3\\n        V_m = abs(np.linalg.det(box_matrix * 1e-10))\\n    else:\\n        # fallback to cubic\\n        L = system_data[\'box length\'] * 1e-10\\n        recip_box = (2.0 * np.pi / L) * np.eye(3)\\n        V_m = (system_data[\'box length\'] * 1e-10)**3\\n\\n    prefactor = coulomb_factor / (2.0 * V_m)\\n    alpha_m = alpha * 1e10\\n\\n    fourier_energy = 0.0\\n\\n    # We\'ll iterate over i, j, k from -kmax..kmax, build the wave vector from the reciprocal box.\\n    # Then we skip if it\'s near zero or outside the cutoff in k-space.\\n\\n    for i in range(-kmax, kmax+1):\\n        for j in range(-kmax, kmax+1):\\n            for k in range(-kmax, kmax+1):\\n                if i == 0 and j == 0 and k == 0:\\n                    continue\\n                # k-vector in reciprocal space\\n                k_vec = i * recip_box[:,0] + j * recip_box[:,1] + k * recip_box[:,2]\\n                k_sq = np.dot(k_vec, k_vec)\\n                if k_sq < 1e-14:\\n                    continue\\n                # optional restriction if we want to approximate a spherical cutoff\\n                # or replicate old logic: if i^2 + j^2 + k^2 >= kmax^2 + 2: skip\\n                if (i*i + j*j + k*k) >= (kmax*kmax + 2):\\n                    continue\\n\\n                real_part = 0.0\\n                imag_part = 0.0\\n                for idx, (xj, yj, zj) in enumerate(positions_m):\\n                    kr = k_vec[0]*xj + k_vec[1]*yj + k_vec[2]*zj\\n                    real_part += charges_c[idx]*math.cos(kr)\\n                    imag_part += charges_c[idx]*math.sin(kr)\\n\\n                sk_sq = real_part*real_part + imag_part*imag_part\\n                exponent = math.exp(-k_sq/(4.0*alpha_m**2))\\n                term = prefactor * (4.0*math.pi / k_sq) * exponent * sk_sq\\n                fourier_energy += term\\n\\n    return fourier_energy\\n\\n###############################################\\n# Self-Energy and Intra-molecular (unchanged except references)\\n###############################################\\ndef compute_self_energies(system_data, configuration, force_field):\\n    e_charge = 1.602176634e-19\\n    coulomb_factor = 8.9875517923e9 / 1.3806488e-23\\n\\n    alpha = system_data[\'alpha\']\\n    atom_types = configuration[\'Atom Type\'].values\\n    charges = np.array([force_field.loc[t,\'charge\'] for t in atom_types])\\n    charges_c = charges * e_charge\\n    alpha_m = alpha*1e10\\n    sum_q2 = np.sum(charges_c**2)\\n    self_energy = - coulomb_factor * (alpha_m / math.sqrt(math.pi)) * sum_q2\\n    return self_energy\\n\\ndef compute_intra_energies(system_data, configuration, force_field):\\n    e_charge = 1.602176634e-19\\n    coulomb_factor = 8.9875517923e9 / 1.3806488e-23\\n\\n    alpha = system_data[\'alpha\']\\n    positions = configuration[[\'X\', \'Y\', \'Z\']].values\\n    atom_types = configuration[\'Atom Type\'].values\\n    charges = np.array([force_field.loc[t,"charge"] for t in atom_types])\\n    mol_ids = configuration[\'Molecule\'].values\\n\\n    intra_energy = 0.0\\n    unique_mols = np.unique(mol_ids)\\n\\n    for m_id in unique_mols:\\n        idxs = np.where(mol_ids == m_id)[0]\\n        n_mol_atoms = len(idxs)\\n        for i in range(n_mol_atoms - 1):\\n            for j in range(i+1, n_mol_atoms):\\n                idx_i = idxs[i]\\n                idx_j = idxs[j]\\n                r_ij = positions[idx_j] - positions[idx_i]\\n                # For intramolecular, we might still want periodic images if the molecule crosses a boundary\\n                r_ij = apply_minimum_image(r_ij, configuration, system_data)\\n                r = np.linalg.norm(r_ij)\\n                if r > 1e-14:\\n                    q_i = charges[idx_i]*e_charge\\n                    q_j = charges[idx_j]*e_charge\\n                    r_m = r*1e-10\\n                    erf_val = erf(alpha*r)\\n                    val = coulomb_factor*(q_i*q_j / r_m)*erf_val\\n                    intra_energy -= val\\n\\n    return intra_energy\\n\\n###############################################\\n# Create DataFrames and Run Calculations\\n###############################################\\nsystem, force_field, NIST_SPC_E_Water = creating_dataframes(file_paths, atom_properties, NIST_TRICLINIC_SPC_E_Water)\\n\\nresults = pd.DataFrame()\\nresults[\'Number of Particles\'] = system[\'number of particles\'].astype(int)\\n\\nresults[\'lrc_Energies\'] = system.apply(\\n    lambda row: compute_lrc_energy(row, force_field), axis=1\\n)\\n\\nresults[\'dispersion_energies\'] = system[\'file_paths\'].apply(\\n    lambda file_path: pair_dispersion_energy(\\n        system[system[\'file_paths\'] == file_path].iloc[0],\\n        extracting_positions(file_path),\\n        force_field\\n    )\\n)\\n\\nresults[\'real_energies\'] = system[\'file_paths\'].apply(\\n    lambda file_path: compute_real_energies(\\n        system[system[\'file_paths\'] == file_path].iloc[0],\\n        extracting_positions(file_path),\\n        force_field\\n    )\\n)\\n\\nresults[\'fourier_energies\'] = system[\'file_paths\'].apply(\\n    lambda file_path: compute_fourier_energies(\\n        system[system[\'file_paths\'] == file_path].iloc[0],\\n        extracting_positions(file_path),\\n        force_field\\n    )\\n)\\n\\nresults[\'self_energies\'] = system[\'file_paths\'].apply(\\n    lambda file_path: compute_self_energies(\\n        system[system[\'file_paths\'] == file_path].iloc[0],\\n        extracting_positions(file_path),\\n        force_field\\n    )\\n)\\n\\nresults[\'intra_energies\'] = system[\'file_paths\'].apply(\\n    lambda file_path: compute_intra_energies(\\n        system[system[\'file_paths\'] == file_path].iloc[0],\\n        extracting_positions(file_path),\\n        force_field\\n    )\\n)\\n\\ndef compare_LJ_coulomb_energy(df1, df2, tolerance=1e-4):\\n    df_merged = df1.merge(df2, left_on=\'Number of Particles\', right_on=\'M (number of SPC/E molecules)\', how=\'left\')\\n\\n    matched_real = matched_fourier = matched_self = matched_intra = 0\\n    matched_dispersion = matched_lrc = 0\\n    not_matched_real = not_matched_fourier = not_matched_self = not_matched_intra = 0\\n    not_matched_dispersion = not_matched_lrc = 0\\n\\n    real_energy_output, fourier_energy_output = [], []\\n    self_energy_output, intra_energy_output = [], []\\n    lrc_energy_output, dispersion_energy_output = [], []\\n\\n    for idx, row in df_merged.iterrows():\\n        real_energy = row[\'real_energies\']\\n        fourier_energy = row[\'fourier_energies\']\\n        self_energy = row[\'self_energies\']\\n        intra_energy = row[\'intra_energies\']\\n        num_molecules = row[\'Number of Particles\']\\n        lrc_energy = row[\'lrc_Energies\']\\n        dispersion_energy = row[\'dispersion_energies\']\\n\\n        if pd.isna(row[\'Ereal/kB (K)\']):\\n            continue\\n\\n        nist_real_energy = float(row[\'Ereal/kB (K)\'])\\n        nist_fourier_energy = float(row[\'Efourier/kB (K)\'])\\n        nist_self_energy = float(row[\'Eself/kB (K)\'])\\n        nist_intra_energy = float(row[\'Eintra/kB (K)\'])\\n        nist_lrc_energy = float(row[\'ELRC/kB (K)\'])\\n        nist_dispersion_energy = float(row[\'Edisp/kB (K)\'])\\n\\n        match_real = np.isclose(real_energy, nist_real_energy, atol=tolerance)\\n        match_fourier = np.isclose(fourier_energy, nist_fourier_energy, atol=tolerance)\\n        match_self = np.isclose(self_energy, nist_self_energy, atol=tolerance)\\n        match_intra = np.isclose(intra_energy, nist_intra_energy, atol=tolerance)\\n        match_dispersion = np.isclose(dispersion_energy, nist_dispersion_energy, atol=tolerance)\\n        match_lrc = np.isclose(lrc_energy, nist_lrc_energy, atol=tolerance)\\n\\n        matched_real += int(match_real)\\n        not_matched_real += int(not match_real)\\n        matched_fourier += int(match_fourier)\\n        not_matched_fourier += int(not match_fourier)\\n        matched_self += int(match_self)\\n        not_matched_self += int(not match_self)\\n        matched_intra += int(match_intra)\\n        not_matched_intra += int(not match_intra)\\n        matched_dispersion += int(match_dispersion)\\n        not_matched_dispersion += int(not match_dispersion)\\n        matched_lrc += int(match_lrc)\\n        not_matched_lrc += int(not match_lrc)\\n\\n        dispersion_energy_output.append(f"Test {idx+1} ({num_molecules} molecules): Computed: {dispersion_energy:.4E}, NIST: {nist_dispersion_energy:.4E}, Match: {match_dispersion}")\\n        lrc_energy_output.append(f"Test {idx+1} ({num_molecules} molecules): Computed: {lrc_energy:.4E}, NIST: {nist_lrc_energy:.4E}, Match: {match_lrc}")\\n        real_energy_output.append(f"Test {idx+1} ({num_molecules} molecules): Computed: {real_energy:.4E}, NIST: {nist_real_energy:.4E}, Match: {match_real}")\\n        fourier_energy_output.append(f"Test {idx+1} ({num_molecules} molecules): Computed: {fourier_energy:.4E}, NIST: {nist_fourier_energy:.4E}, Match: {match_fourier}")\\n        self_energy_output.append(f"Test {idx+1} ({num_molecules} molecules): Computed: {self_energy:.4E}, NIST: {nist_self_energy:.4E}, Match: {match_self}")\\n        intra_energy_output.append(f"Test {idx+1} ({num_molecules} molecules): Computed: {intra_energy:.4E}, NIST: {nist_intra_energy:.4E}, Match: {match_intra}")\\n\\n    print()\\n    print("Lennard-Jones Pair Dispersion Energy Comparison:")\\n    print(*dispersion_energy_output, sep=\'\\n\')\\n    print("Lennard-Jones long-range corrections Energy Comparison:")\\n    print(*lrc_energy_output, sep=\'\\n\')\\n    print("Real Energy Comparison:")\\n    print(*real_energy_output, sep=\'\\n\')\\n    print("Fourier Energy Comparison:")\\n    print(*fourier_energy_output, sep=\'\\n\')\\n    print("Self Energy Comparison:")\\n    print(*self_energy_output, sep=\'\\n\')\\n    print("Intra Energy Comparison:")\\n    print(*intra_energy_output, sep=\'\\n\')\\n    print()\\n    print(f"Count of correct pairwise answers: {matched_dispersion}")\\n    print(f"Count of incorrect pairwise answers: {not_matched_dispersion}")\\n    print(f"Count of correct LRC answers: {matched_lrc}")\\n    print(f"Count of incorrect LRC answers: {not_matched_lrc}")\\n    print(f"Count of correct Real Energy answers: {matched_real}")\\n    print(f"Count of incorrect Real Energy answers: {not_matched_real}")\\n    print(f"Count of correct Fourier Energy answers: {matched_fourier}")\\n    print(f"Count of incorrect Fourier Energy answers: {not_matched_fourier}")\\n    print(f"Count of correct Self Energy answers: {matched_self}")\\n    print(f"Count of incorrect Self Energy answers: {not_matched_self}")\\n    print(f"Count of correct Intra Energy answers: {matched_intra}")\\n    print(f"Count of incorrect Intra Energy answers: {not_matched_intra}")\\n    print()\\n\\n    total_correct = (matched_real + matched_fourier + matched_self +\\n                     matched_intra + matched_dispersion + matched_lrc)\\n    total_incorrect = (not_matched_real + not_matched_fourier + not_matched_self +\\n                       not_matched_intra + not_matched_dispersion + not_matched_lrc)\\n    print(f"Total correct answers: {total_correct}")\\n    print(f"Total incorrect answers: {total_incorrect}")\\n\\n# Example usage:\\n# compare_LJ_coulomb_energy(results, NIST_SPC_E_Water)\\n'